# JOBS

In [1]:
import os
import pandas as pd
import numpy as np
import json
import re
import gensim
import nltk
import requests
import datetime
import math

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup, Tag, NavigableString
from datetime import datetime

## Job Portal - JOBLUM

In [3]:
#Get number of jobs per category (JOBLUM)
def getNumJobs(soup):
    NUM_JOBS = soup.find_all('p',{'class':'no-of-jobs'})
    NUM_JOBS =  re.findall(r'(?s)(?<= of ).*?(?= jobs)',str(NUM_JOBS[1]))
    if NUM_JOBS:
        return int(NUM_JOBS[0])
    else:
        return 0

In [4]:
#Loading the page and parsing HTML data (JOBLUM)
def getSoup(JOBLUM_JOB_URL):
    page = requests.get(JOBLUM_JOB_URL)
    return BeautifulSoup(page.content, 'html.parser')

In [5]:
#Find number of pages (JOBLUM)
def getNumPages(NUM_JOBS):
    return math.ceil(NUM_JOBS/10)

In [6]:
#Get the links of each page (JOBLUM)
def getLinks(Num_Pages, JOBLUM_URLs):
    JOB_LINKS = []
    for j in range(1,NUM_PAGES+1):
        JOB_LINKS.append(JOBLUM_URLs + str(j))
    return JOB_LINKS

In [7]:
#Geting the URLs of each job posting (JOBLUM)
def getJobURL(JOBLUM_SOUP):
    JOBLUM_JOBS_URL = JOBLUM_SOUP.find_all('div',{'class':'mobile-company-logo hidden-md hidden-lg'})
    return re.findall(r'(?s)(?<= href=").*?(?=" )',str(JOBLUM_JOBS_URL))    

In [8]:
#Geting the Job Title (JOBLUM)
def getJobTitle(JOB_SOUP):
    JOB_TITLE = JOB_SOUP.find('h1',{'class':'job-title'})
    if JOB_TITLE is not None:
        return JOB_TITLE.text.strip()
    else:
        return "Not Specified"

In [9]:
#Geting the Job Company (JOBLUM)
def getJobCompany(JOB_SOUP):
    JOB_COMPANY = JOB_SOUP.find('a',{'class':'job-company-name'})
    if JOB_COMPANY is not None:
        return JOB_COMPANY.text.strip()
    else:
        return "Not Specified"

In [10]:
#Geting the Job Location (JOBLUM)
def getJobLocation(JOB_SOUP):
    JOB_LOCATION = JOB_SOUP.find('span',{'class':'job-company-location'})
    if JOB_LOCATION is not None:
        return JOB_LOCATION.text.strip()
    else:
        return "Not Specified"

In [11]:
#Geting Date Posted (JOBLUM)
def getDatePosted(JOB_SOUP):
    try:
        JOB_DATE = JOB_SOUP.find('time',{'itemprop':'datePosted'}).text.strip()
        return datetime.strptime(JOB_DATE, '%B %d, %Y').date()
    except: 
        return None

In [19]:
#Geting Job Description (JOBLUM)
def getJobDescription(JOB_SOUP):
    try:
        JOB_INFO = JOB_SOUP.find('span',{'itemprop':'description'})
        JOB_INFO_ARRAY = []
        try:
            for n in range(len(JOB_INFO.contents[1].contents[0])):
                if(isinstance(JOB_INFO.contents[1].contents[0].contents[n],NavigableString)):
                    JOB_INFO_ARRAY.append(JOB_INFO.contents[1].contents[0].contents[n])
                else:
                    for s in range(len(JOB_INFO.contents[1].contents[0].contents[n])):
                        if(isinstance(JOB_INFO.contents[1].contents[0].contents[n].contents[s], Tag)):
                            JOB_INFO_ARRAY.append(JOB_INFO.contents[1].contents[0].contents[n].contents[s].text)
                        else:
                            JOB_INFO_ARRAY.append(JOB_INFO.contents[1].contents[0].contents[n].contents[s])
            JOB_DESCRIPTION = ' '.join(JOB_INFO_ARRAY)
            JOB_DESCRIPTION = JOB_DESCRIPTION.replace("\xa0"," ")
            JOB_DESCRIPTION = re.sub(' +', ' ', JOB_DESCRIPTION) 
            return JOB_DESCRIPTION
        except:
            return None
    except:
        return None

In [13]:
#Geting Job Salary (JOBLUM)
def getJobSalary(JOB_SOUP):
    JOB_INFO = JOB_SOUP.find('span',{'itemprop':'description'})
    JOB_SALARY_FINDER = JOB_SOUP.find('p',{'class':'job-subinfo'})
    JOB_SALARY_ARRAY = re.findall('[0-9]+,[0-9]+',str(JOB_SALARY_FINDER))
    JOB_SALARY = ""
    JOB_SALARY_MIN = "Not Specified"
    JOB_SALARY_MAX = "Not Specified"
    JOB_SALARY_CHECKER = re.findall('(?i)Salary',str(JOB_SALARY_FINDER))
    if not JOB_SALARY_CHECKER:
        JOB_SALARY_CHECKER = re.findall('(?i)Salary',str(JOB_INFO))
    if not JOB_SALARY_CHECKER:
        JOB_SALARY_CHECKER = re.findall('(?i)PHP',str(JOB_INFO))
    if not JOB_SALARY_CHECKER:
        JOB_SALARY_CHECKER = re.findall('(?i)pesos',str(JOB_INFO))
    if not JOB_SALARY_ARRAY:
        JOB_SALARY_ARRAY = re.findall('[0-9]+,[0-9]+',str(JOB_INFO))
    if JOB_SALARY_CHECKER:
        if (len(JOB_SALARY_ARRAY)==2):
            if (int(JOB_SALARY_ARRAY[1].replace(",","")) > int(JOB_SALARY_ARRAY[0].replace(",",""))):
                if (int(JOB_SALARY_ARRAY[1].replace(",","")) > 1000):
                    JOB_SALARY = '-'.join(JOB_SALARY_ARRAY)
                    JOB_SALARY_MIN = JOB_SALARY_ARRAY[0]
                    JOB_SALARY_MAX = JOB_SALARY_ARRAY[1]                
        elif (len(JOB_SALARY_ARRAY)==1):
            JOB_SALARY = JOB_SALARY_ARRAY[0]
            JOB_SALARY_MIN = JOB_SALARY_ARRAY[0]
            JOB_SALARY_MAX = JOB_SALARY_ARRAY[0]
    return JOB_SALARY, JOB_SALARY_MIN, JOB_SALARY_MAX

In [14]:
#Geting Job Type (JOBLUM)
def getJobType(JOB_SOUP):
    JOB_INFO = JOB_SOUP.find('div',{'class':'col-sm-8 job-main-col'})
    if (len(re.findall('(?i)Full-time',str(JOB_INFO)))>0):
        return "Full Time"
    elif (len(re.findall('(?i)Fulltime',str(JOB_INFO)))>0):
        return "Full Time"
    elif (len(re.findall('(?i)Full time',str(JOB_INFO)))>0):
        return "Full Time"
    elif (len(re.findall('(?i)Part-time',str(JOB_INFO)))>0):
        return "Part Time"
    elif (len(re.findall('(?i)Parttime',str(JOB_INFO)))>0):
        return "Part Time"
    elif (len(re.findall('(?i)Parttime',str(JOB_INFO)))>0):
        return "Part Time"
    elif (len(re.findall('(?i)Temporary',str(JOB_INFO)))>0):
        return "Contract"
    elif (len(re.findall('(?i)Contract',str(JOB_INFO)))>0):
        return "Contract"
    elif (len(re.findall('(?i)Permanent',str(JOB_INFO)))>0):
        return "Full Time"
    elif (len(re.findall('(?i)Regular',str(JOB_INFO)))>0):
        return "Full Time"
    elif (len(re.findall('(?i)OJT',str(JOB_INFO)))>0):
        return "OJT"
    elif (len(re.findall('(?i)Intern',str(JOB_INFO)))>0):
        return "OJT"
    elif (len(re.findall('(?i)Freelance',str(JOB_INFO)))>0):
        return "Freelance"
    elif (len(re.findall('(?i)Project Base',str(JOB_INFO)))>0):
        return "Project Base"
    else:
        return "Not Specified"

In [15]:
#Scraping data of each job posting (JOBLUM)
def scrapeJob(URL, JOB_TITLE_LIST, JOB_COMPANY_LIST, JOB_DATE_LIST, 
              JOB_STATUS_LIST, JOB_LOCATION_LIST, JOB_SALARY_LIST, 
              JOB_SALARY_MIN_LIST, JOB_SALARY_MAX_LIST, JOB_DESCRIPTION_LIST):
    
    for m in range(len(URL)):
        JOB_SOUP = getSoup('https://ph.joblum.com' + URL[m])
        JOB_TITLE = getJobTitle(JOB_SOUP)
        JOB_COMPANY = getJobCompany(JOB_SOUP)
        JOB_LOCATION = getJobLocation(JOB_SOUP)
        JOB_DATEPOSTED = getDatePosted(JOB_SOUP)
        JOB_DESCRIPTION = getJobDescription(JOB_SOUP)
        JOB_SALARY, JOB_SALARY_MIN, JOB_SALARY_MAX = getJobSalary(JOB_SOUP)
        JOB_STATUS = getJobType(JOB_SOUP)
        JOB_TITLE_LIST.append(JOB_TITLE)
        JOB_COMPANY_LIST.append(JOB_COMPANY)
        JOB_DATE_LIST.append(JOB_DATEPOSTED)
        JOB_STATUS_LIST.append(JOB_STATUS)
        JOB_LOCATION_LIST.append(JOB_LOCATION)
        JOB_SALARY_LIST.append(JOB_SALARY)
        JOB_SALARY_MIN_LIST.append(JOB_SALARY_MIN)
        JOB_SALARY_MAX_LIST.append(JOB_SALARY_MAX)
        JOB_DESCRIPTION_LIST.append(JOB_DESCRIPTION)

In [167]:
#Scraping data of Actuarial/Statistics (JOBLUM) - FIRST HALF

STAT_TITLE_FIRST = []
STAT_COMPANY_FIRST = []
STAT_DATE_FIRST = []
STAT_LOCATION_FIRST = []
STAT_STATUS_FIRST = []
STAT_SALARY_FIRST = []
STAT_SALARY_MIN_FIRST = []
STAT_SALARY_MAX_FIRST = []
STAT_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-actuarial-statistics?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, STAT_TITLE_FIRST, STAT_COMPANY_FIRST, STAT_DATE_FIRST, 
              STAT_STATUS_FIRST, STAT_LOCATION_FIRST, STAT_SALARY_FIRST, 
              STAT_SALARY_MIN_FIRST, STAT_SALARY_MAX_FIRST, STAT_DESCRIPTION_FIRST)

In [168]:
#Scraping data of Actuarial/Statistics (JOBLUM) - SECOND HALF

STAT_TITLE_SECOND = []
STAT_COMPANY_SECOND = []
STAT_DATE_SECOND = []
STAT_LOCATION_SECOND = []
STAT_STATUS_SECOND = []
STAT_SALARY_SECOND = []
STAT_SALARY_MIN_SECOND = []
STAT_SALARY_MAX_SECOND = []
STAT_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, STAT_TITLE_SECOND, STAT_COMPANY_SECOND, STAT_DATE_SECOND, 
              STAT_STATUS_SECOND, STAT_LOCATION_SECOND, STAT_SALARY_SECOND, 
              STAT_SALARY_MIN_SECOND, STAT_SALARY_MAX_SECOND, STAT_DESCRIPTION_FIRST)

In [169]:
#Joining Scrape data of Actuarial/Statistics (JOBLUM) 

STAT_TITLE_LIST = np.concatenate((STAT_TITLE_FIRST, STAT_TITLE_SECOND))
STAT_COMPANY_LIST = np.concatenate((STAT_COMPANY_FIRST, STAT_COMPANY_SECOND))
STAT_DATE_LIST = np.concatenate((STAT_DATE_FIRST, STAT_DATE_SECOND))
STAT_LOCATION_LIST = np.concatenate((STAT_LOCATION_FIRST, STAT_LOCATION_SECOND))
STAT_STATUS_LIST = np.concatenate((STAT_STATUS_FIRST, STAT_STATUS_SECOND))
STAT_SALARY_LIST = np.concatenate((STAT_SALARY_FIRST, STAT_SALARY_SECOND))
STAT_SALARY_MIN_LIST = np.concatenate((STAT_SALARY_MIN_FIRST, STAT_SALARY_MIN_SECOND))
STAT_SALARY_MAX_LIST = np.concatenate((STAT_SALARY_MAX_FIRST, STAT_SALARY_MAX_SECOND))
STAT_DESCRIPTION_LIST = np.concatenate((STAT_DESCRIPTION_FIRST, STAT_DESCRIPTION_SECOND))

In [170]:
#Creating Data Frame for Actuarial/Statistics (JOBLUM) 
STAT={'Website': "Joblum",
      'Job Title': STAT_TITLE_LIST, 
      'Category': "Actuarial/Statistics", 
      'Company': STAT_COMPANY_LIST, 
      'Date Posted': STAT_DATE_LIST, 
      'Location': STAT_LOCATION_LIST, 
      'Status': STAT_STATUS_LIST, 
      'Salary': STAT_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': STAT_DESCRIPTION_LIST,
      'Min Salary': STAT_SALARY_MIN_LIST,
      'Max Salary': STAT_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Mathematics"}
STAT_df = pd.DataFrame(data=STAT)
STAT_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,With Signing Bonus* | Data Visualization Senio...,Actuarial/Statistics,Accenture,2021-05-29,"Central Luzon, National Capital Reg, Calabarzo...",Not Specified,,Not Specified / In Description,Not Specified / In Description,Go through a faster and more convenient recrui...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
1,Joblum,Statistician 1,Actuarial/Statistics,Philippine National Police Region X - Government,2021-05-29,Northern Mindanao,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 11,...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
2,Joblum,Statistician I,Actuarial/Statistics,"Municipal Government of Surallah, South Cotaba...",2021-05-29,Soccsksargen,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 11,...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
3,Joblum,Risk Underwriter,Actuarial/Statistics,Lightspeed,2021-05-28,Pasig City,Full Time,,Not Specified / In Description,Not Specified / In Description,Perks and Benefits Participate in the Lightspe...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
4,Joblum,Statistician III,Actuarial/Statistics,Civil Service Commission - Government,2021-05-28,National Capital Reg,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 31, ...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,Joblum,Statistician Aide (010-0465),Actuarial/Statistics,Provincial Government of Albay - Government,2021-05-01,Bicol Region,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 7, 2...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
245,Joblum,ELECTION ASSISTANT II,Actuarial/Statistics,Commission on Elections REGION II - Government,2021-04-30,Cagayan Valley,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 15, ...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
246,Joblum,ELECTION ASSISTANT I,Actuarial/Statistics,Commission on Elections REGION II - Government,2021-04-30,Cagayan Valley,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 15, ...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
247,Joblum,Reporting Analyst,Actuarial/Statistics,Alta Resources (Philippines) Corporation,2021-04-29,National Capital Reg,Contract,,Not Specified / In Description,Not Specified / In Description,The Reporting Analyst will be responsible for ...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics


In [171]:
STAT_df.to_csv ('JOBLUM-STAT.csv', index = False)

In [127]:
#Scraping data of Agriculture (JOBLUM) - FIRST HALF

AGRI_TITLE_FIRST = []
AGRI_COMPANY_FIRST = []
AGRI_DATE_FIRST = []
AGRI_LOCATION_FIRST = []
AGRI_STATUS_FIRST = []
AGRI_SALARY_FIRST = []
AGRI_SALARY_MIN_FIRST = []
AGRI_SALARY_MAX_FIRST = []
AGRI_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-agriculture?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, AGRI_TITLE_FIRST, AGRI_COMPANY_FIRST, AGRI_DATE_FIRST, 
              AGRI_STATUS_FIRST, AGRI_LOCATION_FIRST, AGRI_SALARY_FIRST, 
              AGRI_SALARY_MIN_FIRST, AGRI_SALARY_MAX_FIRST, AGRI_DESCRIPTION_FIRST)

In [128]:
#Scraping data of Agriculture (JOBLUM) - SECOND HALF

AGRI_TITLE_SECOND = []
AGRI_COMPANY_SECOND = []
AGRI_DATE_SECOND = []
AGRI_LOCATION_SECOND = []
AGRI_STATUS_SECOND = []
AGRI_SALARY_SECOND = []
AGRI_SALARY_MIN_SECOND = []
AGRI_SALARY_MAX_SECOND = []
AGRI_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, AGRI_TITLE_SECOND, AGRI_COMPANY_SECOND, AGRI_DATE_SECOND, 
              AGRI_STATUS_SECOND, AGRI_LOCATION_SECOND, AGRI_SALARY_SECOND, 
              AGRI_SALARY_MIN_SECOND, AGRI_SALARY_MAX_SECOND, AGRI_DESCRIPTION_FIRST)

In [129]:
#Joining Scrape data of Agriculture (JOBLUM) 

AGRI_TITLE_LIST = np.concatenate((AGRI_TITLE_FIRST, AGRI_TITLE_SECOND))
AGRI_COMPANY_LIST = np.concatenate((AGRI_COMPANY_FIRST, AGRI_COMPANY_SECOND))
AGRI_DATE_LIST = np.concatenate((AGRI_DATE_FIRST, AGRI_DATE_SECOND))
AGRI_LOCATION_LIST = np.concatenate((AGRI_LOCATION_FIRST, AGRI_LOCATION_SECOND))
AGRI_STATUS_LIST = np.concatenate((AGRI_STATUS_FIRST, AGRI_STATUS_SECOND))
AGRI_SALARY_LIST = np.concatenate((AGRI_SALARY_FIRST, AGRI_SALARY_SECOND))
AGRI_SALARY_MIN_LIST = np.concatenate((AGRI_SALARY_MIN_FIRST, AGRI_SALARY_MIN_SECOND))
AGRI_SALARY_MAX_LIST = np.concatenate((AGRI_SALARY_MAX_FIRST, AGRI_SALARY_MAX_SECOND))
AGRI_DESCRIPTION_LIST = np.concatenate((AGRI_DESCRIPTION_FIRST, AGRI_DESCRIPTION_SECOND))

In [130]:
#Creating Data Frame for Agriculture (JOBLUM) 
AGRI={'Website': "Joblum",
      'Job Title': AGRI_TITLE_LIST, 
      'Category': "Agriculture", 
      'Company': AGRI_COMPANY_LIST, 
      'Date Posted': AGRI_DATE_LIST, 
      'Location': AGRI_LOCATION_LIST, 
      'Status': AGRI_STATUS_LIST, 
      'Salary': AGRI_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': AGRI_DESCRIPTION_LIST,
      'Min Salary': AGRI_SALARY_MIN_LIST,
      'Max Salary': AGRI_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Agriculture"}
AGRI_df = pd.DataFrame(data=AGRI)
AGRI_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Technical Sales Representative (Mindanao),Agriculture,La Filipina Uy Gongco Group of Companies,2021-05-29,Bukidnon,Contract,,Not Specified / In Description,Not Specified / In Description,QUALIFICATIONS Candidate must possess at least...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
1,Joblum,Farm Worker II,Agriculture,"Municipal Government of Agoo, La Union",2021-05-29,Ilocos Region,Not Specified,"12,960",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 4, ...","12,960","12,960",Not Specified,Not Specified,Mathematics
2,Joblum,AGRICULTURAL TECHNOLOGIST,Agriculture,"Municipal Government of Magdiwang, Romblon",2021-05-29,Calabarzon & Mimaropa,Not Specified,"14,153",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 4, ...","14,153","14,153",Not Specified,Not Specified,Mathematics
3,Joblum,Farm Worker - I,Agriculture,"Municipal Government of Santa Teresita, Cagayan",2021-05-29,Cagayan Valley,Not Specified,"9,207",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 7, ...","9,207","9,207",Not Specified,Not Specified,Mathematics
4,Joblum,Senior Aquaculturist,Agriculture,"Municipal Government of Parang, Maguindanao",2021-05-29,Armm,Not Specified,"37,943",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 1, ...","37,943","37,943",Not Specified,Not Specified,Mathematics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,Joblum,Agriculturist I,Agriculture,"City Government of Surigao, Surigao del Norte",2021-04-30,Caraga,Not Specified,"21,489",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 9, 2...","21,489","21,489",Not Specified,Not Specified,Mathematics
693,Joblum,Agricultural Technologist,Agriculture,"City Government of Surigao, Surigao del Norte",2021-04-30,Caraga,Not Specified,"19,085",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 9, 2...","19,085","19,085",Not Specified,Not Specified,Mathematics
694,Joblum,Greenhouse Farm Worker,Agriculture,Czark Mak Group of Companies,2021-04-30,Cavite,Not Specified,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Bachelor's/Col...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
695,Joblum,Municipal Agricultural Officer,Agriculture,"Municipal Government of Alburquerque, Bohol - ...",2021-04-30,Central Visayas,Not Specified,"36,892",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 7, 2...","36,892","36,892",Not Specified,Not Specified,Mathematics


In [132]:
AGRI_df.to_csv ('JOBLUM-AGRI.csv', index = False)

In [195]:
#Scraping data of Architect/Interior Design (JOBLUM) - FIRST HALF

ARCHI_TITLE_FIRST = []
ARCHI_COMPANY_FIRST = []
ARCHI_DATE_FIRST = []
ARCHI_LOCATION_FIRST = []
ARCHI_STATUS_FIRST = []
ARCHI_SALARY_FIRST = []
ARCHI_SALARY_MIN_FIRST = []
ARCHI_SALARY_MAX_FIRST = []
ARCHI_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-architect-interior-design?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ARCHI_TITLE_FIRST, ARCHI_COMPANY_FIRST, ARCHI_DATE_FIRST, 
              ARCHI_STATUS_FIRST, ARCHI_LOCATION_FIRST, ARCHI_SALARY_FIRST, 
              ARCHI_SALARY_MIN_FIRST, ARCHI_SALARY_MAX_FIRST, ARCHI_DESCRIPTION_FIRST)

In [196]:
#Scraping data of Architect/Interior Design (JOBLUM) - SECOND HALF

ARCHI_TITLE_SECOND = []
ARCHI_COMPANY_SECOND = []
ARCHI_DATE_SECOND = []
ARCHI_LOCATION_SECOND = []
ARCHI_STATUS_SECOND = []
ARCHI_SALARY_SECOND = []
ARCHI_SALARY_MIN_SECOND = []
ARCHI_SALARY_MAX_SECOND = []
ARCHI_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ARCHI_TITLE_SECOND, ARCHI_COMPANY_SECOND, ARCHI_DATE_SECOND, 
              ARCHI_STATUS_SECOND, ARCHI_LOCATION_SECOND, ARCHI_SALARY_SECOND, 
              ARCHI_SALARY_MIN_SECOND, ARCHI_SALARY_MAX_SECOND, ARCHI_DESCRIPTION_FIRST)

In [197]:
#Joining Scrape data of Architect/Interior Design (JOBLUM) 

ARCHI_TITLE_LIST = np.concatenate((ARCHI_TITLE_FIRST, ARCHI_TITLE_SECOND))
ARCHI_COMPANY_LIST = np.concatenate((ARCHI_COMPANY_FIRST, ARCHI_COMPANY_SECOND))
ARCHI_DATE_LIST = np.concatenate((ARCHI_DATE_FIRST, ARCHI_DATE_SECOND))
ARCHI_LOCATION_LIST = np.concatenate((ARCHI_LOCATION_FIRST, ARCHI_LOCATION_SECOND))
ARCHI_STATUS_LIST = np.concatenate((ARCHI_STATUS_FIRST, ARCHI_STATUS_SECOND))
ARCHI_SALARY_LIST = np.concatenate((ARCHI_SALARY_FIRST, ARCHI_SALARY_SECOND))
ARCHI_SALARY_MIN_LIST = np.concatenate((ARCHI_SALARY_MIN_FIRST, ARCHI_SALARY_MIN_SECOND))
ARCHI_SALARY_MAX_LIST = np.concatenate((ARCHI_SALARY_MAX_FIRST, ARCHI_SALARY_MAX_SECOND))
ARCHI_DESCRIPTION_LIST = np.concatenate((ARCHI_DESCRIPTION_FIRST, ARCHI_DESCRIPTION_SECOND))

In [198]:
#Creating Data Frame for Architect/Interior Design (JOBLUM) 
ARCHI={'Website': "Joblum",
      'Job Title': ARCHI_TITLE_LIST, 
      'Category': "Architect/Interior Design", 
      'Company': ARCHI_COMPANY_LIST, 
      'Date Posted': ARCHI_DATE_LIST, 
      'Location': ARCHI_LOCATION_LIST, 
      'Status': ARCHI_STATUS_LIST, 
      'Salary': ARCHI_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ARCHI_DESCRIPTION_LIST,
      'Min Salary': ARCHI_SALARY_MIN_LIST,
      'Max Salary': ARCHI_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
ARCHI_df = pd.DataFrame(data=ARCHI)
ARCHI_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,ARCHITECT,Architect/Interior Design,Puyat Sports Inc.,2021-05-29,Makati City,Full Time,,Not Specified / In Description,Not Specified / In Description,Qualifications: BS Architecture graduate from ...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
1,Joblum,Interior Designer,Architect/Interior Design,DMCI Homes,2021-05-29,National Capital Reg,Contract,,Not Specified / In Description,Not Specified / In Description,<>QUALIFICATIONS Graduate of Bachelor's/Colleg...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
2,Joblum,SALES DESIGNER (Luxury Home Furnishing Brands),Architect/Interior Design,Focus Global Inc.,2021-05-29,National Capital Reg,Contract,,Not Specified / In Description,Not Specified / In Description,The Sales Designer will join Focus Global’s te...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
3,Joblum,Product Designer - Furniture,Architect/Interior Design,ELECTROPARTS DEVELOPMENT CORPORATION,2021-05-29,San Juan City,OJT,"18,000-30,000",Not Specified / In Description,Not Specified / In Description,Lead representative for communications with bo...,"18,000","30,000",Not Specified,Not Specified,Engineering
4,Joblum,Archtectural Draftsman,Architect/Interior Design,TRANSFORMATIONAL HUB CORPORATION,2021-05-29,Cebu (Cebu City),OJT,"25,000-30,000",Not Specified / In Description,Not Specified / In Description,Experience in BIM modelling / workflow - parti...,"25,000","30,000",Not Specified,Not Specified,Engineering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Joblum,CAD Operator,Architect/Interior Design,Not Specified,2021-04-30,Misamis Oriental (CDO),Not Specified,,Not Specified / In Description,Not Specified / In Description,Cad Operators are responsible for the: Prepara...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
348,Joblum,Architect,Architect/Interior Design,ARBOR VITAE CONSTRUCTION GROUP,2021-04-30,Cebu (Cebu City),Full Time,"30,000-42,000",Not Specified / In Description,Not Specified / In Description,We are looking for interior designers and arch...,"30,000","42,000",Not Specified,Not Specified,Engineering
349,Joblum,Junior Project Architect,Architect/Interior Design,"DMC Urban Property Developers, Inc.",2021-04-30,Makati City,Not Specified,"19,000-21,000",Not Specified / In Description,Not Specified / In Description,Graduate of B.S Architecture At least 2-3 year...,"19,000","21,000",Not Specified,Not Specified,Engineering
350,Joblum,Ht doctor test saved 1 [Don't apply],Architect/Interior Design,Maxicare Healthcare Corporation,2021-04-29,Philippines,Part Time,"7,000-9,800",Not Specified / In Description,Not Specified / In Description,Included some rr question test save in ph test...,"7,000","9,800",Not Specified,Not Specified,Engineering


In [199]:
ARCHI_df.to_csv ('JOBLUM-ARCHI.csv', index = False)

In [138]:
#Scraping data of Aviation (JOBLUM) - FIRST HALF

AVI_TITLE_FIRST = []
AVI_COMPANY_FIRST = []
AVI_DATE_FIRST = []
AVI_LOCATION_FIRST = []
AVI_STATUS_FIRST = []
AVI_SALARY_FIRST = []
AVI_SALARY_MIN_FIRST = []
AVI_SALARY_MAX_FIRST = []
AVI_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-aviation?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, AVI_TITLE_FIRST, AVI_COMPANY_FIRST, AVI_DATE_FIRST, 
              AVI_STATUS_FIRST, AVI_LOCATION_FIRST, AVI_SALARY_FIRST, 
              AVI_SALARY_MIN_FIRST, AVI_SALARY_MAX_FIRST, AVI_DESCRIPTION_FIRST)

In [139]:
#Scraping data of Aviation (JOBLUM) - SECOND HALF

AVI_TITLE_SECOND = []
AVI_COMPANY_SECOND = []
AVI_DATE_SECOND = []
AVI_LOCATION_SECOND = []
AVI_STATUS_SECOND = []
AVI_SALARY_SECOND = []
AVI_SALARY_MIN_SECOND = []
AVI_SALARY_MAX_SECOND = []
AVI_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, AVI_TITLE_SECOND, AVI_COMPANY_SECOND, AVI_DATE_SECOND, 
              AVI_STATUS_SECOND, AVI_LOCATION_SECOND, AVI_SALARY_SECOND, 
              AVI_SALARY_MIN_SECOND, AVI_SALARY_MAX_SECOND, AVI_DESCRIPTION_FIRST)

In [140]:
#Joining Scrape data of Aviation (JOBLUM) 

AVI_TITLE_LIST = np.concatenate((AVI_TITLE_FIRST, AVI_TITLE_SECOND))
AVI_COMPANY_LIST = np.concatenate((AVI_COMPANY_FIRST, AVI_COMPANY_SECOND))
AVI_DATE_LIST = np.concatenate((AVI_DATE_FIRST, AVI_DATE_SECOND))
AVI_LOCATION_LIST = np.concatenate((AVI_LOCATION_FIRST, AVI_LOCATION_SECOND))
AVI_STATUS_LIST = np.concatenate((AVI_STATUS_FIRST, AVI_STATUS_SECOND))
AVI_SALARY_LIST = np.concatenate((AVI_SALARY_FIRST, AVI_SALARY_SECOND))
AVI_SALARY_MIN_LIST = np.concatenate((AVI_SALARY_MIN_FIRST, AVI_SALARY_MIN_SECOND))
AVI_SALARY_MAX_LIST = np.concatenate((AVI_SALARY_MAX_FIRST, AVI_SALARY_MAX_SECOND))
AVI_DESCRIPTION_LIST = np.concatenate((AVI_DESCRIPTION_FIRST, AVI_DESCRIPTION_SECOND))

In [141]:
#Creating Data Frame for Aviation (JOBLUM) 
AVI={'Website': "Joblum",
      'Job Title': AVI_TITLE_LIST, 
      'Category': "Aviation", 
      'Company': AVI_COMPANY_LIST, 
      'Date Posted': AVI_DATE_LIST, 
      'Location': AVI_LOCATION_LIST, 
      'Status': AVI_STATUS_LIST, 
      'Salary': AVI_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': AVI_DESCRIPTION_LIST,
      'Min Salary': AVI_SALARY_MIN_LIST,
      'Max Salary': AVI_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
AVI_df = pd.DataFrame(data=AVI)
AVI_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Aircraft Maintenance Support Analyst,Aviation,"Traxxall Technologies, Inc. - Philippine Branch",2021-05-27,Pampanga (Angeles City),Not Specified,,Not Specified / In Description,Not Specified / In Description,Role and Responsibilities In the role of Aircr...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
1,Joblum,Aircraft Technician,Aviation,ARAM ENTERPRISES INC,2021-05-27,Saudi Arabia,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Professional...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
2,Joblum,Compensation and Benefits Supervisor,Aviation,Asian Aerospace,2021-05-25,Pasay City,OJT,,Not Specified / In Description,Not Specified / In Description,Job Summary The Compensation and Benefits Supe...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
3,Joblum,Storekeeper,Aviation,Airbus Helicopters Philippines Inc.,2021-05-23,Pasay City,Full Time,,Not Specified / In Description,Not Specified / In Description,Job Qualifications BS in Aircraft Maintenance ...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
4,Joblum,System Data Analyst,Aviation,"Global Virtuoso, Inc.",2021-05-17,National Capital Reg,OJT,,Not Specified / In Description,Not Specified / In Description,Global Virtuoso is actively looking for System...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
5,Joblum,Aircraft Mechanic II (18-1998),Aviation,PHILIPPINE AIR FORCE - Government,2021-05-14,National Capital Reg,OJT,"18,251",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 17, ...","18,251","18,251",Not Specified,Not Specified,Mathematics
6,Joblum,Aircraft Mechanic II (34-1998),Aviation,PHILIPPINE AIR FORCE - Government,2021-05-14,National Capital Reg,OJT,"18,251",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 17, ...","18,251","18,251",Not Specified,Not Specified,Mathematics
7,Joblum,Aircraft Mechanic II (27-1998),Aviation,PHILIPPINE AIR FORCE - Government,2021-05-14,National Capital Reg,OJT,"18,251",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 17, ...","18,251","18,251",Not Specified,Not Specified,Mathematics
8,Joblum,Aircraft Maintenance Technologist III,Aviation,PHILIPPINE AIR FORCE - Government,2021-05-14,National Capital Reg,OJT,"43,681",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 17, ...","43,681","43,681",Not Specified,Not Specified,Mathematics
9,Joblum,Aircraft Maintenance Technologist III,Aviation,PHILIPPINE AIR FORCE - Government,2021-05-12,National Capital Reg,OJT,"43,681",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 17, ...","43,681","43,681",Not Specified,Not Specified,Mathematics


In [142]:
AVI_df.to_csv ('JOBLUM-AVI.csv', index = False)

In [ ]:
#Scraping data of Biomedical (JOBLUM) - FIRST HALF

BIOMED_TITLE_FIRST = []
BIOMED_COMPANY_FIRST = []
BIOMED_DATE_FIRST = []
BIOMED_LOCATION_FIRST = []
BIOMED_STATUS_FIRST = []
BIOMED_SALARY_FIRST = []
BIOMED_SALARY_MIN_FIRST = []
BIOMED_SALARY_MAX_FIRST = []
BIOMED_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-biomedical?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, BIOMED_TITLE_FIRST, BIOMED_COMPANY_FIRST, BIOMED_DATE_FIRST, 
              BIOMED_STATUS_FIRST, BIOMED_LOCATION_FIRST, BIOMED_SALARY_FIRST, 
              BIOMED_SALARY_MIN_FIRST, BIOMED_SALARY_MAX_FIRST, BIOMED_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Biomedical (JOBLUM) - SECOND HALF

BIOMED_TITLE_SECOND = []
BIOMED_COMPANY_SECOND = []
BIOMED_DATE_SECOND = []
BIOMED_LOCATION_SECOND = []
BIOMED_STATUS_SECOND = []
BIOMED_SALARY_SECOND = []
BIOMED_SALARY_MIN_SECOND = []
BIOMED_SALARY_MAX_SECOND = []
BIOMED_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, BIOMED_TITLE_SECOND, BIOMED_COMPANY_SECOND, BIOMED_DATE_SECOND, 
              BIOMED_STATUS_SECOND, BIOMED_LOCATION_SECOND, BIOMED_SALARY_SECOND, 
              BIOMED_SALARY_MIN_SECOND, BIOMED_SALARY_MAX_SECOND, BIOMED_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Biomedical (JOBLUM) 

BIOMED_TITLE_LIST = np.concatenate((BIOMED_TITLE_FIRST, BIOMED_TITLE_SECOND))
BIOMED_COMPANY_LIST = np.concatenate((BIOMED_COMPANY_FIRST, BIOMED_COMPANY_SECOND))
BIOMED_DATE_LIST = np.concatenate((BIOMED_DATE_FIRST, BIOMED_DATE_SECOND))
BIOMED_LOCATION_LIST = np.concatenate((BIOMED_LOCATION_FIRST, BIOMED_LOCATION_SECOND))
BIOMED_STATUS_LIST = np.concatenate((BIOMED_STATUS_FIRST, BIOMED_STATUS_SECOND))
BIOMED_SALARY_LIST = np.concatenate((BIOMED_SALARY_FIRST, BIOMED_SALARY_SECOND))
BIOMED_SALARY_MIN_LIST = np.concatenate((BIOMED_SALARY_MIN_FIRST, BIOMED_SALARY_MIN_SECOND))
BIOMED_SALARY_MAX_LIST = np.concatenate((BIOMED_SALARY_MAX_FIRST, BIOMED_SALARY_MAX_SECOND))
BIOMED_DESCRIPTION_LIST = np.concatenate((BIOMED_DESCRIPTION_FIRST, BIOMED_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Biomedical (JOBLUM) 
BIOMED={'Website': "Joblum",
      'Job Title': BIOMED_TITLE_LIST, 
      'Category': "Biomedical", 
      'Company': BIOMED_COMPANY_LIST, 
      'Date Posted': BIOMED_DATE_LIST, 
      'Location': BIOMED_LOCATION_LIST, 
      'Status': BIOMED_STATUS_LIST, 
      'Salary': BIOMED_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': BIOMED_DESCRIPTION_LIST,
      'Min Salary': BIOMED_SALARY_MIN_LIST,
      'Max Salary': BIOMED_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Medicine"}
BIOMED_df = pd.DataFrame(data=BIOMED)
BIOMED_df

In [ ]:
BIOMED_df.to_csv ('JOBLUM-BIOMED.csv', index = False)

In [ ]:
#Scraping data of Biotechnology (JOBLUM) - FIRST HALF

BIOTECH_TITLE_FIRST = []
BIOTECH_COMPANY_FIRST = []
BIOTECH_DATE_FIRST = []
BIOTECH_LOCATION_FIRST = []
BIOTECH_STATUS_FIRST = []
BIOTECH_SALARY_FIRST = []
BIOTECH_SALARY_MIN_FIRST = []
BIOTECH_SALARY_MAX_FIRST = []
BIOTECH_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-biotechnology?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, BIOTECH_TITLE_FIRST, BIOTECH_COMPANY_FIRST, BIOTECH_DATE_FIRST, 
              BIOTECH_STATUS_FIRST, BIOTECH_LOCATION_FIRST, BIOTECH_SALARY_FIRST, 
              BIOTECH_SALARY_MIN_FIRST, BIOTECH_SALARY_MAX_FIRST, BIOTECH_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Biotechnology (JOBLUM) - SECOND HALF

BIOTECH_TITLE_SECOND = []
BIOTECH_COMPANY_SECOND = []
BIOTECH_DATE_SECOND = []
BIOTECH_LOCATION_SECOND = []
BIOTECH_STATUS_SECOND = []
BIOTECH_SALARY_SECOND = []
BIOTECH_SALARY_MIN_SECOND = []
BIOTECH_SALARY_MAX_SECOND = []
BIOTECH_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, BIOTECH_TITLE_SECOND, BIOTECH_COMPANY_SECOND, BIOTECH_DATE_SECOND, 
              BIOTECH_STATUS_SECOND, BIOTECH_LOCATION_SECOND, BIOTECH_SALARY_SECOND, 
              BIOTECH_SALARY_MIN_SECOND, BIOTECH_SALARY_MAX_SECOND, BIOTECH_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Biotechnology (JOBLUM) 

BIOTECH_TITLE_LIST = np.concatenate((BIOTECH_TITLE_FIRST, BIOTECH_TITLE_SECOND))
BIOTECH_COMPANY_LIST = np.concatenate((BIOTECH_COMPANY_FIRST, BIOTECH_COMPANY_SECOND))
BIOTECH_DATE_LIST = np.concatenate((BIOTECH_DATE_FIRST, BIOTECH_DATE_SECOND))
BIOTECH_LOCATION_LIST = np.concatenate((BIOTECH_LOCATION_FIRST, BIOTECH_LOCATION_SECOND))
BIOTECH_STATUS_LIST = np.concatenate((BIOTECH_STATUS_FIRST, BIOTECH_STATUS_SECOND))
BIOTECH_SALARY_LIST = np.concatenate((BIOTECH_SALARY_FIRST, BIOTECH_SALARY_SECOND))
BIOTECH_SALARY_MIN_LIST = np.concatenate((BIOTECH_SALARY_MIN_FIRST, BIOTECH_SALARY_MIN_SECOND))
BIOTECH_SALARY_MAX_LIST = np.concatenate((BIOTECH_SALARY_MAX_FIRST, BIOTECH_SALARY_MAX_SECOND))
BIOTECH_DESCRIPTION_LIST = np.concatenate((BIOTECH_DESCRIPTION_FIRST, BIOTECH_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Biotechnology (JOBLUM) 
BIOTECH={'Website': "Joblum",
      'Job Title': BIOTECH_TITLE_LIST, 
      'Category': "Biotechnology", 
      'Company': BIOTECH_COMPANY_LIST, 
      'Date Posted': BIOTECH_DATE_LIST, 
      'Location': BIOTECH_LOCATION_LIST, 
      'Status': BIOTECH_STATUS_LIST, 
      'Salary': BIOTECH_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': BIOTECH_DESCRIPTION_LIST,
      'Min Salary': BIOTECH_SALARY_MIN_LIST,
      'Max Salary': BIOTECH_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Science"}
BIOTECH_df = pd.DataFrame(data=BIOTECH)
BIOTECH_df

In [ ]:
BIOTECH_df.to_csv ('JOBLUM-BIOTECH.csv', index = False)

In [ ]:
#Scraping data of Chemical Engineering (JOBLUM) - FIRST HALF

CHEMENG_TITLE_FIRST = []
CHEMENG_COMPANY_FIRST = []
CHEMENG_DATE_FIRST = []
CHEMENG_LOCATION_FIRST = []
CHEMENG_STATUS_FIRST = []
CHEMENG_SALARY_FIRST = []
CHEMENG_SALARY_MIN_FIRST = []
CHEMENG_SALARY_MAX_FIRST = []
CHEMENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-chemical-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CHEMENG_TITLE_FIRST, CHEMENG_COMPANY_FIRST, CHEMENG_DATE_FIRST, 
              CHEMENG_STATUS_FIRST, CHEMENG_LOCATION_FIRST, CHEMENG_SALARY_FIRST, 
              CHEMENG_SALARY_MIN_FIRST, CHEMENG_SALARY_MAX_FIRST, CHEMENG_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Chemical Engineering (JOBLUM) - SECOND HALF

CHEMENG_TITLE_SECOND = []
CHEMENG_COMPANY_SECOND = []
CHEMENG_DATE_SECOND = []
CHEMENG_LOCATION_SECOND = []
CHEMENG_STATUS_SECOND = []
CHEMENG_SALARY_SECOND = []
CHEMENG_SALARY_MIN_SECOND = []
CHEMENG_SALARY_MAX_SECOND = []
CHEMENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CHEMENG_TITLE_SECOND, CHEMENG_COMPANY_SECOND, CHEMENG_DATE_SECOND, 
              CHEMENG_STATUS_SECOND, CHEMENG_LOCATION_SECOND, CHEMENG_SALARY_SECOND, 
              CHEMENG_SALARY_MIN_SECOND, CHEMENG_SALARY_MAX_SECOND, CHEMENG_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Chemical Engineering (JOBLUM) 

CHEMENG_TITLE_LIST = np.concatenate((CHEMENG_TITLE_FIRST, CHEMENG_TITLE_SECOND))
CHEMENG_COMPANY_LIST = np.concatenate((CHEMENG_COMPANY_FIRST, CHEMENG_COMPANY_SECOND))
CHEMENG_DATE_LIST = np.concatenate((CHEMENG_DATE_FIRST, CHEMENG_DATE_SECOND))
CHEMENG_LOCATION_LIST = np.concatenate((CHEMENG_LOCATION_FIRST, CHEMENG_LOCATION_SECOND))
CHEMENG_STATUS_LIST = np.concatenate((CHEMENG_STATUS_FIRST, CHEMENG_STATUS_SECOND))
CHEMENG_SALARY_LIST = np.concatenate((CHEMENG_SALARY_FIRST, CHEMENG_SALARY_SECOND))
CHEMENG_SALARY_MIN_LIST = np.concatenate((CHEMENG_SALARY_MIN_FIRST, CHEMENG_SALARY_MIN_SECOND))
CHEMENG_SALARY_MAX_LIST = np.concatenate((CHEMENG_SALARY_MAX_FIRST, CHEMENG_SALARY_MAX_SECOND))
CHEMENG_DESCRIPTION_LIST = np.concatenate((CHEMENG_DESCRIPTION_FIRST, CHEMENG_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Chemical Engineering (JOBLUM) 
CHEMENG={'Website': "Joblum",
      'Job Title': CHEMENG_TITLE_LIST, 
      'Category': "Chemical Engineering", 
      'Company': CHEMENG_COMPANY_LIST, 
      'Date Posted': CHEMENG_DATE_LIST, 
      'Location': CHEMENG_LOCATION_LIST, 
      'Status': CHEMENG_STATUS_LIST, 
      'Salary': CHEMENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': CHEMENG_DESCRIPTION_LIST,
      'Min Salary': CHEMENG_SALARY_MIN_LIST,
      'Max Salary': CHEMENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
CHEMENG_df = pd.DataFrame(data=CHEMENG)
CHEMENG_df

In [ ]:
CHEMENG_df.to_csv ('JOBLUM-CHEMENG.csv', index = False)

In [ ]:
#Scraping data of Chemistry (JOBLUM) - FIRST HALF

CHEM_TITLE_FIRST = []
CHEM_COMPANY_FIRST = []
CHEM_DATE_FIRST = []
CHEM_LOCATION_FIRST = []
CHEM_STATUS_FIRST = []
CHEM_SALARY_FIRST = []
CHEM_SALARY_MIN_FIRST = []
CHEM_SALARY_MAX_FIRST = []
CHEM_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-chemistry?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CHEM_TITLE_FIRST, CHEM_COMPANY_FIRST, CHEM_DATE_FIRST, 
              CHEM_STATUS_FIRST, CHEM_LOCATION_FIRST, CHEM_SALARY_FIRST, 
              CHEM_SALARY_MIN_FIRST, CHEM_SALARY_MAX_FIRST, CHEM_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Chemistry (JOBLUM) - SECOND HALF

CHEM_TITLE_SECOND = []
CHEM_COMPANY_SECOND = []
CHEM_DATE_SECOND = []
CHEM_LOCATION_SECOND = []
CHEM_STATUS_SECOND = []
CHEM_SALARY_SECOND = []
CHEM_SALARY_MIN_SECOND = []
CHEM_SALARY_MAX_SECOND = []
CHEM_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CHEM_TITLE_SECOND, CHEM_COMPANY_SECOND, CHEM_DATE_SECOND, 
              CHEM_STATUS_SECOND, CHEM_LOCATION_SECOND, CHEM_SALARY_SECOND, 
              CHEM_SALARY_MIN_SECOND, CHEM_SALARY_MAX_SECOND, CHEM_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Chemistry (JOBLUM) 

CHEM_TITLE_LIST = np.concatenate((CHEM_TITLE_FIRST, CHEM_TITLE_SECOND))
CHEM_COMPANY_LIST = np.concatenate((CHEM_COMPANY_FIRST, CHEM_COMPANY_SECOND))
CHEM_DATE_LIST = np.concatenate((CHEM_DATE_FIRST, CHEM_DATE_SECOND))
CHEM_LOCATION_LIST = np.concatenate((CHEM_LOCATION_FIRST, CHEM_LOCATION_SECOND))
CHEM_STATUS_LIST = np.concatenate((CHEM_STATUS_FIRST, CHEM_STATUS_SECOND))
CHEM_SALARY_LIST = np.concatenate((CHEM_SALARY_FIRST, CHEM_SALARY_SECOND))
CHEM_SALARY_MIN_LIST = np.concatenate((CHEM_SALARY_MIN_FIRST, CHEM_SALARY_MIN_SECOND))
CHEM_SALARY_MAX_LIST = np.concatenate((CHEM_SALARY_MAX_FIRST, CHEM_SALARY_MAX_SECOND))
CHEM_DESCRIPTION_LIST = np.concatenate((CHEM_DESCRIPTION_FIRST, CHEM_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Chemistry (JOBLUM) 
CHEM={'Website': "Joblum",
      'Job Title': CHEM_TITLE_LIST, 
      'Category': "Chemistry", 
      'Company': CHEM_COMPANY_LIST, 
      'Date Posted': CHEM_DATE_LIST, 
      'Location': CHEM_LOCATION_LIST, 
      'Status': CHEM_STATUS_LIST, 
      'Salary': CHEM_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': CHEM_DESCRIPTION_LIST,
      'Min Salary': CHEM_SALARY_MIN_LIST,
      'Max Salary': CHEM_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Science"}
CHEM_df = pd.DataFrame(data=CHEM)
CHEM_df

In [ ]:
CHEM_df.to_csv ('JOBLUM-CHEM.csv', index = False)

In [ ]:
#Scraping data of Civil Engineering/Construction (JOBLUM) - FIRST HALF

CIVILENG_TITLE_FIRST = []
CIVILENG_COMPANY_FIRST = []
CIVILENG_DATE_FIRST = []
CIVILENG_LOCATION_FIRST = []
CIVILENG_STATUS_FIRST = []
CIVILENG_SALARY_FIRST = []
CIVILENG_SALARY_MIN_FIRST = []
CIVILENG_SALARY_MAX_FIRST = []
CIVILENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-civil-engineering-construction?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CIVILENG_TITLE_FIRST, CIVILENG_COMPANY_FIRST, CIVILENG_DATE_FIRST, 
              CIVILENG_STATUS_FIRST, CIVILENG_LOCATION_FIRST, CIVILENG_SALARY_FIRST, 
              CIVILENG_SALARY_MIN_FIRST, CIVILENG_SALARY_MAX_FIRST, CIVILENG_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Civil Engineering/Construction (JOBLUM) - SECOND HALF

CIVILENG_TITLE_SECOND = []
CIVILENG_COMPANY_SECOND = []
CIVILENG_DATE_SECOND = []
CIVILENG_LOCATION_SECOND = []
CIVILENG_STATUS_SECOND = []
CIVILENG_SALARY_SECOND = []
CIVILENG_SALARY_MIN_SECOND = []
CIVILENG_SALARY_MAX_SECOND = []
CIVILENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CIVILENG_TITLE_SECOND, CIVILENG_COMPANY_SECOND, CIVILENG_DATE_SECOND, 
              CIVILENG_STATUS_SECOND, CIVILENG_LOCATION_SECOND, CIVILENG_SALARY_SECOND, 
              CIVILENG_SALARY_MIN_SECOND, CIVILENG_SALARY_MAX_SECOND, CIVILENG_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Civil Engineering/Construction (JOBLUM) 

CIVILENG_TITLE_LIST = np.concatenate((CIVILENG_TITLE_FIRST, CIVILENG_TITLE_SECOND))
CIVILENG_COMPANY_LIST = np.concatenate((CIVILENG_COMPANY_FIRST, CIVILENG_COMPANY_SECOND))
CIVILENG_DATE_LIST = np.concatenate((CIVILENG_DATE_FIRST, CIVILENG_DATE_SECOND))
CIVILENG_LOCATION_LIST = np.concatenate((CIVILENG_LOCATION_FIRST, CIVILENG_LOCATION_SECOND))
CIVILENG_STATUS_LIST = np.concatenate((CIVILENG_STATUS_FIRST, CIVILENG_STATUS_SECOND))
CIVILENG_SALARY_LIST = np.concatenate((CIVILENG_SALARY_FIRST, CIVILENG_SALARY_SECOND))
CIVILENG_SALARY_MIN_LIST = np.concatenate((CIVILENG_SALARY_MIN_FIRST, CIVILENG_SALARY_MIN_SECOND))
CIVILENG_SALARY_MAX_LIST = np.concatenate((CIVILENG_SALARY_MAX_FIRST, CIVILENG_SALARY_MAX_SECOND))
CIVILENG_DESCRIPTION_LIST = np.concatenate((CIVILENG_DESCRIPTION_FIRST, CIVILENG_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Civil Engineering/Construction (JOBLUM) 
CIVILENG={'Website': "Joblum",
      'Job Title': CIVILENG_TITLE_LIST, 
      'Category': "Civil Engineering/Construction", 
      'Company': CIVILENG_COMPANY_LIST, 
      'Date Posted': CIVILENG_DATE_LIST, 
      'Location': CIVILENG_LOCATION_LIST, 
      'Status': CIVILENG_STATUS_LIST, 
      'Salary': CIVILENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': CIVILENG_DESCRIPTION_LIST,
      'Min Salary': CIVILENG_SALARY_MIN_LIST,
      'Max Salary': CIVILENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
CIVILENG_df = pd.DataFrame(data=CIVILENG)
CIVILENG_df

In [ ]:
CIVILENG_df.to_csv ('JOBLUM-CIVILENG.csv', index = False)

In [ ]:
#Scraping data of Civil/Construction (JOBLUM) - FIRST HALF

CONSTRUCTION_TITLE_FIRST = []
CONSTRUCTION_COMPANY_FIRST = []
CONSTRUCTION_DATE_FIRST = []
CONSTRUCTION_LOCATION_FIRST = []
CONSTRUCTION_STATUS_FIRST = []
CONSTRUCTION_SALARY_FIRST = []
CONSTRUCTION_SALARY_MIN_FIRST = []
CONSTRUCTION_SALARY_MAX_FIRST = []
CONSTRUCTION_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-civil-construction?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CONSTRUCTION_TITLE_FIRST, CONSTRUCTION_COMPANY_FIRST, CONSTRUCTION_DATE_FIRST, 
              CONSTRUCTION_STATUS_FIRST, CONSTRUCTION_LOCATION_FIRST, CONSTRUCTION_SALARY_FIRST, 
              CONSTRUCTION_SALARY_MIN_FIRST, CONSTRUCTION_SALARY_MAX_FIRST, CONSTRUCTION_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Civil/Construction (JOBLUM) - SECOND HALF

CONSTRUCTION_TITLE_SECOND = []
CONSTRUCTION_COMPANY_SECOND = []
CONSTRUCTION_DATE_SECOND = []
CONSTRUCTION_LOCATION_SECOND = []
CONSTRUCTION_STATUS_SECOND = []
CONSTRUCTION_SALARY_SECOND = []
CONSTRUCTION_SALARY_MIN_SECOND = []
CONSTRUCTION_SALARY_MAX_SECOND = []
CONSTRUCTION_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, CONSTRUCTION_TITLE_SECOND, CONSTRUCTION_COMPANY_SECOND, CONSTRUCTION_DATE_SECOND, 
              CONSTRUCTION_STATUS_SECOND, CONSTRUCTION_LOCATION_SECOND, CONSTRUCTION_SALARY_SECOND, 
              CONSTRUCTION_SALARY_MIN_SECOND, CONSTRUCTION_SALARY_MAX_SECOND, CONSTRUCTION_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Civil/Construction (JOBLUM) 

CONSTRUCTION_TITLE_LIST = np.concatenate((CONSTRUCTION_TITLE_FIRST, CONSTRUCTION_TITLE_SECOND))
CONSTRUCTION_COMPANY_LIST = np.concatenate((CONSTRUCTION_COMPANY_FIRST, CONSTRUCTION_COMPANY_SECOND))
CONSTRUCTION_DATE_LIST = np.concatenate((CONSTRUCTION_DATE_FIRST, CONSTRUCTION_DATE_SECOND))
CONSTRUCTION_LOCATION_LIST = np.concatenate((CONSTRUCTION_LOCATION_FIRST, CONSTRUCTION_LOCATION_SECOND))
CONSTRUCTION_STATUS_LIST = np.concatenate((CONSTRUCTION_STATUS_FIRST, CONSTRUCTION_STATUS_SECOND))
CONSTRUCTION_SALARY_LIST = np.concatenate((CONSTRUCTION_SALARY_FIRST, CONSTRUCTION_SALARY_SECOND))
CONSTRUCTION_SALARY_MIN_LIST = np.concatenate((CONSTRUCTION_SALARY_MIN_FIRST, CONSTRUCTION_SALARY_MIN_SECOND))
CONSTRUCTION_SALARY_MAX_LIST = np.concatenate((CONSTRUCTION_SALARY_MAX_FIRST, CONSTRUCTION_SALARY_MAX_SECOND))
CONSTRUCTION_DESCRIPTION_LIST = np.concatenate((CONSTRUCTION_DESCRIPTION_FIRST, CONSTRUCTION_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Civil/Construction (JOBLUM) 
CONSTRUCTION={'Website': "Joblum",
      'Job Title': CONSTRUCTION_TITLE_LIST, 
      'Category': "Civil/Construction", 
      'Company': CONSTRUCTION_COMPANY_LIST, 
      'Date Posted': CONSTRUCTION_DATE_LIST, 
      'Location': CONSTRUCTION_LOCATION_LIST, 
      'Status': CONSTRUCTION_STATUS_LIST, 
      'Salary': CONSTRUCTION_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': CONSTRUCTION_DESCRIPTION_LIST,
      'Min Salary': CONSTRUCTION_SALARY_MIN_LIST,
      'Max Salary': CONSTRUCTION_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
CONSTRUCTION_df = pd.DataFrame(data=CONSTRUCTION)
CONSTRUCTION_df

In [ ]:
CONSTRUCTION_df.to_csv ('JOBLUM-CONSTRUCTION.csv', index = False)

In [ ]:
#Scraping data of Diagnosis/Others (JOBLUM) - FIRST HALF

DIAGNOSIS_TITLE_FIRST = []
DIAGNOSIS_COMPANY_FIRST = []
DIAGNOSIS_DATE_FIRST = []
DIAGNOSIS_LOCATION_FIRST = []
DIAGNOSIS_STATUS_FIRST = []
DIAGNOSIS_SALARY_FIRST = []
DIAGNOSIS_SALARY_MIN_FIRST = []
DIAGNOSIS_SALARY_MAX_FIRST = []
DIAGNOSIS_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-diagnosis-others?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, DIAGNOSIS_TITLE_FIRST, DIAGNOSIS_COMPANY_FIRST, DIAGNOSIS_DATE_FIRST, 
              DIAGNOSIS_STATUS_FIRST, DIAGNOSIS_LOCATION_FIRST, DIAGNOSIS_SALARY_FIRST, 
              DIAGNOSIS_SALARY_MIN_FIRST, DIAGNOSIS_SALARY_MAX_FIRST, DIAGNOSIS_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Diagnosis/Others (JOBLUM) - SECOND HALF

DIAGNOSIS_TITLE_SECOND = []
DIAGNOSIS_COMPANY_SECOND = []
DIAGNOSIS_DATE_SECOND = []
DIAGNOSIS_LOCATION_SECOND = []
DIAGNOSIS_STATUS_SECOND = []
DIAGNOSIS_SALARY_SECOND = []
DIAGNOSIS_SALARY_MIN_SECOND = []
DIAGNOSIS_SALARY_MAX_SECOND = []
DIAGNOSIS_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, DIAGNOSIS_TITLE_SECOND, DIAGNOSIS_COMPANY_SECOND, DIAGNOSIS_DATE_SECOND, 
              DIAGNOSIS_STATUS_SECOND, DIAGNOSIS_LOCATION_SECOND, DIAGNOSIS_SALARY_SECOND, 
              DIAGNOSIS_SALARY_MIN_SECOND, DIAGNOSIS_SALARY_MAX_SECOND, DIAGNOSIS_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Diagnosis/Others (JOBLUM) 

DIAGNOSIS_TITLE_LIST = np.concatenate((DIAGNOSIS_TITLE_FIRST, DIAGNOSIS_TITLE_SECOND))
DIAGNOSIS_COMPANY_LIST = np.concatenate((DIAGNOSIS_COMPANY_FIRST, DIAGNOSIS_COMPANY_SECOND))
DIAGNOSIS_DATE_LIST = np.concatenate((DIAGNOSIS_DATE_FIRST, DIAGNOSIS_DATE_SECOND))
DIAGNOSIS_LOCATION_LIST = np.concatenate((DIAGNOSIS_LOCATION_FIRST, DIAGNOSIS_LOCATION_SECOND))
DIAGNOSIS_STATUS_LIST = np.concatenate((DIAGNOSIS_STATUS_FIRST, DIAGNOSIS_STATUS_SECOND))
DIAGNOSIS_SALARY_LIST = np.concatenate((DIAGNOSIS_SALARY_FIRST, DIAGNOSIS_SALARY_SECOND))
DIAGNOSIS_SALARY_MIN_LIST = np.concatenate((DIAGNOSIS_SALARY_MIN_FIRST, DIAGNOSIS_SALARY_MIN_SECOND))
DIAGNOSIS_SALARY_MAX_LIST = np.concatenate((DIAGNOSIS_SALARY_MAX_FIRST, DIAGNOSIS_SALARY_MAX_SECOND))
DIAGNOSIS_DESCRIPTION_LIST = np.concatenate((DIAGNOSIS_DESCRIPTION_FIRST, DIAGNOSIS_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Diagnosis/Others (JOBLUM) 
DIAGNOSIS={'Website': "Joblum",
      'Job Title': DIAGNOSIS_TITLE_LIST, 
      'Category': "Diagnosis/Others", 
      'Company': DIAGNOSIS_COMPANY_LIST, 
      'Date Posted': DIAGNOSIS_DATE_LIST, 
      'Location': DIAGNOSIS_LOCATION_LIST, 
      'Status': DIAGNOSIS_STATUS_LIST, 
      'Salary': DIAGNOSIS_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': DIAGNOSIS_DESCRIPTION_LIST,
      'Min Salary': DIAGNOSIS_SALARY_MIN_LIST,
      'Max Salary': DIAGNOSIS_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Medicine"}
DIAGNOSIS_df = pd.DataFrame(data=DIAGNOSIS)
DIAGNOSIS_df

In [ ]:
DIAGNOSIS_df.to_csv ('JOBLUM-DIAGNOSIS.csv', index = False)

In [ ]:
#Scraping data of Doctor/Diagnosis (JOBLUM) - FIRST HALF

DOCTOR_TITLE_FIRST = []
DOCTOR_COMPANY_FIRST = []
DOCTOR_DATE_FIRST = []
DOCTOR_LOCATION_FIRST = []
DOCTOR_STATUS_FIRST = []
DOCTOR_SALARY_FIRST = []
DOCTOR_SALARY_MIN_FIRST = []
DOCTOR_SALARY_MAX_FIRST = []
DOCTOR_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-doctor-diagnosis?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, DOCTOR_TITLE_FIRST, DOCTOR_COMPANY_FIRST, DOCTOR_DATE_FIRST, 
              DOCTOR_STATUS_FIRST, DOCTOR_LOCATION_FIRST, DOCTOR_SALARY_FIRST, 
              DOCTOR_SALARY_MIN_FIRST, DOCTOR_SALARY_MAX_FIRST, DOCTOR_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Doctor/Diagnosis (JOBLUM) - SECOND HALF

DOCTOR_TITLE_SECOND = []
DOCTOR_COMPANY_SECOND = []
DOCTOR_DATE_SECOND = []
DOCTOR_LOCATION_SECOND = []
DOCTOR_STATUS_SECOND = []
DOCTOR_SALARY_SECOND = []
DOCTOR_SALARY_MIN_SECOND = []
DOCTOR_SALARY_MAX_SECOND = []
DOCTOR_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, DOCTOR_TITLE_SECOND, DOCTOR_COMPANY_SECOND, DOCTOR_DATE_SECOND, 
              DOCTOR_STATUS_SECOND, DOCTOR_LOCATION_SECOND, DOCTOR_SALARY_SECOND, 
              DOCTOR_SALARY_MIN_SECOND, DOCTOR_SALARY_MAX_SECOND, DOCTOR_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Doctor/Diagnosis (JOBLUM) 

DOCTOR_TITLE_LIST = np.concatenate((DOCTOR_TITLE_FIRST, DOCTOR_TITLE_SECOND))
DOCTOR_COMPANY_LIST = np.concatenate((DOCTOR_COMPANY_FIRST, DOCTOR_COMPANY_SECOND))
DOCTOR_DATE_LIST = np.concatenate((DOCTOR_DATE_FIRST, DOCTOR_DATE_SECOND))
DOCTOR_LOCATION_LIST = np.concatenate((DOCTOR_LOCATION_FIRST, DOCTOR_LOCATION_SECOND))
DOCTOR_STATUS_LIST = np.concatenate((DOCTOR_STATUS_FIRST, DOCTOR_STATUS_SECOND))
DOCTOR_SALARY_LIST = np.concatenate((DOCTOR_SALARY_FIRST, DOCTOR_SALARY_SECOND))
DOCTOR_SALARY_MIN_LIST = np.concatenate((DOCTOR_SALARY_MIN_FIRST, DOCTOR_SALARY_MIN_SECOND))
DOCTOR_SALARY_MAX_LIST = np.concatenate((DOCTOR_SALARY_MAX_FIRST, DOCTOR_SALARY_MAX_SECOND))
DOCTOR_DESCRIPTION_LIST = np.concatenate((DOCTOR_DESCRIPTION_FIRST, DOCTOR_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Doctor/Diagnosis (JOBLUM) 
DOCTOR={'Website': "Joblum",
      'Job Title': DOCTOR_TITLE_LIST, 
      'Category': "Doctor/DOCTOR", 
      'Company': DOCTOR_COMPANY_LIST, 
      'Date Posted': DOCTOR_DATE_LIST, 
      'Location': DOCTOR_LOCATION_LIST, 
      'Status': DOCTOR_STATUS_LIST, 
      'Salary': DOCTOR_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': DOCTOR_DESCRIPTION_LIST,
      'Min Salary': DOCTOR_SALARY_MIN_LIST,
      'Max Salary': DOCTOR_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Medicine"}
DOCTOR_df = pd.DataFrame(data=DOCTOR)
DOCTOR_df

In [ ]:
DOCTOR_df.to_csv ('JOBLUM-DOCTOR.csv', index = False)

In [ ]:
#Scraping data of Electrical (JOBLUM) - FIRST HALF

ELEC_TITLE_FIRST = []
ELEC_COMPANY_FIRST = []
ELEC_DATE_FIRST = []
ELEC_LOCATION_FIRST = []
ELEC_STATUS_FIRST = []
ELEC_SALARY_FIRST = []
ELEC_SALARY_MIN_FIRST = []
ELEC_SALARY_MAX_FIRST = []
ELEC_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-electrical?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELEC_TITLE_FIRST, ELEC_COMPANY_FIRST, ELEC_DATE_FIRST, 
              ELEC_STATUS_FIRST, ELEC_LOCATION_FIRST, ELEC_SALARY_FIRST, 
              ELEC_SALARY_MIN_FIRST, ELEC_SALARY_MAX_FIRST, ELEC_DESCRIPTION_FIRST)

In [ ]:
#Scraping data of Electrical (JOBLUM) - SECOND HALF

ELEC_TITLE_SECOND = []
ELEC_COMPANY_SECOND = []
ELEC_DATE_SECOND = []
ELEC_LOCATION_SECOND = []
ELEC_STATUS_SECOND = []
ELEC_SALARY_SECOND = []
ELEC_SALARY_MIN_SECOND = []
ELEC_SALARY_MAX_SECOND = []
ELEC_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELEC_TITLE_SECOND, ELEC_COMPANY_SECOND, ELEC_DATE_SECOND, 
              ELEC_STATUS_SECOND, ELEC_LOCATION_SECOND, ELEC_SALARY_SECOND, 
              ELEC_SALARY_MIN_SECOND, ELEC_SALARY_MAX_SECOND, ELEC_DESCRIPTION_FIRST)

In [ ]:
#Joining Scrape data of Electrical (JOBLUM) 

ELEC_TITLE_LIST = np.concatenate((ELEC_TITLE_FIRST, ELEC_TITLE_SECOND))
ELEC_COMPANY_LIST = np.concatenate((ELEC_COMPANY_FIRST, ELEC_COMPANY_SECOND))
ELEC_DATE_LIST = np.concatenate((ELEC_DATE_FIRST, ELEC_DATE_SECOND))
ELEC_LOCATION_LIST = np.concatenate((ELEC_LOCATION_FIRST, ELEC_LOCATION_SECOND))
ELEC_STATUS_LIST = np.concatenate((ELEC_STATUS_FIRST, ELEC_STATUS_SECOND))
ELEC_SALARY_LIST = np.concatenate((ELEC_SALARY_FIRST, ELEC_SALARY_SECOND))
ELEC_SALARY_MIN_LIST = np.concatenate((ELEC_SALARY_MIN_FIRST, ELEC_SALARY_MIN_SECOND))
ELEC_SALARY_MAX_LIST = np.concatenate((ELEC_SALARY_MAX_FIRST, ELEC_SALARY_MAX_SECOND))
ELEC_DESCRIPTION_LIST = np.concatenate((ELEC_DESCRIPTION_FIRST, ELEC_DESCRIPTION_SECOND))

In [ ]:
#Creating Data Frame for Electrical (JOBLUM) 
ELEC={'Website': "Joblum",
      'Job Title': ELEC_TITLE_LIST, 
      'Category': "Electrical", 
      'Company': ELEC_COMPANY_LIST, 
      'Date Posted': ELEC_DATE_LIST, 
      'Location': ELEC_LOCATION_LIST, 
      'Status': ELEC_STATUS_LIST, 
      'Salary': ELEC_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ELEC_DESCRIPTION_LIST,
      'Min Salary': ELEC_SALARY_MIN_LIST,
      'Max Salary': ELEC_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
ELEC_df = pd.DataFrame(data=ELEC)
ELEC_df

In [ ]:
ELEC_df.to_csv ('JOBLUM-ELEC.csv', index = False)

In [101]:
#Scraping data of Electrical Engineering (JOBLUM) - FIRST HALF

ELECENG_TITLE_FIRST = []
ELECENG_COMPANY_FIRST = []
ELECENG_DATE_FIRST = []
ELECENG_LOCATION_FIRST = []
ELECENG_STATUS_FIRST = []
ELECENG_SALARY_FIRST = []
ELECENG_SALARY_MIN_FIRST = []
ELECENG_SALARY_MAX_FIRST = []
ELECENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-electrical-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELECENG_TITLE_FIRST, ELECENG_COMPANY_FIRST, ELECENG_DATE_FIRST, 
              ELECENG_STATUS_FIRST, ELECENG_LOCATION_FIRST, ELECENG_SALARY_FIRST, 
              ELECENG_SALARY_MIN_FIRST, ELECENG_SALARY_MAX_FIRST, ELECENG_DESCRIPTION_FIRST)

In [102]:
#Scraping data of Electrical Engineering (JOBLUM) - SECOND HALF

ELECENG_TITLE_SECOND = []
ELECENG_COMPANY_SECOND = []
ELECENG_DATE_SECOND = []
ELECENG_LOCATION_SECOND = []
ELECENG_STATUS_SECOND = []
ELECENG_SALARY_SECOND = []
ELECENG_SALARY_MIN_SECOND = []
ELECENG_SALARY_MAX_SECOND = []
ELECENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELECENG_TITLE_SECOND, ELECENG_COMPANY_SECOND, ELECENG_DATE_SECOND, 
              ELECENG_STATUS_SECOND, ELECENG_LOCATION_SECOND, ELECENG_SALARY_SECOND, 
              ELECENG_SALARY_MIN_SECOND, ELECENG_SALARY_MAX_SECOND, ELECENG_DESCRIPTION_FIRST)

In [103]:
#Joining Scrape data of Electrical Engineering (JOBLUM) 

ELECENG_TITLE_LIST = np.concatenate((ELECENG_TITLE_FIRST, ELECENG_TITLE_SECOND))
ELECENG_COMPANY_LIST = np.concatenate((ELECENG_COMPANY_FIRST, ELECENG_COMPANY_SECOND))
ELECENG_DATE_LIST = np.concatenate((ELECENG_DATE_FIRST, ELECENG_DATE_SECOND))
ELECENG_LOCATION_LIST = np.concatenate((ELECENG_LOCATION_FIRST, ELECENG_LOCATION_SECOND))
ELECENG_STATUS_LIST = np.concatenate((ELECENG_STATUS_FIRST, ELECENG_STATUS_SECOND))
ELECENG_SALARY_LIST = np.concatenate((ELECENG_SALARY_FIRST, ELECENG_SALARY_SECOND))
ELECENG_SALARY_MIN_LIST = np.concatenate((ELECENG_SALARY_MIN_FIRST, ELECENG_SALARY_MIN_SECOND))
ELECENG_SALARY_MAX_LIST = np.concatenate((ELECENG_SALARY_MAX_FIRST, ELECENG_SALARY_MAX_SECOND))
ELECENG_DESCRIPTION_LIST = np.concatenate((ELECENG_DESCRIPTION_FIRST, ELECENG_DESCRIPTION_SECOND))

In [104]:
#Creating Data Frame for Electrical Engineering (JOBLUM) 
ELECENG={'Website': "Joblum",
      'Job Title': ELECENG_TITLE_LIST, 
      'Category': "Electrical Engineering", 
      'Company': ELECENG_COMPANY_LIST, 
      'Date Posted': ELECENG_DATE_LIST, 
      'Location': ELECENG_LOCATION_LIST, 
      'Status': ELECENG_STATUS_LIST, 
      'Salary': ELECENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ELECENG_DESCRIPTION_LIST,
      'Min Salary': ELECENG_SALARY_MIN_LIST,
      'Max Salary': ELECENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
ELECENG_df = pd.DataFrame(data=ELECENG)
ELECENG_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Sr. Manager for Cybersecurity,Electrical Engineering,Axis Online Ventures Inc.,2021-05-26,Makati,Not Specified,"99,000-100,000",Not Specified / In Description,Not Specified / In Description,Certification/Experience: knowledgeable in ISO...,"99,000","100,000",Not Specified,Not Specified,Engineering


In [105]:
ELECENG_df.to_csv ('JOBLUM-ELECENG.csv', index = False)

In [96]:
#Scraping data of Electronics (JOBLUM) - FIRST HALF

ELECTRO_TITLE_FIRST = []
ELECTRO_COMPANY_FIRST = []
ELECTRO_DATE_FIRST = []
ELECTRO_LOCATION_FIRST = []
ELECTRO_STATUS_FIRST = []
ELECTRO_SALARY_FIRST = []
ELECTRO_SALARY_MIN_FIRST = []
ELECTRO_SALARY_MAX_FIRST = []
ELECTRO_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-electronics?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELECTRO_TITLE_FIRST, ELECTRO_COMPANY_FIRST, ELECTRO_DATE_FIRST, 
              ELECTRO_STATUS_FIRST, ELECTRO_LOCATION_FIRST, ELECTRO_SALARY_FIRST, 
              ELECTRO_SALARY_MIN_FIRST, ELECTRO_SALARY_MAX_FIRST, ELECTRO_DESCRIPTION_FIRST)

In [97]:
#Scraping data of Electronics (JOBLUM) - SECOND HALF

ELECTRO_TITLE_SECOND = []
ELECTRO_COMPANY_SECOND = []
ELECTRO_DATE_SECOND = []
ELECTRO_LOCATION_SECOND = []
ELECTRO_STATUS_SECOND = []
ELECTRO_SALARY_SECOND = []
ELECTRO_SALARY_MIN_SECOND = []
ELECTRO_SALARY_MAX_SECOND = []
ELECTRO_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELECTRO_TITLE_SECOND, ELECTRO_COMPANY_SECOND, ELECTRO_DATE_SECOND, 
              ELECTRO_STATUS_SECOND, ELECTRO_LOCATION_SECOND, ELECTRO_SALARY_SECOND, 
              ELECTRO_SALARY_MIN_SECOND, ELECTRO_SALARY_MAX_SECOND, ELECTRO_DESCRIPTION_FIRST)

In [98]:
#Joining Scrape data of Electronics (JOBLUM) 

ELECTRO_TITLE_LIST = np.concatenate((ELECTRO_TITLE_FIRST, ELECTRO_TITLE_SECOND))
ELECTRO_COMPANY_LIST = np.concatenate((ELECTRO_COMPANY_FIRST, ELECTRO_COMPANY_SECOND))
ELECTRO_DATE_LIST = np.concatenate((ELECTRO_DATE_FIRST, ELECTRO_DATE_SECOND))
ELECTRO_LOCATION_LIST = np.concatenate((ELECTRO_LOCATION_FIRST, ELECTRO_LOCATION_SECOND))
ELECTRO_STATUS_LIST = np.concatenate((ELECTRO_STATUS_FIRST, ELECTRO_STATUS_SECOND))
ELECTRO_SALARY_LIST = np.concatenate((ELECTRO_SALARY_FIRST, ELECTRO_SALARY_SECOND))
ELECTRO_SALARY_MIN_LIST = np.concatenate((ELECTRO_SALARY_MIN_FIRST, ELECTRO_SALARY_MIN_SECOND))
ELECTRO_SALARY_MAX_LIST = np.concatenate((ELECTRO_SALARY_MAX_FIRST, ELECTRO_SALARY_MAX_SECOND))
ELECTRO_DESCRIPTION_LIST = np.concatenate((ELECTRO_DESCRIPTION_FIRST, ELECTRO_DESCRIPTION_SECOND))

In [99]:
#Creating Data Frame for Electronics (JOBLUM) 
ELECTRO={'Website': "Joblum",
      'Job Title': ELECTRO_TITLE_LIST, 
      'Category': "Electronics", 
      'Company': ELECTRO_COMPANY_LIST, 
      'Date Posted': ELECTRO_DATE_LIST, 
      'Location': ELECTRO_LOCATION_LIST, 
      'Status': ELECTRO_STATUS_LIST, 
      'Salary': ELECTRO_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ELECTRO_DESCRIPTION_LIST,
      'Min Salary': ELECTRO_SALARY_MIN_LIST,
      'Max Salary': ELECTRO_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
ELECTRO_df = pd.DataFrame(data=ELECTRO)
ELECTRO_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Project Supervisor,Electronics,"[email protected] Knowledge Specialists, Inc.",2021-05-30,"Davao del Norte, Davao del Sur (Davao City), D...",Contract,,Not Specified / In Description,Not Specified / In Description,Job Qualifications:- Must be currently living ...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
1,Joblum,Assistant Project Manager,Electronics,"[email protected] Knowledge Specialists, Inc.",2021-05-30,"Davao del Norte, Davao del Sur (Davao City), D...",OJT,,Not Specified / In Description,Not Specified / In Description,Qualifications:• Must be have a Bachelor's deg...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
2,Joblum,Quality Engineer,Electronics,Amkor Technology Phils. Inc.,2021-05-30,National Capital Reg,Full Time,"25,000-45,000",Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,"25,000","45,000",Not Specified,Not Specified,Engineering
3,Joblum,Senior Quality Engineer,Electronics,Amkor Technology Phils. Inc.,2021-05-30,"Muntinlupa City, Laguna (Others)",OJT,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
4,Joblum,Biomedical Engineer/Technician,Electronics,Globequipt Corporation,2021-05-30,Calabarzon & Mimaropa,Contract,"17,000-23,800",Not Specified / In Description,Not Specified / In Description,"Biomedical Engineer/Technician PHP 17,000 - 23...","17,000","23,800",Not Specified,Not Specified,Engineering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Joblum,CABLE TECHNICIAN (Ilocos Norte),Electronics,Planet Cable Inc.,2021-04-30,Ilocos Region,OJT,,Not Specified / In Description,Not Specified / In Description,QUALIFICATIONS: Must possess at least any IT a...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
521,Joblum,TRANSFORMER DESIGNER,Electronics,Mega Masterlink Fabricator and Electrical Serv...,2021-04-30,Valenzuela City,Full Time,,Not Specified / In Description,Not Specified / In Description,Knowledge Candidate must possess at least a Ba...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
522,Joblum,Site Engineer(ECE),Electronics,Megacity General Construction and Development ...,2021-04-30,National Capital Reg,Not Specified,"25,000-35,000",Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Bachelor's/Col...,"25,000","35,000",Not Specified,Not Specified,Engineering
523,Joblum,Process and Equipment Technician,Electronics,Maxeon Solar Technologies Pte. Ltd. (SunPower ...,2021-04-29,Laguna (Sta. Rosa City),Not Specified,,Not Specified / In Description,Not Specified / In Description,Organizes and communicates ideas through a var...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering


In [100]:
ELECTRO_df.to_csv ('JOBLUM-ELECTRO.csv', index = False)

In [91]:
#Scraping data of Electronics Engineering (JOBLUM) - FIRST HALF

ELECTROENG_TITLE_FIRST = []
ELECTROENG_COMPANY_FIRST = []
ELECTROENG_DATE_FIRST = []
ELECTROENG_LOCATION_FIRST = []
ELECTROENG_STATUS_FIRST = []
ELECTROENG_SALARY_FIRST = []
ELECTROENG_SALARY_MIN_FIRST = []
ELECTROENG_SALARY_MAX_FIRST = []
ELECTROENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-electronics-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELECTROENG_TITLE_FIRST, ELECTROENG_COMPANY_FIRST, ELECTROENG_DATE_FIRST, 
              ELECTROENG_STATUS_FIRST, ELECTROENG_LOCATION_FIRST, ELECTROENG_SALARY_FIRST, 
              ELECTROENG_SALARY_MIN_FIRST, ELECTROENG_SALARY_MAX_FIRST, ELECTROENG_DESCRIPTION_FIRST)

In [92]:
#Scraping data of Electronics Engineering (JOBLUM) - SECOND HALF

ELECTROENG_TITLE_SECOND = []
ELECTROENG_COMPANY_SECOND = []
ELECTROENG_DATE_SECOND = []
ELECTROENG_LOCATION_SECOND = []
ELECTROENG_STATUS_SECOND = []
ELECTROENG_SALARY_SECOND = []
ELECTROENG_SALARY_MIN_SECOND = []
ELECTROENG_SALARY_MAX_SECOND = []
ELECTROENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ELECTROENG_TITLE_SECOND, ELECTROENG_COMPANY_SECOND, ELECTROENG_DATE_SECOND, 
              ELECTROENG_STATUS_SECOND, ELECTROENG_LOCATION_SECOND, ELECTROENG_SALARY_SECOND, 
              ELECTROENG_SALARY_MIN_SECOND, ELECTROENG_SALARY_MAX_SECOND, ELECTROENG_DESCRIPTION_FIRST)

In [93]:
#Joining Scrape data of Electronics Engineering (JOBLUM) 

ELECTROENG_TITLE_LIST = np.concatenate((ELECTROENG_TITLE_FIRST, ELECTROENG_TITLE_SECOND))
ELECTROENG_COMPANY_LIST = np.concatenate((ELECTROENG_COMPANY_FIRST, ELECTROENG_COMPANY_SECOND))
ELECTROENG_DATE_LIST = np.concatenate((ELECTROENG_DATE_FIRST, ELECTROENG_DATE_SECOND))
ELECTROENG_LOCATION_LIST = np.concatenate((ELECTROENG_LOCATION_FIRST, ELECTROENG_LOCATION_SECOND))
ELECTROENG_STATUS_LIST = np.concatenate((ELECTROENG_STATUS_FIRST, ELECTROENG_STATUS_SECOND))
ELECTROENG_SALARY_LIST = np.concatenate((ELECTROENG_SALARY_FIRST, ELECTROENG_SALARY_SECOND))
ELECTROENG_SALARY_MIN_LIST = np.concatenate((ELECTROENG_SALARY_MIN_FIRST, ELECTROENG_SALARY_MIN_SECOND))
ELECTROENG_SALARY_MAX_LIST = np.concatenate((ELECTROENG_SALARY_MAX_FIRST, ELECTROENG_SALARY_MAX_SECOND))
ELECTROENG_DESCRIPTION_LIST = np.concatenate((ELECTROENG_DESCRIPTION_FIRST, ELECTROENG_DESCRIPTION_SECOND))

In [94]:
#Creating Data Frame for Electronics Engineering (JOBLUM) 
ELECTROENG={'Website': "Joblum",
      'Job Title': ELECTROENG_TITLE_LIST, 
      'Category': "Electronics Engineering", 
      'Company': ELECTROENG_COMPANY_LIST, 
      'Date Posted': ELECTROENG_DATE_LIST, 
      'Location': ELECTROENG_LOCATION_LIST, 
      'Status': ELECTROENG_STATUS_LIST, 
      'Salary': ELECTROENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ELECTROENG_DESCRIPTION_LIST,
      'Min Salary': ELECTROENG_SALARY_MIN_LIST,
      'Max Salary': ELECTROENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
ELECTROENG_df = pd.DataFrame(data=ELECTROENG)
ELECTROENG_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field


In [95]:
ELECTROENG_df.to_csv ('JOBLUM-ELECTROENG.csv', index = False)

In [86]:
#Scraping data of Environmental (JOBLUM) - FIRST HALF

ENVI_TITLE_FIRST = []
ENVI_COMPANY_FIRST = []
ENVI_DATE_FIRST = []
ENVI_LOCATION_FIRST = []
ENVI_STATUS_FIRST = []
ENVI_SALARY_FIRST = []
ENVI_SALARY_MIN_FIRST = []
ENVI_SALARY_MAX_FIRST = []
ENVI_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-environmental?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ENVI_TITLE_FIRST, ENVI_COMPANY_FIRST, ENVI_DATE_FIRST, 
              ENVI_STATUS_FIRST, ENVI_LOCATION_FIRST, ENVI_SALARY_FIRST, 
              ENVI_SALARY_MIN_FIRST, ENVI_SALARY_MAX_FIRST, ENVI_DESCRIPTION_FIRST)

In [87]:
#Scraping data of Environmental (JOBLUM) - SECOND HALF

ENVI_TITLE_SECOND = []
ENVI_COMPANY_SECOND = []
ENVI_DATE_SECOND = []
ENVI_LOCATION_SECOND = []
ENVI_STATUS_SECOND = []
ENVI_SALARY_SECOND = []
ENVI_SALARY_MIN_SECOND = []
ENVI_SALARY_MAX_SECOND = []
ENVI_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ENVI_TITLE_SECOND, ENVI_COMPANY_SECOND, ENVI_DATE_SECOND, 
              ENVI_STATUS_SECOND, ENVI_LOCATION_SECOND, ENVI_SALARY_SECOND, 
              ENVI_SALARY_MIN_SECOND, ENVI_SALARY_MAX_SECOND, ENVI_DESCRIPTION_FIRST)

In [88]:
#Joining Scrape data of Environmental (JOBLUM) 

ENVI_TITLE_LIST = np.concatenate((ENVI_TITLE_FIRST, ENVI_TITLE_SECOND))
ENVI_COMPANY_LIST = np.concatenate((ENVI_COMPANY_FIRST, ENVI_COMPANY_SECOND))
ENVI_DATE_LIST = np.concatenate((ENVI_DATE_FIRST, ENVI_DATE_SECOND))
ENVI_LOCATION_LIST = np.concatenate((ENVI_LOCATION_FIRST, ENVI_LOCATION_SECOND))
ENVI_STATUS_LIST = np.concatenate((ENVI_STATUS_FIRST, ENVI_STATUS_SECOND))
ENVI_SALARY_LIST = np.concatenate((ENVI_SALARY_FIRST, ENVI_SALARY_SECOND))
ENVI_SALARY_MIN_LIST = np.concatenate((ENVI_SALARY_MIN_FIRST, ENVI_SALARY_MIN_SECOND))
ENVI_SALARY_MAX_LIST = np.concatenate((ENVI_SALARY_MAX_FIRST, ENVI_SALARY_MAX_SECOND))
ENVI_DESCRIPTION_LIST = np.concatenate((ENVI_DESCRIPTION_FIRST, ENVI_DESCRIPTION_SECOND))

In [89]:
#Creating Data Frame for Environmental (JOBLUM) 
ENVI={'Website': "Joblum",
      'Job Title': ENVI_TITLE_LIST, 
      'Category': "Environmental", 
      'Company': ENVI_COMPANY_LIST, 
      'Date Posted': ENVI_DATE_LIST, 
      'Location': ENVI_LOCATION_LIST, 
      'Status': ENVI_STATUS_LIST, 
      'Salary': ENVI_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ENVI_DESCRIPTION_LIST,
      'Min Salary': ENVI_SALARY_MIN_LIST,
      'Max Salary': ENVI_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Science"}
ENVI_df = pd.DataFrame(data=ENVI)
ENVI_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Safety Officer (Iloilo City),Environmental,La Filipina Uy Gongco Group of Companies,2021-05-29,Iloilo (Iloilo City),Contract,,Not Specified / In Description,Not Specified / In Description,"QUALIFICATIONS Graduate of BS in Civil, Mechan...",Not Specified,Not Specified,Not Specified,Not Specified,Science
1,Joblum,SAFETY OFFICER,Environmental,Systech Lighting & Controls Inc.,2021-05-29,National Capital Reg,Full Time,"15,000-16,000",Not Specified / In Description,Not Specified / In Description,JOB DESCRIPTION Ø Graduate of any Four (4) yea...,"15,000","16,000",Not Specified,Not Specified,Science
2,Joblum,Safety Officer,Environmental,Zeller Plastik Phils. Inc.,2021-05-29,Muntinlupa City,Not Specified,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Bachelor's/Col...,Not Specified,Not Specified,Not Specified,Not Specified,Science
3,Joblum,SAFETY OFFICER,Environmental,PHILKO UBINS LTD. CORP.,2021-05-29,Pasig City,Full Time,,Not Specified / In Description,Not Specified / In Description,JOB REQUIREMENTS: College Degree holder in any...,Not Specified,Not Specified,Not Specified,Not Specified,Science
4,Joblum,Environment Health and Safety Supervisor,Environmental,Flash Express,2021-05-29,Taguig City,Contract,"30,000-42,000",Not Specified / In Description,Not Specified / In Description,Establish and plan a training course in accord...,"30,000","42,000",Not Specified,Not Specified,Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,Joblum,Local Disaster Risk Reduction Management Offic...,Environmental,"Municipal Government of Malabang, Lanao del Sur",2021-04-30,Armm,Not Specified,"24,727",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 30, ...","24,727","24,727",Not Specified,Not Specified,Science
419,Joblum,Sanitation Inspector II,Environmental,"Municipal Government of Hungduan, Ifugao",2021-04-30,C.A.R,Not Specified,"18,251",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 7, 2...","18,251","18,251",Not Specified,Not Specified,Science
420,Joblum,Local Disaster Risk Reduction and Management O...,Environmental,"Municipal Government of Hungduan, Ifugao",2021-04-30,C.A.R,Not Specified,"17,908",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 7, 2...","17,908","17,908",Not Specified,Not Specified,Science
421,Joblum,Local Disaster Risk Reduction and Management A...,Environmental,"Municipal Government of Hungduan, Ifugao",2021-04-30,C.A.R,Not Specified,"13,688",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 7, 2...","13,688","13,688",Not Specified,Not Specified,Science


In [90]:
ENVI_df.to_csv ('JOBLUM-ENVI.csv', index = False)

In [81]:
#Scraping data of Environmental Engineering (JOBLUM) - FIRST HALF

ENVIENG_TITLE_FIRST = []
ENVIENG_COMPANY_FIRST = []
ENVIENG_DATE_FIRST = []
ENVIENG_LOCATION_FIRST = []
ENVIENG_STATUS_FIRST = []
ENVIENG_SALARY_FIRST = []
ENVIENG_SALARY_MIN_FIRST = []
ENVIENG_SALARY_MAX_FIRST = []
ENVIENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-environmental-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ENVIENG_TITLE_FIRST, ENVIENG_COMPANY_FIRST, ENVIENG_DATE_FIRST, 
              ENVIENG_STATUS_FIRST, ENVIENG_LOCATION_FIRST, ENVIENG_SALARY_FIRST, 
              ENVIENG_SALARY_MIN_FIRST, ENVIENG_SALARY_MAX_FIRST, ENVIENG_DESCRIPTION_FIRST)

In [82]:
#Scraping data of Environmental Engineering (JOBLUM) - SECOND HALF

ENVIENG_TITLE_SECOND = []
ENVIENG_COMPANY_SECOND = []
ENVIENG_DATE_SECOND = []
ENVIENG_LOCATION_SECOND = []
ENVIENG_STATUS_SECOND = []
ENVIENG_SALARY_SECOND = []
ENVIENG_SALARY_MIN_SECOND = []
ENVIENG_SALARY_MAX_SECOND = []
ENVIENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ENVIENG_TITLE_SECOND, ENVIENG_COMPANY_SECOND, ENVIENG_DATE_SECOND, 
              ENVIENG_STATUS_SECOND, ENVIENG_LOCATION_SECOND, ENVIENG_SALARY_SECOND, 
              ENVIENG_SALARY_MIN_SECOND, ENVIENG_SALARY_MAX_SECOND, ENVIENG_DESCRIPTION_FIRST)

In [83]:
#Joining Scrape data of Environmental Engineering (JOBLUM) 

ENVIENG_TITLE_LIST = np.concatenate((ENVIENG_TITLE_FIRST, ENVIENG_TITLE_SECOND))
ENVIENG_COMPANY_LIST = np.concatenate((ENVIENG_COMPANY_FIRST, ENVIENG_COMPANY_SECOND))
ENVIENG_DATE_LIST = np.concatenate((ENVIENG_DATE_FIRST, ENVIENG_DATE_SECOND))
ENVIENG_LOCATION_LIST = np.concatenate((ENVIENG_LOCATION_FIRST, ENVIENG_LOCATION_SECOND))
ENVIENG_STATUS_LIST = np.concatenate((ENVIENG_STATUS_FIRST, ENVIENG_STATUS_SECOND))
ENVIENG_SALARY_LIST = np.concatenate((ENVIENG_SALARY_FIRST, ENVIENG_SALARY_SECOND))
ENVIENG_SALARY_MIN_LIST = np.concatenate((ENVIENG_SALARY_MIN_FIRST, ENVIENG_SALARY_MIN_SECOND))
ENVIENG_SALARY_MAX_LIST = np.concatenate((ENVIENG_SALARY_MAX_FIRST, ENVIENG_SALARY_MAX_SECOND))
ENVIENG_DESCRIPTION_LIST = np.concatenate((ENVIENG_DESCRIPTION_FIRST, ENVIENG_DESCRIPTION_SECOND))

In [84]:
#Creating Data Frame for Environmental Engineering (JOBLUM) 
ENVIENG={'Website': "Joblum",
      'Job Title': ENVIENG_TITLE_LIST, 
      'Category': "Environmental Engineering", 
      'Company': ENVIENG_COMPANY_LIST, 
      'Date Posted': ENVIENG_DATE_LIST, 
      'Location': ENVIENG_LOCATION_LIST, 
      'Status': ENVIENG_STATUS_LIST, 
      'Salary': ENVIENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ENVIENG_DESCRIPTION_LIST,
      'Min Salary': ENVIENG_SALARY_MIN_LIST,
      'Max Salary': ENVIENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
ENVIENG_df = pd.DataFrame(data=ENVIENG)
ENVIENG_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Senior Ruby on Rails Engineer,Environmental Engineering,Axis Online Ventures Inc.,2021-05-26,Mandaluyong – WFH for now,Not Specified,"74,000-75,000",Not Specified / In Description,Not Specified / In Description,"Senior Ruby on Rails Engineer Product, Enginee...","74,000","75,000",Not Specified,Not Specified,Engineering


In [85]:
ENVIENG_df.to_csv ('JOBLUM-ENVIENG.csv', index = False)

In [76]:
#Scraping data of Food Tech/Nutritionist (JOBLUM) - FIRST HALF

NUTRI_TITLE_FIRST = []
NUTRI_COMPANY_FIRST = []
NUTRI_DATE_FIRST = []
NUTRI_LOCATION_FIRST = []
NUTRI_STATUS_FIRST = []
NUTRI_SALARY_FIRST = []
NUTRI_SALARY_MIN_FIRST = []
NUTRI_SALARY_MAX_FIRST = []
NUTRI_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-food-tech-nutritionist?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, NUTRI_TITLE_FIRST, NUTRI_COMPANY_FIRST, NUTRI_DATE_FIRST, 
              NUTRI_STATUS_FIRST, NUTRI_LOCATION_FIRST, NUTRI_SALARY_FIRST, 
              NUTRI_SALARY_MIN_FIRST, NUTRI_SALARY_MAX_FIRST, NUTRI_DESCRIPTION_FIRST)

In [77]:
#Scraping data of Food Tech/Nutritionist (JOBLUM) - SECOND HALF

NUTRI_TITLE_SECOND = []
NUTRI_COMPANY_SECOND = []
NUTRI_DATE_SECOND = []
NUTRI_LOCATION_SECOND = []
NUTRI_STATUS_SECOND = []
NUTRI_SALARY_SECOND = []
NUTRI_SALARY_MIN_SECOND = []
NUTRI_SALARY_MAX_SECOND = []
NUTRI_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, NUTRI_TITLE_SECOND, NUTRI_COMPANY_SECOND, NUTRI_DATE_SECOND, 
              NUTRI_STATUS_SECOND, NUTRI_LOCATION_SECOND, NUTRI_SALARY_SECOND, 
              NUTRI_SALARY_MIN_SECOND, NUTRI_SALARY_MAX_SECOND, NUTRI_DESCRIPTION_FIRST)

In [78]:
#Joining Scrape data of Food Tech/Nutritionist (JOBLUM) 

NUTRI_TITLE_LIST = np.concatenate((NUTRI_TITLE_FIRST, NUTRI_TITLE_SECOND))
NUTRI_COMPANY_LIST = np.concatenate((NUTRI_COMPANY_FIRST, NUTRI_COMPANY_SECOND))
NUTRI_DATE_LIST = np.concatenate((NUTRI_DATE_FIRST, NUTRI_DATE_SECOND))
NUTRI_LOCATION_LIST = np.concatenate((NUTRI_LOCATION_FIRST, NUTRI_LOCATION_SECOND))
NUTRI_STATUS_LIST = np.concatenate((NUTRI_STATUS_FIRST, NUTRI_STATUS_SECOND))
NUTRI_SALARY_LIST = np.concatenate((NUTRI_SALARY_FIRST, NUTRI_SALARY_SECOND))
NUTRI_SALARY_MIN_LIST = np.concatenate((NUTRI_SALARY_MIN_FIRST, NUTRI_SALARY_MIN_SECOND))
NUTRI_SALARY_MAX_LIST = np.concatenate((NUTRI_SALARY_MAX_FIRST, NUTRI_SALARY_MAX_SECOND))
NUTRI_DESCRIPTION_LIST = np.concatenate((NUTRI_DESCRIPTION_FIRST, NUTRI_DESCRIPTION_SECOND))

In [79]:
#Creating Data Frame for Food Tech/Nutritionist (JOBLUM) 
NUTRI={'Website': "Joblum",
      'Job Title': NUTRI_TITLE_LIST, 
      'Category': "Food Tech/Nutritionist", 
      'Company': NUTRI_COMPANY_LIST, 
      'Date Posted': NUTRI_DATE_LIST, 
      'Location': NUTRI_LOCATION_LIST, 
      'Status': NUTRI_STATUS_LIST, 
      'Salary': NUTRI_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': NUTRI_DESCRIPTION_LIST,
      'Min Salary': NUTRI_SALARY_MIN_LIST,
      'Max Salary': NUTRI_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Science"}
NUTRI_df = pd.DataFrame(data=NUTRI)
NUTRI_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,NUTRITIONIST DIETITIAN I,Food Tech/Nutritionist,Culion Sanitarium and General Hospital - Gover...,2021-05-30,Calabarzon & Mimaropa,Not Specified,"23,877",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 4, ...","23,877","23,877",Not Specified,Not Specified,Science
1,Joblum,FOOD TECHNOLOGIST,Food Tech/Nutritionist,VROSS CORPORATION (Butuan),2021-05-29,Agusan del Norte (Butuan),OJT,,Not Specified / In Description,Not Specified / In Description,excellent written and oral communication skill...,Not Specified,Not Specified,Not Specified,Not Specified,Science
2,Joblum,Food Technologist,Food Tech/Nutritionist,Proyoung Wellness Clinic,2021-05-29,Quezon City,OJT,"14,000-16,000",Not Specified / In Description,Not Specified / In Description,Job Description Graduate of a 4-year course in...,"14,000","16,000",Not Specified,Not Specified,Science
3,Joblum,R&D Manager,Food Tech/Nutritionist,Dizon Farms Produce Inc.,2021-05-29,Taguig City,OJT,,Not Specified / In Description,Not Specified / In Description,Duties R&D manager is responsible for overseei...,Not Specified,Not Specified,Not Specified,Not Specified,Science
4,Joblum,SANITATION INSPECTOR I,Food Tech/Nutritionist,"Municipal Government of Sumilao, Bukidnon",2021-05-29,Northern Mindanao,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : JUNE 7,2...",Not Specified,Not Specified,Not Specified,Not Specified,Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Joblum,Research & Development Specialist / Pastry Chef,Food Tech/Nutritionist,Tri-Vision Ventures Inc,2021-04-30,Pasig City,Contract,"17,500-24,500",Not Specified / In Description,Not Specified / In Description,Overall Responsibility: Responsible for creati...,"17,500","24,500",Not Specified,Not Specified,Science
147,Joblum,Nutritionist-Dietician II,Food Tech/Nutritionist,Provincial Government of Albay - Government,2021-04-30,Bicol Region,OJT,"33,575",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 7, 2...","33,575","33,575",Not Specified,Not Specified,Science
148,Joblum,"Research and Development Specialist - Calamba,...",Food Tech/Nutritionist,Bounty Agro Ventures Inc.,2021-04-30,Laguna (Calamba City),Contract,,Not Specified / In Description,Not Specified / In Description,Qualifications: Candidate must possess Bachelo...,Not Specified,Not Specified,Not Specified,Not Specified,Science
149,Joblum,Sanitation Inspector II,Food Tech/Nutritionist,"Municipal Government of Tantangan, South Cotab...",2021-04-30,Soccsksargen,Not Specified,"18,251",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 10, ...","18,251","18,251",Not Specified,Not Specified,Science


In [80]:
NUTRI_df.to_csv ('JOBLUM-NUTRI.csv', index = False)

In [71]:
#Scraping data of Geology/Geophysics (JOBLUM) - FIRST HALF

GEO_TITLE_FIRST = []
GEO_COMPANY_FIRST = []
GEO_DATE_FIRST = []
GEO_LOCATION_FIRST = []
GEO_STATUS_FIRST = []
GEO_SALARY_FIRST = []
GEO_SALARY_MIN_FIRST = []
GEO_SALARY_MAX_FIRST = []
GEO_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-geology-geophysics?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, GEO_TITLE_FIRST, GEO_COMPANY_FIRST, GEO_DATE_FIRST, 
              GEO_STATUS_FIRST, GEO_LOCATION_FIRST, GEO_SALARY_FIRST, 
              GEO_SALARY_MIN_FIRST, GEO_SALARY_MAX_FIRST, GEO_DESCRIPTION_FIRST)

In [72]:
#Scraping data of Geology/Geophysics (JOBLUM) - SECOND HALF

GEO_TITLE_SECOND = []
GEO_COMPANY_SECOND = []
GEO_DATE_SECOND = []
GEO_LOCATION_SECOND = []
GEO_STATUS_SECOND = []
GEO_SALARY_SECOND = []
GEO_SALARY_MIN_SECOND = []
GEO_SALARY_MAX_SECOND = []
GEO_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, GEO_TITLE_SECOND, GEO_COMPANY_SECOND, GEO_DATE_SECOND, 
              GEO_STATUS_SECOND, GEO_LOCATION_SECOND, GEO_SALARY_SECOND, 
              GEO_SALARY_MIN_SECOND, GEO_SALARY_MAX_SECOND, GEO_DESCRIPTION_FIRST)

In [73]:
#Joining Scrape data of Geology/Geophysics (JOBLUM) 

GEO_TITLE_LIST = np.concatenate((GEO_TITLE_FIRST, GEO_TITLE_SECOND))
GEO_COMPANY_LIST = np.concatenate((GEO_COMPANY_FIRST, GEO_COMPANY_SECOND))
GEO_DATE_LIST = np.concatenate((GEO_DATE_FIRST, GEO_DATE_SECOND))
GEO_LOCATION_LIST = np.concatenate((GEO_LOCATION_FIRST, GEO_LOCATION_SECOND))
GEO_STATUS_LIST = np.concatenate((GEO_STATUS_FIRST, GEO_STATUS_SECOND))
GEO_SALARY_LIST = np.concatenate((GEO_SALARY_FIRST, GEO_SALARY_SECOND))
GEO_SALARY_MIN_LIST = np.concatenate((GEO_SALARY_MIN_FIRST, GEO_SALARY_MIN_SECOND))
GEO_SALARY_MAX_LIST = np.concatenate((GEO_SALARY_MAX_FIRST, GEO_SALARY_MAX_SECOND))
GEO_DESCRIPTION_LIST = np.concatenate((GEO_DESCRIPTION_FIRST, GEO_DESCRIPTION_SECOND))

In [74]:
#Creating Data Frame for Geology/Geophysics (JOBLUM) 
GEO={'Website': "Joblum",
      'Job Title': GEO_TITLE_LIST, 
      'Category': "Geology/Geophysics", 
      'Company': GEO_COMPANY_LIST, 
      'Date Posted': GEO_DATE_LIST, 
      'Location': GEO_LOCATION_LIST, 
      'Status': GEO_STATUS_LIST, 
      'Salary': GEO_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': GEO_DESCRIPTION_LIST,
      'Min Salary': GEO_SALARY_MIN_LIST,
      'Max Salary': GEO_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Science"}
GEO_df = pd.DataFrame(data=GEO)
GEO_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Hydrogeologist,Geology/Geophysics,PrimeWater Infrastructure Corp.,2021-05-29,National Capital Reg,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,Science
1,Joblum,Senior Geologist,Geology/Geophysics,Mines and Geosciences Bureau Region IV-A Calab...,2021-05-28,Calabarzon & Mimaropa,Not Specified,"43,681",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 3, ...","43,681","43,681",Not Specified,Not Specified,Science
2,Joblum,QC/Chemical Engineering/Geochemist (Back Office),Geology/Geophysics,SGS,2021-05-25,Muntinlupa City,OJT,,Not Specified / In Description,Not Specified / In Description,"Data Analyst, Geochemistry PRIMARY RESPONSIBIL...",Not Specified,Not Specified,Not Specified,Not Specified,Science
3,Joblum,LAND ACQUISITION OFFICER,Geology/Geophysics,"Sagittarius Mines, Inc.",2021-05-23,South Cotabato (Others),Contract,,Not Specified / In Description,Not Specified / In Description,This is a 1-year project employment based in T...,Not Specified,Not Specified,Not Specified,Not Specified,Science
4,Joblum,Junior GIS Specialist,Geology/Geophysics,Solar Philippines,2021-05-23,Makati City,OJT,,Not Specified / In Description,Not Specified / In Description,Qualifications: Excellent communication skills...,Not Specified,Not Specified,Not Specified,Not Specified,Science
5,Joblum,GIS Database Administrator (Asset Info Systems...,Geology/Geophysics,"Manila Water Company, Inc.",2021-05-19,Quezon City,Full Time,,Not Specified / In Description,Not Specified / In Description,Job Responsibilities Regulatory compliance • P...,Not Specified,Not Specified,Not Specified,Not Specified,Science
6,Joblum,History Researcher I,Geology/Geophysics,Philippine Veterans Affairs Office - Government,2021-05-14,National Capital Reg,Not Specified,"21,205",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 20, ...","21,205","21,205",Not Specified,Not Specified,Science
7,Joblum,GIS Data Analyst,Geology/Geophysics,ShareSource Australia,2021-05-14,Makati City,Not Specified,"18,000-25,000",Not Specified / In Description,Not Specified / In Description,OUR CLIENT: Our client is a fast-growing B2B S...,"18,000","25,000",Not Specified,Not Specified,Science
8,Joblum,History Researcher I,Geology/Geophysics,Philippine Veterans Affairs Office - Government,2021-05-12,National Capital Reg,Not Specified,"21,205",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 20, ...","21,205","21,205",Not Specified,Not Specified,Science
9,Joblum,Engineer II (132-2014),Geology/Geophysics,Department of Agrarian Reform Region II,2021-05-08,Cagayan Valley,Not Specified,"36,628",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 12, ...","36,628","36,628",Not Specified,Not Specified,Science


In [75]:
GEO_df.to_csv ('JOBLUM-GEO.csv', index = False)

In [66]:
#Scraping data of Industrial Engineering (JOBLUM) - FIRST HALF

INDUSENG_TITLE_FIRST = []
INDUSENG_COMPANY_FIRST = []
INDUSENG_DATE_FIRST = []
INDUSENG_LOCATION_FIRST = []
INDUSENG_STATUS_FIRST = []
INDUSENG_SALARY_FIRST = []
INDUSENG_SALARY_MIN_FIRST = []
INDUSENG_SALARY_MAX_FIRST = []
INDUSENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-industrial-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, INDUSENG_TITLE_FIRST, INDUSENG_COMPANY_FIRST, INDUSENG_DATE_FIRST, 
              INDUSENG_STATUS_FIRST, INDUSENG_LOCATION_FIRST, INDUSENG_SALARY_FIRST, 
              INDUSENG_SALARY_MIN_FIRST, INDUSENG_SALARY_MAX_FIRST, INDUSENG_DESCRIPTION_FIRST)

In [67]:
#Scraping data of Industrial Engineering (JOBLUM) - SECOND HALF

INDUSENG_TITLE_SECOND = []
INDUSENG_COMPANY_SECOND = []
INDUSENG_DATE_SECOND = []
INDUSENG_LOCATION_SECOND = []
INDUSENG_STATUS_SECOND = []
INDUSENG_SALARY_SECOND = []
INDUSENG_SALARY_MIN_SECOND = []
INDUSENG_SALARY_MAX_SECOND = []
INDUSENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, INDUSENG_TITLE_SECOND, INDUSENG_COMPANY_SECOND, INDUSENG_DATE_SECOND, 
              INDUSENG_STATUS_SECOND, INDUSENG_LOCATION_SECOND, INDUSENG_SALARY_SECOND, 
              INDUSENG_SALARY_MIN_SECOND, INDUSENG_SALARY_MAX_SECOND, INDUSENG_DESCRIPTION_FIRST)

In [68]:
#Joining Scrape data of Industrial Engineering (JOBLUM) 

INDUSENG_TITLE_LIST = np.concatenate((INDUSENG_TITLE_FIRST, INDUSENG_TITLE_SECOND))
INDUSENG_COMPANY_LIST = np.concatenate((INDUSENG_COMPANY_FIRST, INDUSENG_COMPANY_SECOND))
INDUSENG_DATE_LIST = np.concatenate((INDUSENG_DATE_FIRST, INDUSENG_DATE_SECOND))
INDUSENG_LOCATION_LIST = np.concatenate((INDUSENG_LOCATION_FIRST, INDUSENG_LOCATION_SECOND))
INDUSENG_STATUS_LIST = np.concatenate((INDUSENG_STATUS_FIRST, INDUSENG_STATUS_SECOND))
INDUSENG_SALARY_LIST = np.concatenate((INDUSENG_SALARY_FIRST, INDUSENG_SALARY_SECOND))
INDUSENG_SALARY_MIN_LIST = np.concatenate((INDUSENG_SALARY_MIN_FIRST, INDUSENG_SALARY_MIN_SECOND))
INDUSENG_SALARY_MAX_LIST = np.concatenate((INDUSENG_SALARY_MAX_FIRST, INDUSENG_SALARY_MAX_SECOND))
INDUSENG_DESCRIPTION_LIST = np.concatenate((INDUSENG_DESCRIPTION_FIRST, INDUSENG_DESCRIPTION_SECOND))

In [69]:
#Creating Data Frame for Industrial Engineering (JOBLUM) 
INDUSENG={'Website': "Joblum",
      'Job Title': INDUSENG_TITLE_LIST, 
      'Category': "Industrial Engineering", 
      'Company': INDUSENG_COMPANY_LIST, 
      'Date Posted': INDUSENG_DATE_LIST, 
      'Location': INDUSENG_LOCATION_LIST, 
      'Status': INDUSENG_STATUS_LIST, 
      'Salary': INDUSENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': INDUSENG_DESCRIPTION_LIST,
      'Min Salary': INDUSENG_SALARY_MIN_LIST,
      'Max Salary': INDUSENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
INDUSENG_df = pd.DataFrame(data=INDUSENG)
INDUSENG_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Industrial Engineer,Industrial Engineering,Amkor Technology Phils. Inc.,2021-05-30,National Capital Reg,Full Time,"21,000-30,000",Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,"21,000","30,000",Not Specified,Not Specified,Engineering
1,Joblum,Logistics Business Planning and Control Jr. E...,Industrial Engineering,"Honda Philippines, Inc.",2021-05-29,Batangas,Full Time,,Not Specified / In Description,Not Specified / In Description,JOB SUMMARY: Successful candidates will focus ...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
2,Joblum,Manufactuirng and Process Engineer,Industrial Engineering,SPT (PHIL.) CLARK CORPORATION,2021-05-29,Pampanga (Angeles City),OJT,"18,000-23,400",Not Specified / In Description,Not Specified / In Description,JOB DUTIES AND RESPONSIBILITIES 1. Conceptuali...,"18,000","23,400",Not Specified,Not Specified,Engineering
3,Joblum,Restaurant Systems Officer (Products and Proce...,Industrial Engineering,JFC Affiliates and Subsidiaries,2021-05-29,Pasig City,OJT,,Not Specified / In Description,Not Specified / In Description,This Restaurant Systems Officer is responsible...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
4,Joblum,Planner,Industrial Engineering,Amkor Technology Phils. Inc.,2021-05-29,National Capital Reg,Not Specified,,Not Specified / In Description,Not Specified / In Description,Graduate of Industrial Engineering or any equi...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Joblum,Business Process Management Officer - Head Office,Industrial Engineering,SM Supermalls (Shopping Center Management Corp...,2021-04-30,National Capital Reg,OJT,,Not Specified / In Description,Not Specified / In Description,The Business Process Management Officer will p...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
190,Joblum,Parts Analyst,Industrial Engineering,Fast Autoworld Philippines Corporation - Mitsu...,2021-04-30,"Cebu (Cebu City), Cebu (Others)",Not Specified,,Not Specified / In Description,Not Specified / In Description,Fast Autoworld Philippines Corporation is curr...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
191,Joblum,IE SPECIALIST,Industrial Engineering,"Century Pacific Food, Inc.",2021-04-30,Zamboanga Del Sur (Others),OJT,"15,000-21,000",Not Specified / In Description,Not Specified / In Description,Preferably Industrial Engineering Previous exp...,"15,000","21,000",Not Specified,Not Specified,Engineering
192,Joblum,Space Planner Assistant Manager,Industrial Engineering,"Watsons Personal Care Stores (Philippines), Inc.",2021-04-30,National Capital Reg,Not Specified,,Not Specified / In Description,Not Specified / In Description,The Space Planner’s general objective is to en...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering


In [70]:
INDUSENG_df.to_csv ('JOBLUM-INDUSENG.csv', index = False)

In [61]:
#Scraping data of IT - Hardware (JOBLUM) - FIRST HALF

IT_HARDWARE_TITLE_FIRST = []
IT_HARDWARE_COMPANY_FIRST = []
IT_HARDWARE_DATE_FIRST = []
IT_HARDWARE_LOCATION_FIRST = []
IT_HARDWARE_STATUS_FIRST = []
IT_HARDWARE_SALARY_FIRST = []
IT_HARDWARE_SALARY_MIN_FIRST = []
IT_HARDWARE_SALARY_MAX_FIRST = []
IT_HARDWARE_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-it-hardware?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, IT_HARDWARE_TITLE_FIRST, IT_HARDWARE_COMPANY_FIRST, IT_HARDWARE_DATE_FIRST, 
              IT_HARDWARE_STATUS_FIRST, IT_HARDWARE_LOCATION_FIRST, IT_HARDWARE_SALARY_FIRST, 
              IT_HARDWARE_SALARY_MIN_FIRST, IT_HARDWARE_SALARY_MAX_FIRST, IT_HARDWARE_DESCRIPTION_FIRST)

In [62]:
#Scraping data of IT - Hardware (JOBLUM) - SECOND HALF

IT_HARDWARE_TITLE_SECOND = []
IT_HARDWARE_COMPANY_SECOND = []
IT_HARDWARE_DATE_SECOND = []
IT_HARDWARE_LOCATION_SECOND = []
IT_HARDWARE_STATUS_SECOND = []
IT_HARDWARE_SALARY_SECOND = []
IT_HARDWARE_SALARY_MIN_SECOND = []
IT_HARDWARE_SALARY_MAX_SECOND = []
IT_HARDWARE_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, IT_HARDWARE_TITLE_SECOND, IT_HARDWARE_COMPANY_SECOND, IT_HARDWARE_DATE_SECOND, 
              IT_HARDWARE_STATUS_SECOND, IT_HARDWARE_LOCATION_SECOND, IT_HARDWARE_SALARY_SECOND, 
              IT_HARDWARE_SALARY_MIN_SECOND, IT_HARDWARE_SALARY_MAX_SECOND, IT_HARDWARE_DESCRIPTION_FIRST)

In [63]:
#Joining Scrape data of IT - Hardware (JOBLUM) 

IT_HARDWARE_TITLE_LIST = np.concatenate((IT_HARDWARE_TITLE_FIRST, IT_HARDWARE_TITLE_SECOND))
IT_HARDWARE_COMPANY_LIST = np.concatenate((IT_HARDWARE_COMPANY_FIRST, IT_HARDWARE_COMPANY_SECOND))
IT_HARDWARE_DATE_LIST = np.concatenate((IT_HARDWARE_DATE_FIRST, IT_HARDWARE_DATE_SECOND))
IT_HARDWARE_LOCATION_LIST = np.concatenate((IT_HARDWARE_LOCATION_FIRST, IT_HARDWARE_LOCATION_SECOND))
IT_HARDWARE_STATUS_LIST = np.concatenate((IT_HARDWARE_STATUS_FIRST, IT_HARDWARE_STATUS_SECOND))
IT_HARDWARE_SALARY_LIST = np.concatenate((IT_HARDWARE_SALARY_FIRST, IT_HARDWARE_SALARY_SECOND))
IT_HARDWARE_SALARY_MIN_LIST = np.concatenate((IT_HARDWARE_SALARY_MIN_FIRST, IT_HARDWARE_SALARY_MIN_SECOND))
IT_HARDWARE_SALARY_MAX_LIST = np.concatenate((IT_HARDWARE_SALARY_MAX_FIRST, IT_HARDWARE_SALARY_MAX_SECOND))
IT_HARDWARE_DESCRIPTION_LIST = np.concatenate((IT_HARDWARE_DESCRIPTION_FIRST, IT_HARDWARE_DESCRIPTION_SECOND))

In [64]:
#Creating Data Frame for IT - Hardware (JOBLUM) 
IT_HARDWARE={'Website': "Joblum",
      'Job Title': IT_HARDWARE_TITLE_LIST, 
      'Category': "IT - Hardware", 
      'Company': IT_HARDWARE_COMPANY_LIST, 
      'Date Posted': IT_HARDWARE_DATE_LIST, 
      'Location': IT_HARDWARE_LOCATION_LIST, 
      'Status': IT_HARDWARE_STATUS_LIST, 
      'Salary': IT_HARDWARE_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': IT_HARDWARE_DESCRIPTION_LIST,
      'Min Salary': IT_HARDWARE_SALARY_MIN_LIST,
      'Max Salary': IT_HARDWARE_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "IT"}
IT_HARDWARE_df = pd.DataFrame(data=IT_HARDWARE)
IT_HARDWARE_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,DB Operations,IT - Hardware,HCL Technologies Philippines Inc,2021-05-30,National Capital Reg,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,IT
1,Joblum,Middleware Operations,IT - Hardware,HCL Technologies Philippines Inc,2021-05-30,National Capital Reg,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,IT
2,Joblum,Infrastructure Operator Support,IT - Hardware,HCL Technologies Philippines Inc,2021-05-30,National Capital Reg,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,IT
3,Joblum,Head of Customer Support,IT - Hardware,"Sysgen RPO, Inc.",2021-05-30,"Bicol Region, C.A.R, Cagayan Valley, Central L...",Not Specified,,Not Specified / In Description,Not Specified / In Description,Job Scope Customer Success is vital to our lon...,Not Specified,Not Specified,Not Specified,Not Specified,IT
4,Joblum,TECHNICAL SUPPORT FOR ECOMMERCE,IT - Hardware,Beyond I. Tech Ventures Corporation,2021-05-29,Quezon City,Not Specified,"13,000-16,000",Not Specified / In Description,Not Specified / In Description,GameOne is currently the brainchild and main b...,"13,000","16,000",Not Specified,Not Specified,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Joblum,URGENT HIRING at VXI QC North EDSA! Earn as mu...,IT - Hardware,VXI Global Holdings B.V. (Philippines) - Quezo...,2021-04-30,National Capital Reg,Not Specified,"20,000-34,000",Not Specified / In Description,Not Specified / In Description,APPLY TODAY and earn up to 34K monthly!Qualifi...,"20,000","34,000",Not Specified,Not Specified,IT
410,Joblum,SYKES Priority Hour | 50% Non-Voice Tech Suppo...,IT - Hardware,Sykes Philippines,2021-04-30,National Capital Reg,OJT,,Not Specified / In Description,Not Specified / In Description,Want to have your application prioritized? App...,Not Specified,Not Specified,Not Specified,Not Specified,IT
411,Joblum,Computer Maintenance Technologist I (10-2016),IT - Hardware,NATIONAL MUSEUM OF THE PHILIPPINES - Government,2021-04-30,National Capital Reg,Full Time,"23,877",Not Specified / In Description,Not Specified / In Description,Deadline for accepting applications : 26 May 2...,"23,877","23,877",Not Specified,Not Specified,IT
412,Joblum,Computer Maintenance Technologist I (5-2007),IT - Hardware,NATIONAL MUSEUM OF THE PHILIPPINES - Government,2021-04-30,National Capital Reg,Full Time,"23,877",Not Specified / In Description,Not Specified / In Description,Deadline for accepting applications : 26 May 2...,"23,877","23,877",Not Specified,Not Specified,IT


In [65]:
IT_HARDWARE_df.to_csv ('JOBLUM-IT_HARDWARE.csv', index = False)

In [56]:
#Scraping data of IT - Network/Sys/DB Admin (JOBLUM) - FIRST HALF

IT_SYS_TITLE_FIRST = []
IT_SYS_COMPANY_FIRST = []
IT_SYS_DATE_FIRST = []
IT_SYS_LOCATION_FIRST = []
IT_SYS_STATUS_FIRST = []
IT_SYS_SALARY_FIRST = []
IT_SYS_SALARY_MIN_FIRST = []
IT_SYS_SALARY_MAX_FIRST = []
IT_SYS_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-it-network-sys-db-admin?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, IT_SYS_TITLE_FIRST, IT_SYS_COMPANY_FIRST, IT_SYS_DATE_FIRST, 
              IT_SYS_STATUS_FIRST, IT_SYS_LOCATION_FIRST, IT_SYS_SALARY_FIRST, 
              IT_SYS_SALARY_MIN_FIRST, IT_SYS_SALARY_MAX_FIRST, IT_SYS_DESCRIPTION_FIRST)

In [57]:
#Scraping data of IT - Network/Sys/DB Admin (JOBLUM) - SECOND HALF

IT_SYS_TITLE_SECOND = []
IT_SYS_COMPANY_SECOND = []
IT_SYS_DATE_SECOND = []
IT_SYS_LOCATION_SECOND = []
IT_SYS_STATUS_SECOND = []
IT_SYS_SALARY_SECOND = []
IT_SYS_SALARY_MIN_SECOND = []
IT_SYS_SALARY_MAX_SECOND = []
IT_SYS_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, IT_SYS_TITLE_SECOND, IT_SYS_COMPANY_SECOND, IT_SYS_DATE_SECOND, 
              IT_SYS_STATUS_SECOND, IT_SYS_LOCATION_SECOND, IT_SYS_SALARY_SECOND, 
              IT_SYS_SALARY_MIN_SECOND, IT_SYS_SALARY_MAX_SECOND, IT_SYS_DESCRIPTION_FIRST)

In [58]:
#Joining Scrape data of IT - Network/Sys/DB Admin (JOBLUM) 

IT_SYS_TITLE_LIST = np.concatenate((IT_SYS_TITLE_FIRST, IT_SYS_TITLE_SECOND))
IT_SYS_COMPANY_LIST = np.concatenate((IT_SYS_COMPANY_FIRST, IT_SYS_COMPANY_SECOND))
IT_SYS_DATE_LIST = np.concatenate((IT_SYS_DATE_FIRST, IT_SYS_DATE_SECOND))
IT_SYS_LOCATION_LIST = np.concatenate((IT_SYS_LOCATION_FIRST, IT_SYS_LOCATION_SECOND))
IT_SYS_STATUS_LIST = np.concatenate((IT_SYS_STATUS_FIRST, IT_SYS_STATUS_SECOND))
IT_SYS_SALARY_LIST = np.concatenate((IT_SYS_SALARY_FIRST, IT_SYS_SALARY_SECOND))
IT_SYS_SALARY_MIN_LIST = np.concatenate((IT_SYS_SALARY_MIN_FIRST, IT_SYS_SALARY_MIN_SECOND))
IT_SYS_SALARY_MAX_LIST = np.concatenate((IT_SYS_SALARY_MAX_FIRST, IT_SYS_SALARY_MAX_SECOND))
IT_SYS_DESCRIPTION_LIST = np.concatenate((IT_SYS_DESCRIPTION_FIRST, IT_SYS_DESCRIPTION_SECOND))

In [59]:
#Creating Data Frame for IT - Network/Sys/DB Admin (JOBLUM) 
IT_SYS={'Website': "Joblum",
      'Job Title': IT_SYS_TITLE_LIST, 
      'Category': "IT - Network/Sys/DB Admin", 
      'Company': IT_SYS_COMPANY_LIST, 
      'Date Posted': IT_SYS_DATE_LIST, 
      'Location': IT_SYS_LOCATION_LIST, 
      'Status': IT_SYS_STATUS_LIST, 
      'Salary': IT_SYS_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': IT_SYS_DESCRIPTION_LIST,
      'Min Salary': IT_SYS_SALARY_MIN_LIST,
      'Max Salary': IT_SYS_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "IT"}
IT_SYS_df = pd.DataFrame(data=IT_SYS)
IT_SYS_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Business Development Officer,IT - Network/Sys/DB Admin,Axis Online Ventures Inc.,2021-05-26,Quezon City,Not Specified,"35,000-50,000",Not Specified / In Description,Not Specified / In Description,Graduate of any 4-year course or college degre...,"35,000","50,000",Not Specified,Not Specified,IT
1,Joblum,Senior Microsoft Dynamics Admin/ Developer,IT - Network/Sys/DB Admin,Axis Online Ventures Inc.,2021-05-26,Pasig City - WFH for now,Contract,"55,000-60,000",Not Specified / In Description,Not Specified / In Description,2+ years’ experience with Microsoft Dynamics/P...,"55,000","60,000",Not Specified,Not Specified,IT
2,Joblum,Level 2 Network Engineer/ Global Service Devic...,IT - Network/Sys/DB Admin,Axis Online Ventures Inc.,2021-05-26,Makati but WFH for now,Contract,"50,000-65,000",Not Specified / In Description,Not Specified / In Description,Actively working tickets in the Global SDM ENG...,"50,000","65,000",Not Specified,Not Specified,IT
3,Joblum,USRN Nurse Associate,IT - Network/Sys/DB Admin,Axis Online Ventures Inc.,2021-05-26,BGC Taguig,Not Specified,"30,000-50,000",Not Specified / In Description,Not Specified / In Description,Must be an NCLEX Passer with active US Mainlan...,"30,000","50,000",Not Specified,Not Specified,IT
4,Joblum,Systems Engineer,IT - Network/Sys/DB Admin,Axis Online Ventures Inc.,2021-05-26,can be anywhere,Not Specified,"129,000-130,000",Not Specified / In Description,Not Specified / In Description,"Java- Spring Boot, MongoDB, Kafka, WSO2, and o...","129,000","130,000",Not Specified,Not Specified,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,Joblum,Systems Administrator,IT - Network/Sys/DB Admin,Southern Philippines College,2021-04-29,Misamis Oriental (CDO),Not Specified,,Not Specified / In Description,Not Specified / In Description,Qualifications: Bachelor's degree in Informati...,Not Specified,Not Specified,Not Specified,Not Specified,IT
2605,Joblum,Data Reports Analyst,IT - Network/Sys/DB Admin,VXI Global Holdings B.V. (Philippines) - Quezo...,2021-04-29,Quezon City,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Duties and Responsibilities: Generates, collat...",Not Specified,Not Specified,Not Specified,Not Specified,IT
2606,Joblum,QA Engineer (Telco / Network),IT - Network/Sys/DB Admin,Asticom Technology Inc,2021-04-29,National Capital Reg,Not Specified,,Not Specified / In Description,Not Specified / In Description,Job Description Draft quality assurance polici...,Not Specified,Not Specified,Not Specified,Not Specified,IT
2607,Joblum,"Reports Developer Senior Analyst | SQL, SSIS &...",IT - Network/Sys/DB Admin,Accenture,2021-04-29,"Central Luzon, National Capital Reg, Calabarzo...",Not Specified,,Not Specified / In Description,Not Specified / In Description,Go through a faster and more convenient recrui...,Not Specified,Not Specified,Not Specified,Not Specified,IT


In [60]:
IT_SYS_df.to_csv ('JOBLUM-IT_SYS.csv', index = False)

In [50]:
#Scraping data of IT - Software (JOBLUM) - FIRST HALF

IT_SOFTWARE_TITLE_FIRST = []
IT_SOFTWARE_COMPANY_FIRST = []
IT_SOFTWARE_DATE_FIRST = []
IT_SOFTWARE_LOCATION_FIRST = []
IT_SOFTWARE_STATUS_FIRST = []
IT_SOFTWARE_SALARY_FIRST = []
IT_SOFTWARE_SALARY_MIN_FIRST = []
IT_SOFTWARE_SALARY_MAX_FIRST = []
IT_SOFTWARE_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-it-software?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, IT_SOFTWARE_TITLE_FIRST, IT_SOFTWARE_COMPANY_FIRST, IT_SOFTWARE_DATE_FIRST, 
              IT_SOFTWARE_STATUS_FIRST, IT_SOFTWARE_LOCATION_FIRST, IT_SOFTWARE_SALARY_FIRST, 
              IT_SOFTWARE_SALARY_MIN_FIRST, IT_SOFTWARE_SALARY_MAX_FIRST, IT_SOFTWARE_DESCRIPTION_FIRST)

In [51]:
#Scraping data of IT - Software (JOBLUM) - SECOND HALF

IT_SOFTWARE_TITLE_SECOND = []
IT_SOFTWARE_COMPANY_SECOND = []
IT_SOFTWARE_DATE_SECOND = []
IT_SOFTWARE_LOCATION_SECOND = []
IT_SOFTWARE_STATUS_SECOND = []
IT_SOFTWARE_SALARY_SECOND = []
IT_SOFTWARE_SALARY_MIN_SECOND = []
IT_SOFTWARE_SALARY_MAX_SECOND = []
IT_SOFTWARE_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, IT_SOFTWARE_TITLE_SECOND, IT_SOFTWARE_COMPANY_SECOND, IT_SOFTWARE_DATE_SECOND, 
              IT_SOFTWARE_STATUS_SECOND, IT_SOFTWARE_LOCATION_SECOND, IT_SOFTWARE_SALARY_SECOND, 
              IT_SOFTWARE_SALARY_MIN_SECOND, IT_SOFTWARE_SALARY_MAX_SECOND, IT_SOFTWARE_DESCRIPTION_FIRST)

In [52]:
#Joining Scrape data of IT - Software (JOBLUM) 

IT_SOFTWARE_TITLE_LIST = np.concatenate((IT_SOFTWARE_TITLE_FIRST, IT_SOFTWARE_TITLE_SECOND))
IT_SOFTWARE_COMPANY_LIST = np.concatenate((IT_SOFTWARE_COMPANY_FIRST, IT_SOFTWARE_COMPANY_SECOND))
IT_SOFTWARE_DATE_LIST = np.concatenate((IT_SOFTWARE_DATE_FIRST, IT_SOFTWARE_DATE_SECOND))
IT_SOFTWARE_LOCATION_LIST = np.concatenate((IT_SOFTWARE_LOCATION_FIRST, IT_SOFTWARE_LOCATION_SECOND))
IT_SOFTWARE_STATUS_LIST = np.concatenate((IT_SOFTWARE_STATUS_FIRST, IT_SOFTWARE_STATUS_SECOND))
IT_SOFTWARE_SALARY_LIST = np.concatenate((IT_SOFTWARE_SALARY_FIRST, IT_SOFTWARE_SALARY_SECOND))
IT_SOFTWARE_SALARY_MIN_LIST = np.concatenate((IT_SOFTWARE_SALARY_MIN_FIRST, IT_SOFTWARE_SALARY_MIN_SECOND))
IT_SOFTWARE_SALARY_MAX_LIST = np.concatenate((IT_SOFTWARE_SALARY_MAX_FIRST, IT_SOFTWARE_SALARY_MAX_SECOND))
IT_SOFTWARE_DESCRIPTION_LIST = np.concatenate((IT_SOFTWARE_DESCRIPTION_FIRST, IT_SOFTWARE_DESCRIPTION_SECOND))

In [53]:
#Creating Data Frame for IT - Software (JOBLUM) 
IT_SOFTWARE={'Website': "Joblum",
      'Job Title': IT_SOFTWARE_TITLE_LIST, 
      'Category': "IT - Software", 
      'Company': IT_SOFTWARE_COMPANY_LIST, 
      'Date Posted': IT_SOFTWARE_DATE_LIST, 
      'Location': IT_SOFTWARE_LOCATION_LIST, 
      'Status': IT_SOFTWARE_STATUS_LIST, 
      'Salary': IT_SOFTWARE_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': IT_SOFTWARE_DESCRIPTION_LIST,
      'Min Salary': IT_SOFTWARE_SALARY_MIN_LIST,
      'Max Salary': IT_SOFTWARE_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "IT"}
IT_SOFTWARE_df = pd.DataFrame(data=IT_SOFTWARE)
IT_SOFTWARE_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,SAP and Salesforce IT HD,IT - Software,Axis Online Ventures Inc.,2021-05-27,WFH,Not Specified,"30,000-45,000",Not Specified / In Description,Not Specified / In Description,Degree in Computer Science or Information Syst...,"30,000","45,000",Not Specified,Not Specified,IT
1,Joblum,NOC Tier II,IT - Software,Axis Online Ventures Inc.,2021-05-27,WFH,Contract,"35,000-55,000",Not Specified / In Description,Not Specified / In Description,"Degree in IT, Engineering, Computer science, I...","35,000","55,000",Not Specified,Not Specified,IT
2,Joblum,Automation Test Analyst,IT - Software,Axis Online Ventures Inc.,2021-05-27,Makati,Not Specified,"90,000-150,000",Not Specified / In Description,Not Specified / In Description,5 years of experience as an Automation Softwar...,"90,000","150,000",Not Specified,Not Specified,IT
3,Joblum,WordPress Developer,IT - Software,Axis Online Ventures Inc.,2021-05-27,WFH,Part Time,"44,000-45,000",Not Specified / In Description,Not Specified / In Description,2-3 years experience in Web Development (CMS: ...,"44,000","45,000",Not Specified,Not Specified,IT
4,Joblum,Sales Associates,IT - Software,CUROGRAM,2021-05-27,Cebu City,Not Specified,"15,000-17,000",Not Specified / In Description,Not Specified / In Description,Job Description Duties and Responsibilities: M...,"15,000","17,000",Not Specified,Not Specified,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7064,Joblum,Temp WFH - ASP.Net MVC Developer,IT - Software,247.ai Philippines,2021-04-29,National Capital Reg,Contract,,Not Specified / In Description,Not Specified / In Description,QUALIFICATIONS: • Bachelor’s Degree in Informa...,Not Specified,Not Specified,Not Specified,Not Specified,IT
7065,Joblum,Sr. IT Business Analyst,IT - Software,Asticom Technology Inc,2021-04-29,National Capital Reg,OJT,,Not Specified / In Description,Not Specified / In Description,The RequirementsBachelor’s Degree in Informat...,Not Specified,Not Specified,Not Specified,Not Specified,IT
7066,Joblum,Customer Experience Officer (Visual Design Foc...,IT - Software,Universal Robina Corporation,2021-04-29,National Capital Reg,OJT,,Not Specified / In Description,Not Specified / In Description,JOB DESCRIPTION We are looking for an experien...,Not Specified,Not Specified,Not Specified,Not Specified,IT
7067,Joblum,UI/UX Developer,IT - Software,"SYNTEL INFOTECH, INC. doing business as Atos S...",2021-04-29,National Capital Reg,Not Specified,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,IT


In [54]:
IT_SOFTWARE_df.to_csv ('JOBLUM-IT_SOFTWARE.csv', index = False)

In [17]:
#Scraping data of Maintenance (JOBLUM) - FIRST HALF

MAINTENANCE_TITLE_FIRST = []
MAINTENANCE_COMPANY_FIRST = []
MAINTENANCE_DATE_FIRST = []
MAINTENANCE_LOCATION_FIRST = []
MAINTENANCE_STATUS_FIRST = []
MAINTENANCE_SALARY_FIRST = []
MAINTENANCE_SALARY_MIN_FIRST = []
MAINTENANCE_SALARY_MAX_FIRST = []
MAINTENANCE_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-maintenance?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, MAINTENANCE_TITLE_FIRST, MAINTENANCE_COMPANY_FIRST, MAINTENANCE_DATE_FIRST, 
              MAINTENANCE_STATUS_FIRST, MAINTENANCE_LOCATION_FIRST, MAINTENANCE_SALARY_FIRST, 
              MAINTENANCE_SALARY_MIN_FIRST, MAINTENANCE_SALARY_MAX_FIRST, MAINTENANCE_DESCRIPTION_FIRST)

In [18]:
#Scraping data of Maintenance (JOBLUM) - SECOND HALF

MAINTENANCE_TITLE_SECOND = []
MAINTENANCE_COMPANY_SECOND = []
MAINTENANCE_DATE_SECOND = []
MAINTENANCE_LOCATION_SECOND = []
MAINTENANCE_STATUS_SECOND = []
MAINTENANCE_SALARY_SECOND = []
MAINTENANCE_SALARY_MIN_SECOND = []
MAINTENANCE_SALARY_MAX_SECOND = []
MAINTENANCE_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, MAINTENANCE_TITLE_SECOND, MAINTENANCE_COMPANY_SECOND, MAINTENANCE_DATE_SECOND, 
              MAINTENANCE_STATUS_SECOND, MAINTENANCE_LOCATION_SECOND, MAINTENANCE_SALARY_SECOND, 
              MAINTENANCE_SALARY_MIN_SECOND, MAINTENANCE_SALARY_MAX_SECOND, MAINTENANCE_DESCRIPTION_FIRST)

In [19]:
#Joining Scrape data of Maintenance (JOBLUM) 

MAINTENANCE_TITLE_LIST = np.concatenate((MAINTENANCE_TITLE_FIRST, MAINTENANCE_TITLE_SECOND))
MAINTENANCE_COMPANY_LIST = np.concatenate((MAINTENANCE_COMPANY_FIRST, MAINTENANCE_COMPANY_SECOND))
MAINTENANCE_DATE_LIST = np.concatenate((MAINTENANCE_DATE_FIRST, MAINTENANCE_DATE_SECOND))
MAINTENANCE_LOCATION_LIST = np.concatenate((MAINTENANCE_LOCATION_FIRST, MAINTENANCE_LOCATION_SECOND))
MAINTENANCE_STATUS_LIST = np.concatenate((MAINTENANCE_STATUS_FIRST, MAINTENANCE_STATUS_SECOND))
MAINTENANCE_SALARY_LIST = np.concatenate((MAINTENANCE_SALARY_FIRST, MAINTENANCE_SALARY_SECOND))
MAINTENANCE_SALARY_MIN_LIST = np.concatenate((MAINTENANCE_SALARY_MIN_FIRST, MAINTENANCE_SALARY_MIN_SECOND))
MAINTENANCE_SALARY_MAX_LIST = np.concatenate((MAINTENANCE_SALARY_MAX_FIRST, MAINTENANCE_SALARY_MAX_SECOND))
MAINTENANCE_DESCRIPTION_LIST = np.concatenate((MAINTENANCE_DESCRIPTION_FIRST, MAINTENANCE_DESCRIPTION_SECOND))

In [20]:
#Creating Data Frame for Maintenance (JOBLUM) 
MAINTENANCE={'Website': "Joblum",
      'Job Title': MAINTENANCE_TITLE_LIST, 
      'Category': "Maintenance", 
      'Company': MAINTENANCE_COMPANY_LIST, 
      'Date Posted': MAINTENANCE_DATE_LIST, 
      'Location': MAINTENANCE_LOCATION_LIST, 
      'Status': MAINTENANCE_STATUS_LIST, 
      'Salary': MAINTENANCE_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': MAINTENANCE_DESCRIPTION_LIST,
      'Min Salary': MAINTENANCE_SALARY_MIN_LIST,
      'Max Salary': MAINTENANCE_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
MAINTENANCE_df = pd.DataFrame(data=MAINTENANCE)
MAINTENANCE_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Machine Technician,Maintenance,BEAUTY ELEMENTS VENTURES MANUFACTURING INC.,2021-05-05,"518 Brgy. San Agustin, Km. 76 Calabarzon Road ...",Contract,,Not Specified / In Description,Not Specified / In Description,Responsibilities: Conducting routine inspectio...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering


In [21]:
MAINTENANCE_df.to_csv ('JOBLUM-MAINTENANCE.csv', index = False)

In [26]:
#Scraping data of Mechanical (JOBLUM) - FIRST HALF

MECH_TITLE_FIRST = []
MECH_COMPANY_FIRST = []
MECH_DATE_FIRST = []
MECH_LOCATION_FIRST = []
MECH_STATUS_FIRST = []
MECH_SALARY_FIRST = []
MECH_SALARY_MIN_FIRST = []
MECH_SALARY_MAX_FIRST = []
MECH_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-mechanical?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, MECH_TITLE_FIRST, MECH_COMPANY_FIRST, MECH_DATE_FIRST, 
              MECH_STATUS_FIRST, MECH_LOCATION_FIRST, MECH_SALARY_FIRST, 
              MECH_SALARY_MIN_FIRST, MECH_SALARY_MAX_FIRST, MECH_DESCRIPTION_FIRST)

In [27]:
#Scraping data of Mechanical (JOBLUM) - SECOND HALF

MECH_TITLE_SECOND = []
MECH_COMPANY_SECOND = []
MECH_DATE_SECOND = []
MECH_LOCATION_SECOND = []
MECH_STATUS_SECOND = []
MECH_SALARY_SECOND = []
MECH_SALARY_MIN_SECOND = []
MECH_SALARY_MAX_SECOND = []
MECH_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, MECH_TITLE_SECOND, MECH_COMPANY_SECOND, MECH_DATE_SECOND, 
              MECH_STATUS_SECOND, MECH_LOCATION_SECOND, MECH_SALARY_SECOND, 
              MECH_SALARY_MIN_SECOND, MECH_SALARY_MAX_SECOND, MECH_DESCRIPTION_FIRST)

In [28]:
#Joining Scrape data of Mechanical (JOBLUM) 

MECH_TITLE_LIST = np.concatenate((MECH_TITLE_FIRST, MECH_TITLE_SECOND))
MECH_COMPANY_LIST = np.concatenate((MECH_COMPANY_FIRST, MECH_COMPANY_SECOND))
MECH_DATE_LIST = np.concatenate((MECH_DATE_FIRST, MECH_DATE_SECOND))
MECH_LOCATION_LIST = np.concatenate((MECH_LOCATION_FIRST, MECH_LOCATION_SECOND))
MECH_STATUS_LIST = np.concatenate((MECH_STATUS_FIRST, MECH_STATUS_SECOND))
MECH_SALARY_LIST = np.concatenate((MECH_SALARY_FIRST, MECH_SALARY_SECOND))
MECH_SALARY_MIN_LIST = np.concatenate((MECH_SALARY_MIN_FIRST, MECH_SALARY_MIN_SECOND))
MECH_SALARY_MAX_LIST = np.concatenate((MECH_SALARY_MAX_FIRST, MECH_SALARY_MAX_SECOND))
MECH_DESCRIPTION_LIST = np.concatenate((MECH_DESCRIPTION_FIRST, MECH_DESCRIPTION_SECOND))

In [32]:
#Creating Data Frame for Mechanical (JOBLUM) 
MECH={'Website': "Joblum",
      'Job Title': MECH_TITLE_LIST, 
      'Category': "Mechanical", 
      'Company': MECH_COMPANY_LIST, 
      'Date Posted': MECH_DATE_LIST, 
      'Location': MECH_LOCATION_LIST, 
      'Status': MECH_STATUS_LIST, 
      'Salary': MECH_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': MECH_DESCRIPTION_LIST,
      'Min Salary': MECH_SALARY_MIN_LIST,
      'Max Salary': MECH_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
MECH_df = pd.DataFrame(data=MECH)
MECH_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,MOTORPOOL HEAD (LAGUNA),Mechanical,Livan Trade Corporation,2021-05-29,Laguna (Others),Contract,"25,000-29,000",Not Specified / In Description,Not Specified / In Description,LIVAN TRADE CORPORATION - a Utility Constructi...,"25,000","29,000",Not Specified,Not Specified,Engineering
1,Joblum,Cadet Engineer,Mechanical,Newton Electrical Equipment Co. Inc,2021-05-29,National Capital Reg,Full Time,,Not Specified / In Description,Not Specified / In Description,Qualifications: Candidate must possess at leas...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
2,Joblum,Building Engineer,Mechanical,Ortigas Land Corporation,2021-05-29,National Capital Reg,Full Time,"23,000-25,000",Not Specified / In Description,Not Specified / In Description,Duties and Responsibilities Include but are no...,"23,000","25,000",Not Specified,Not Specified,Engineering
3,Joblum,MEPFS Coordinator,Mechanical,"Vertical Space Interiors, Inc.",2021-05-29,Cebu (Cebu City),Contract,"15,000-20,000",Not Specified / In Description,Not Specified / In Description,Educational Background: Certificate in Archite...,"15,000","20,000",Not Specified,Not Specified,Engineering
4,Joblum,Maintenance Planner - Valenzuela,Mechanical,SFI Fresh Bakers Corporation,2021-05-29,Valenzuela City,Not Specified,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Bachelor's/Col...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,Joblum,Equipment Engineer,Mechanical,Devex Incorporated,2021-04-30,National Capital Reg,Contract,"23,000-29,900",Not Specified / In Description,Not Specified / In Description,Duties and Responsibility: Provide technical a...,"23,000","29,900",Not Specified,Not Specified,Engineering
732,Joblum,ASSISTANT PRODUCTION SECTION MANAGER (Laguna-B...,Mechanical,Not Specified,2021-04-30,Calabarzon & Mimaropa,Not Specified,,Not Specified / In Description,Not Specified / In Description,Shall be responsible for planning and managing...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
733,Joblum,Parts Man,Mechanical,Sakura Autoworld Inc.,2021-04-30,Iloilo (Others),Full Time,,Not Specified / In Description,Not Specified / In Description,SAKURA AUTOWORLD INC. Sakura Autoworld Inc. co...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
734,Joblum,SERVICE MECHANIC - SUZUKI OUTBOARD MOTOR,Mechanical,Suzuki Philippines Inc.,2021-04-30,Calabarzon & Mimaropa,OJT,,Not Specified / In Description,Not Specified / In Description,THE JOB: Shall be responsible for providing te...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering


In [33]:
MECH_df.to_csv ('JOBLUM-MECH.csv', index = False)

In [34]:
#Scraping data of Mechanical/Automotive Engineering (JOBLUM) - FIRST HALF

MECHENG_TITLE_FIRST = []
MECHENG_COMPANY_FIRST = []
MECHENG_DATE_FIRST = []
MECHENG_LOCATION_FIRST = []
MECHENG_STATUS_FIRST = []
MECHENG_SALARY_FIRST = []
MECHENG_SALARY_MIN_FIRST = []
MECHENG_SALARY_MAX_FIRST = []
MECHENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-mechanical-automotive-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, MECHENG_TITLE_FIRST, MECHENG_COMPANY_FIRST, MECHENG_DATE_FIRST, 
              MECHENG_STATUS_FIRST, MECHENG_LOCATION_FIRST, MECHENG_SALARY_FIRST, 
              MECHENG_SALARY_MIN_FIRST, MECHENG_SALARY_MAX_FIRST, MECHENG_DESCRIPTION_FIRST)

In [35]:
#Scraping data of Mechanical/Automotive Engineering (JOBLUM) - SECOND HALF

MECHENG_TITLE_SECOND = []
MECHENG_COMPANY_SECOND = []
MECHENG_DATE_SECOND = []
MECHENG_LOCATION_SECOND = []
MECHENG_STATUS_SECOND = []
MECHENG_SALARY_SECOND = []
MECHENG_SALARY_MIN_SECOND = []
MECHENG_SALARY_MAX_SECOND = []
MECHENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, MECHENG_TITLE_SECOND, MECHENG_COMPANY_SECOND, MECHENG_DATE_SECOND, 
              MECHENG_STATUS_SECOND, MECHENG_LOCATION_SECOND, MECHENG_SALARY_SECOND, 
              MECHENG_SALARY_MIN_SECOND, MECHENG_SALARY_MAX_SECOND, MECHENG_DESCRIPTION_FIRST)

In [36]:
#Joining Scrape data of Mechanical/Automotive Engineering (JOBLUM) 

MECHENG_TITLE_LIST = np.concatenate((MECHENG_TITLE_FIRST, MECHENG_TITLE_SECOND))
MECHENG_COMPANY_LIST = np.concatenate((MECHENG_COMPANY_FIRST, MECHENG_COMPANY_SECOND))
MECHENG_DATE_LIST = np.concatenate((MECHENG_DATE_FIRST, MECHENG_DATE_SECOND))
MECHENG_LOCATION_LIST = np.concatenate((MECHENG_LOCATION_FIRST, MECHENG_LOCATION_SECOND))
MECHENG_STATUS_LIST = np.concatenate((MECHENG_STATUS_FIRST, MECHENG_STATUS_SECOND))
MECHENG_SALARY_LIST = np.concatenate((MECHENG_SALARY_FIRST, MECHENG_SALARY_SECOND))
MECHENG_SALARY_MIN_LIST = np.concatenate((MECHENG_SALARY_MIN_FIRST, MECHENG_SALARY_MIN_SECOND))
MECHENG_SALARY_MAX_LIST = np.concatenate((MECHENG_SALARY_MAX_FIRST, MECHENG_SALARY_MAX_SECOND))
MECHENG_DESCRIPTION_LIST = np.concatenate((MECHENG_DESCRIPTION_FIRST, MECHENG_DESCRIPTION_SECOND))

In [37]:
#Creating Data Frame for Mechanical/Automotive Engineering (JOBLUM) 
MECHENG={'Website': "Joblum",
      'Job Title': MECHENG_TITLE_LIST, 
      'Category': "Mechanical/Automotive Engineering", 
      'Company': MECHENG_COMPANY_LIST, 
      'Date Posted': MECHENG_DATE_LIST, 
      'Location': MECHENG_LOCATION_LIST, 
      'Status': MECHENG_STATUS_LIST, 
      'Salary': MECHENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': MECHENG_DESCRIPTION_LIST,
      'Min Salary': MECHENG_SALARY_MIN_LIST,
      'Max Salary': MECHENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
MECHENG_df = pd.DataFrame(data=MECHENG)
MECHENG_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field


In [38]:
MECHENG_df.to_csv ('JOBLUM-MECHENG.csv', index = False)

In [39]:
#Scraping data of Nurse/Medical Support (JOBLUM) - FIRST HALF

NURSE_TITLE_FIRST = []
NURSE_COMPANY_FIRST = []
NURSE_DATE_FIRST = []
NURSE_LOCATION_FIRST = []
NURSE_STATUS_FIRST = []
NURSE_SALARY_FIRST = []
NURSE_SALARY_MIN_FIRST = []
NURSE_SALARY_MAX_FIRST = []
NURSE_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-nurse-medical-support?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, NURSE_TITLE_FIRST, NURSE_COMPANY_FIRST, NURSE_DATE_FIRST, 
              NURSE_STATUS_FIRST, NURSE_LOCATION_FIRST, NURSE_SALARY_FIRST, 
              NURSE_SALARY_MIN_FIRST, NURSE_SALARY_MAX_FIRST, NURSE_DESCRIPTION_FIRST)

In [40]:
#Scraping data of Nurse/Medical Support (JOBLUM) - SECOND HALF

NURSE_TITLE_SECOND = []
NURSE_COMPANY_SECOND = []
NURSE_DATE_SECOND = []
NURSE_LOCATION_SECOND = []
NURSE_STATUS_SECOND = []
NURSE_SALARY_SECOND = []
NURSE_SALARY_MIN_SECOND = []
NURSE_SALARY_MAX_SECOND = []
NURSE_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, NURSE_TITLE_SECOND, NURSE_COMPANY_SECOND, NURSE_DATE_SECOND, 
              NURSE_STATUS_SECOND, NURSE_LOCATION_SECOND, NURSE_SALARY_SECOND, 
              NURSE_SALARY_MIN_SECOND, NURSE_SALARY_MAX_SECOND, NURSE_DESCRIPTION_FIRST)

In [41]:
#Joining Scrape data of Nurse/Medical Support (JOBLUM) 

NURSE_TITLE_LIST = np.concatenate((NURSE_TITLE_FIRST, NURSE_TITLE_SECOND))
NURSE_COMPANY_LIST = np.concatenate((NURSE_COMPANY_FIRST, NURSE_COMPANY_SECOND))
NURSE_DATE_LIST = np.concatenate((NURSE_DATE_FIRST, NURSE_DATE_SECOND))
NURSE_LOCATION_LIST = np.concatenate((NURSE_LOCATION_FIRST, NURSE_LOCATION_SECOND))
NURSE_STATUS_LIST = np.concatenate((NURSE_STATUS_FIRST, NURSE_STATUS_SECOND))
NURSE_SALARY_LIST = np.concatenate((NURSE_SALARY_FIRST, NURSE_SALARY_SECOND))
NURSE_SALARY_MIN_LIST = np.concatenate((NURSE_SALARY_MIN_FIRST, NURSE_SALARY_MIN_SECOND))
NURSE_SALARY_MAX_LIST = np.concatenate((NURSE_SALARY_MAX_FIRST, NURSE_SALARY_MAX_SECOND))
NURSE_DESCRIPTION_LIST = np.concatenate((NURSE_DESCRIPTION_FIRST, NURSE_DESCRIPTION_SECOND))

In [42]:
#Creating Data Frame for Nurse/Medical Support (JOBLUM) 
NURSE={'Website': "Joblum",
      'Job Title': NURSE_TITLE_LIST, 
      'Category': "Nurse/Medical Support", 
      'Company': NURSE_COMPANY_LIST, 
      'Date Posted': NURSE_DATE_LIST, 
      'Location': NURSE_LOCATION_LIST, 
      'Status': NURSE_STATUS_LIST, 
      'Salary': NURSE_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': NURSE_DESCRIPTION_LIST,
      'Min Salary': NURSE_SALARY_MIN_LIST,
      'Max Salary': NURSE_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Medicine"}
NURSE_df = pd.DataFrame(data=NURSE)
NURSE_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Clinic Nurse - Ortigas,Nurse/Medical Support,"Aventus Medical Care, Inc.",2021-05-27,Ortigas,Contract,,Not Specified / In Description,Not Specified / In Description,Qualifications: Graduate of Nursing With at le...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
1,Joblum,Clinic Nurse - Pasay,Nurse/Medical Support,"Aventus Medical Care, Inc.",2021-05-27,Pasay,Contract,,Not Specified / In Description,Not Specified / In Description,Qualifications: Graduate of Nursing With at le...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
2,Joblum,Clinic Nurse - Makati,Nurse/Medical Support,"Aventus Medical Care, Inc.",2021-05-27,Makati,Contract,,Not Specified / In Description,Not Specified / In Description,Qualifications: Graduate of Nursing With at le...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
3,Joblum,Clinic Nurse - BGC Taguig,Nurse/Medical Support,"Aventus Medical Care, Inc.",2021-05-27,Taguig,Contract,,Not Specified / In Description,Not Specified / In Description,Qualifications: Graduate of Nursing With at le...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
4,Joblum,Regulatory Pharmacist,Nurse/Medical Support,Axis Online Ventures Inc.,2021-05-26,Paranaque,Not Specified,"24,000-25,000",Not Specified / In Description,Not Specified / In Description,Responsible in ensuring the correctness and co...,"24,000","25,000",Not Specified,Not Specified,Medicine
5,Joblum,USRN Assistant Manager,Nurse/Medical Support,Axis Online Ventures Inc.,2021-05-26,BGC – work from home for now,Not Specified,"85,000-90,000",Not Specified / In Description,Not Specified / In Description,At least 4-6 years of experience in healthcare...,"85,000","90,000",Not Specified,Not Specified,Medicine
6,Joblum,USRN IPA Assistant Manager,Nurse/Medical Support,Axis Online Ventures Inc.,2021-05-26,BGC Taguig,Not Specified,"79,000-80,000",Not Specified / In Description,Not Specified / In Description,"At least 1 year Utilization Management, Prior ...","79,000","80,000",Not Specified,Not Specified,Medicine
7,Joblum,Regulatory Pharmacist,Nurse/Medical Support,Axis Online Ventures Inc.,2021-05-26,Paranaque,Not Specified,"24,000-25,000",Not Specified / In Description,Not Specified / In Description,Responsible in ensuring the correctness and co...,"24,000","25,000",Not Specified,Not Specified,Medicine
8,Joblum,USRN,Nurse/Medical Support,Axis Online Ventures Inc.,2021-05-26,BGC Taguig,Not Specified,"64,000-65,000",Not Specified / In Description,Not Specified / In Description,Clinical bedside care or healthcare accounts i...,"64,000","65,000",Not Specified,Not Specified,Medicine
9,Joblum,PHRN,Nurse/Medical Support,Axis Online Ventures Inc.,2021-05-26,BGC Taguig,Not Specified,"20,000-30,000",Not Specified / In Description,Not Specified / In Description,Clinical bedside care or healthcare accounts i...,"20,000","30,000",Not Specified,Not Specified,Medicine


In [43]:
NURSE_df.to_csv ('JOBLUM-NURSE.csv', index = False)

In [44]:
#Scraping data of Oil/Gas (JOBLUM) - FIRST HALF

OIL_TITLE_FIRST = []
OIL_COMPANY_FIRST = []
OIL_DATE_FIRST = []
OIL_LOCATION_FIRST = []
OIL_STATUS_FIRST = []
OIL_SALARY_FIRST = []
OIL_SALARY_MIN_FIRST = []
OIL_SALARY_MAX_FIRST = []
OIL_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-oil-gas?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, OIL_TITLE_FIRST, OIL_COMPANY_FIRST, OIL_DATE_FIRST, 
              OIL_STATUS_FIRST, OIL_LOCATION_FIRST, OIL_SALARY_FIRST, 
              OIL_SALARY_MIN_FIRST, OIL_SALARY_MAX_FIRST, OIL_DESCRIPTION_FIRST)

In [45]:
#Scraping data of Oil/Gas (JOBLUM) - SECOND HALF

OIL_TITLE_SECOND = []
OIL_COMPANY_SECOND = []
OIL_DATE_SECOND = []
OIL_LOCATION_SECOND = []
OIL_STATUS_SECOND = []
OIL_SALARY_SECOND = []
OIL_SALARY_MIN_SECOND = []
OIL_SALARY_MAX_SECOND = []
OIL_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, OIL_TITLE_SECOND, OIL_COMPANY_SECOND, OIL_DATE_SECOND, 
              OIL_STATUS_SECOND, OIL_LOCATION_SECOND, OIL_SALARY_SECOND, 
              OIL_SALARY_MIN_SECOND, OIL_SALARY_MAX_SECOND, OIL_DESCRIPTION_FIRST)

In [46]:
#Joining Scrape data of Oil/Gas (JOBLUM) 

OIL_TITLE_LIST = np.concatenate((OIL_TITLE_FIRST, OIL_TITLE_SECOND))
OIL_COMPANY_LIST = np.concatenate((OIL_COMPANY_FIRST, OIL_COMPANY_SECOND))
OIL_DATE_LIST = np.concatenate((OIL_DATE_FIRST, OIL_DATE_SECOND))
OIL_LOCATION_LIST = np.concatenate((OIL_LOCATION_FIRST, OIL_LOCATION_SECOND))
OIL_STATUS_LIST = np.concatenate((OIL_STATUS_FIRST, OIL_STATUS_SECOND))
OIL_SALARY_LIST = np.concatenate((OIL_SALARY_FIRST, OIL_SALARY_SECOND))
OIL_SALARY_MIN_LIST = np.concatenate((OIL_SALARY_MIN_FIRST, OIL_SALARY_MIN_SECOND))
OIL_SALARY_MAX_LIST = np.concatenate((OIL_SALARY_MAX_FIRST, OIL_SALARY_MAX_SECOND))
OIL_DESCRIPTION_LIST = np.concatenate((OIL_DESCRIPTION_FIRST, OIL_DESCRIPTION_SECOND))

In [47]:
#Creating Data Frame for Oil/Gas (JOBLUM) 
OIL={'Website': "Joblum",
      'Job Title': OIL_TITLE_LIST, 
      'Category': "Oil/Gas", 
      'Company': OIL_COMPANY_LIST, 
      'Date Posted': OIL_DATE_LIST, 
      'Location': OIL_LOCATION_LIST, 
      'Status': OIL_STATUS_LIST, 
      'Salary': OIL_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': OIL_DESCRIPTION_LIST,
      'Min Salary': OIL_SALARY_MIN_LIST,
      'Max Salary': OIL_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
OIL_df = pd.DataFrame(data=OIL)
OIL_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,STRESS ENGINEER,Oil/Gas,"ZACHRY PHILIPPINES, INC.",2021-05-29,Pasig City,Contract,,Not Specified / In Description,Not Specified / In Description,I. Job Purpose To provide technical work by pe...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
1,Joblum,Technical Support Engineer (DCS/SCADA/PLC),Oil/Gas,"Nezda Technologies, Inc.",2021-05-28,National Capital Reg,Contract,"50,000-90,000",Not Specified / In Description,Not Specified / In Description,Responsibilities: • Responsible for providing ...,"50,000","90,000",Not Specified,Not Specified,Engineering
2,Joblum,Technical Writer,Oil/Gas,"Nezda Technologies, Inc.",2021-05-27,National Capital Reg,Contract,"60,000-70,000",Not Specified / In Description,Not Specified / In Description,Job Summary We are looking for a technical wri...,"60,000","70,000",Not Specified,Not Specified,Engineering
3,Joblum,Fabrication Estimator/QS,Oil/Gas,"CYBALINK SOLUTIONS, INC.",2021-05-27,Pasig City,Contract,"50,000-65,000",Not Specified / In Description,Not Specified / In Description,Cybalink Solutions is an Australian owned Outs...,"50,000","65,000",Not Specified,Not Specified,Engineering
4,Joblum,Workpack Engineer,Oil/Gas,"CYBALINK SOLUTIONS, INC.",2021-05-27,Pasig City,OJT,"50,000-60,000",Not Specified / In Description,Not Specified / In Description,Cybalink Solutions is an Australian owned Outs...,"50,000","60,000",Not Specified,Not Specified,Engineering
5,Joblum,Operations Assistant,Oil/Gas,Northern Star Energy Corporation,2021-05-27,Makati City,Not Specified,,Not Specified / In Description,Not Specified / In Description,Job Duties: To perform clerical and administra...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
6,Joblum,Gas Station Supervisor,Oil/Gas,Honda Oem Touch Up Paint Philippines,2021-05-26,Quezon City,Not Specified,,Not Specified / In Description,Not Specified / In Description,The hirer is still in the process of verificat...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
7,Joblum,Bunkering Fuel Assistant,Oil/Gas,Philippine Span Asia Carrier Corp. (Manila),2021-05-25,Manila City,Not Specified,,Not Specified / In Description,Not Specified / In Description,What awaits you if you consider this opportuni...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
8,Joblum,Service Sales,Oil/Gas,Qarah Industries Corp.,2021-05-25,Paranaque City,Full Time,"15,000-25,000",Not Specified / In Description,Not Specified / In Description,Responsibilities: · Responsible for maintainin...,"15,000","25,000",Not Specified,Not Specified,Engineering
9,Joblum,Sales Engineer,Oil/Gas,Qarah Industries Corp.,2021-05-25,Paranaque City,Full Time,"15,000-21,000",Not Specified / In Description,Not Specified / In Description,Job Description Serve as the technical product...,"15,000","21,000",Not Specified,Not Specified,Engineering


In [48]:
OIL_df.to_csv ('JOBLUM-OIL.csv', index = False)

In [49]:
#Scraping data of Oil/Gas Engineering (JOBLUM) - FIRST HALF

OILENG_TITLE_FIRST = []
OILENG_COMPANY_FIRST = []
OILENG_DATE_FIRST = []
OILENG_LOCATION_FIRST = []
OILENG_STATUS_FIRST = []
OILENG_SALARY_FIRST = []
OILENG_SALARY_MIN_FIRST = []
OILENG_SALARY_MAX_FIRST = []
OILENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-oil-gas-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, OILENG_TITLE_FIRST, OILENG_COMPANY_FIRST, OILENG_DATE_FIRST, 
              OILENG_STATUS_FIRST, OILENG_LOCATION_FIRST, OILENG_SALARY_FIRST, 
              OILENG_SALARY_MIN_FIRST, OILENG_SALARY_MAX_FIRST, OILENG_DESCRIPTION_FIRST)

In [50]:
#Scraping data of Oil/Gas Engineering (JOBLUM) - SECOND HALF

OILENG_TITLE_SECOND = []
OILENG_COMPANY_SECOND = []
OILENG_DATE_SECOND = []
OILENG_LOCATION_SECOND = []
OILENG_STATUS_SECOND = []
OILENG_SALARY_SECOND = []
OILENG_SALARY_MIN_SECOND = []
OILENG_SALARY_MAX_SECOND = []
OILENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, OILENG_TITLE_SECOND, OILENG_COMPANY_SECOND, OILENG_DATE_SECOND, 
              OILENG_STATUS_SECOND, OILENG_LOCATION_SECOND, OILENG_SALARY_SECOND, 
              OILENG_SALARY_MIN_SECOND, OILENG_SALARY_MAX_SECOND, OILENG_DESCRIPTION_FIRST)

In [51]:
#Joining Scrape data of Oil/Gas Engineering (JOBLUM) 

OILENG_TITLE_LIST = np.concatenate((OILENG_TITLE_FIRST, OILENG_TITLE_SECOND))
OILENG_COMPANY_LIST = np.concatenate((OILENG_COMPANY_FIRST, OILENG_COMPANY_SECOND))
OILENG_DATE_LIST = np.concatenate((OILENG_DATE_FIRST, OILENG_DATE_SECOND))
OILENG_LOCATION_LIST = np.concatenate((OILENG_LOCATION_FIRST, OILENG_LOCATION_SECOND))
OILENG_STATUS_LIST = np.concatenate((OILENG_STATUS_FIRST, OILENG_STATUS_SECOND))
OILENG_SALARY_LIST = np.concatenate((OILENG_SALARY_FIRST, OILENG_SALARY_SECOND))
OILENG_SALARY_MIN_LIST = np.concatenate((OILENG_SALARY_MIN_FIRST, OILENG_SALARY_MIN_SECOND))
OILENG_SALARY_MAX_LIST = np.concatenate((OILENG_SALARY_MAX_FIRST, OILENG_SALARY_MAX_SECOND))
OILENG_DESCRIPTION_LIST = np.concatenate((OILENG_DESCRIPTION_FIRST, OILENG_DESCRIPTION_SECOND))

In [52]:
#Creating Data Frame for Oil/Gas Engineering (JOBLUM) 
OILENG={'Website': "Joblum",
      'Job Title': OILENG_TITLE_LIST, 
      'Category': "Oil/Gas Engineering", 
      'Company': OILENG_COMPANY_LIST, 
      'Date Posted': OILENG_DATE_LIST, 
      'Location': OILENG_LOCATION_LIST, 
      'Status': OILENG_STATUS_LIST, 
      'Salary': OILENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': OILENG_DESCRIPTION_LIST,
      'Min Salary': OILENG_SALARY_MIN_LIST,
      'Max Salary': OILENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
OILENG_df = pd.DataFrame(data=OILENG)
OILENG_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field


In [53]:
OILENG_df.to_csv ('JOBLUM-OILENG.csv', index = False)

In [25]:
#Scraping data of Other Engineering (JOBLUM) - FIRST HALF

ENG_TITLE_FIRST = []
ENG_COMPANY_FIRST = []
ENG_DATE_FIRST = []
ENG_LOCATION_FIRST = []
ENG_STATUS_FIRST = []
ENG_SALARY_FIRST = []
ENG_SALARY_MIN_FIRST = []
ENG_SALARY_MAX_FIRST = []
ENG_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-other-engineering?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ENG_TITLE_FIRST, ENG_COMPANY_FIRST, ENG_DATE_FIRST, 
              ENG_STATUS_FIRST, ENG_LOCATION_FIRST, ENG_SALARY_FIRST, 
              ENG_SALARY_MIN_FIRST, ENG_SALARY_MAX_FIRST, ENG_DESCRIPTION_FIRST)

In [26]:
#Scraping data of Other Engineering (JOBLUM) - SECOND HALF

ENG_TITLE_SECOND = []
ENG_COMPANY_SECOND = []
ENG_DATE_SECOND = []
ENG_LOCATION_SECOND = []
ENG_STATUS_SECOND = []
ENG_SALARY_SECOND = []
ENG_SALARY_MIN_SECOND = []
ENG_SALARY_MAX_SECOND = []
ENG_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, ENG_TITLE_SECOND, ENG_COMPANY_SECOND, ENG_DATE_SECOND, 
              ENG_STATUS_SECOND, ENG_LOCATION_SECOND, ENG_SALARY_SECOND, 
              ENG_SALARY_MIN_SECOND, ENG_SALARY_MAX_SECOND, ENG_DESCRIPTION_FIRST)

In [27]:
#Joining Scrape data of Other Engineering (JOBLUM) 

ENG_TITLE_LIST = np.concatenate((ENG_TITLE_FIRST, ENG_TITLE_SECOND))
ENG_COMPANY_LIST = np.concatenate((ENG_COMPANY_FIRST, ENG_COMPANY_SECOND))
ENG_DATE_LIST = np.concatenate((ENG_DATE_FIRST, ENG_DATE_SECOND))
ENG_LOCATION_LIST = np.concatenate((ENG_LOCATION_FIRST, ENG_LOCATION_SECOND))
ENG_STATUS_LIST = np.concatenate((ENG_STATUS_FIRST, ENG_STATUS_SECOND))
ENG_SALARY_LIST = np.concatenate((ENG_SALARY_FIRST, ENG_SALARY_SECOND))
ENG_SALARY_MIN_LIST = np.concatenate((ENG_SALARY_MIN_FIRST, ENG_SALARY_MIN_SECOND))
ENG_SALARY_MAX_LIST = np.concatenate((ENG_SALARY_MAX_FIRST, ENG_SALARY_MAX_SECOND))
ENG_DESCRIPTION_LIST = np.concatenate((ENG_DESCRIPTION_FIRST, ENG_DESCRIPTION_SECOND))

In [28]:
#Creating Data Frame for Other Engineering (JOBLUM) 
ENG={'Website': "Joblum",
      'Job Title': ENG_TITLE_LIST, 
      'Category': "Other Engineering", 
      'Company': ENG_COMPANY_LIST, 
      'Date Posted': ENG_DATE_LIST, 
      'Location': ENG_LOCATION_LIST, 
      'Status': ENG_STATUS_LIST, 
      'Salary': ENG_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': ENG_DESCRIPTION_LIST,
      'Min Salary': ENG_SALARY_MIN_LIST,
      'Max Salary': ENG_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
ENG_df = pd.DataFrame(data=ENG)
ENG_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Lifting Supervisor,Other Engineering,Megawide Construction Corporation,2021-05-29,Pampanga (Others),OJT,"50,000-70,000",Not Specified / In Description,Not Specified / In Description,Supervise the maintenance and operation of all...,"50,000","70,000",Not Specified,Not Specified,Engineering
1,Joblum,"Facilities Officer - Imus, Cavite",Other Engineering,TaskUs,2021-05-29,"National Capital Reg, Calabarzon & Mimaropa",Contract,,Not Specified / In Description,Not Specified / In Description,Imagine yourself going to work with one thing ...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
2,Joblum,Traffic Management Officer (Bulacan),Other Engineering,Not Specified,2021-05-29,Central Luzon,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
3,Joblum,LICENSED GEODETIC ENGINEER (CAVITE),Other Engineering,Vista Land & Lifescapes Inc.,2021-05-29,National Capital Reg,Full Time,,Not Specified / In Description,Not Specified / In Description,JOB DESCRIPTION: Responsible in conducting lan...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
4,Joblum,Aluminum Fabrication Supervisor,Other Engineering,Fasclad Incorporated Philippines,2021-05-29,Valenzuela City,Contract,,Not Specified / In Description,Not Specified / In Description,The hirer is still in the process of verificat...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Joblum,Line man,Other Engineering,RISE MANPOWER SERVICES,2021-04-30,Bahrain,Full Time,,Not Specified / In Description,Not Specified / In Description,Working within a two or four man team you will...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
448,Joblum,RF Engineer,Other Engineering,RISE MANPOWER SERVICES,2021-04-30,Bahrain,Full Time,,Not Specified / In Description,Not Specified / In Description,Has strong knowledge of microwave wireless poi...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
449,Joblum,Piping Engineer,Other Engineering,ARAM ENTERPRISES INC,2021-04-30,Saudi Arabia,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidates must be a Bachelor's / College Degr...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
450,Joblum,Piping Engineer,Other Engineering,ARAM ENTERPRISES INC,2021-04-30,Saudi Arabia,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidates must be a Bachelor's / College Degr...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering


In [29]:
ENG_df.to_csv ('JOBLUM-ENG.csv', index = False)

In [30]:
#Scraping data of Pharmacy (JOBLUM) - FIRST HALF

PHARMA_TITLE_FIRST = []
PHARMA_COMPANY_FIRST = []
PHARMA_DATE_FIRST = []
PHARMA_LOCATION_FIRST = []
PHARMA_STATUS_FIRST = []
PHARMA_SALARY_FIRST = []
PHARMA_SALARY_MIN_FIRST = []
PHARMA_SALARY_MAX_FIRST = []
PHARMA_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-pharmacy?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, PHARMA_TITLE_FIRST, PHARMA_COMPANY_FIRST, PHARMA_DATE_FIRST, 
              PHARMA_STATUS_FIRST, PHARMA_LOCATION_FIRST, PHARMA_SALARY_FIRST, 
              PHARMA_SALARY_MIN_FIRST, PHARMA_SALARY_MAX_FIRST, PHARMA_DESCRIPTION_FIRST)

In [31]:
#Scraping data of Pharmacy (JOBLUM) - SECOND HALF

PHARMA_TITLE_SECOND = []
PHARMA_COMPANY_SECOND = []
PHARMA_DATE_SECOND = []
PHARMA_LOCATION_SECOND = []
PHARMA_STATUS_SECOND = []
PHARMA_SALARY_SECOND = []
PHARMA_SALARY_MIN_SECOND = []
PHARMA_SALARY_MAX_SECOND = []
PHARMA_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, PHARMA_TITLE_SECOND, PHARMA_COMPANY_SECOND, PHARMA_DATE_SECOND, 
              PHARMA_STATUS_SECOND, PHARMA_LOCATION_SECOND, PHARMA_SALARY_SECOND, 
              PHARMA_SALARY_MIN_SECOND, PHARMA_SALARY_MAX_SECOND, PHARMA_DESCRIPTION_FIRST)

In [32]:
#Joining Scrape data of Pharmacy (JOBLUM) 

PHARMA_TITLE_LIST = np.concatenate((PHARMA_TITLE_FIRST, PHARMA_TITLE_SECOND))
PHARMA_COMPANY_LIST = np.concatenate((PHARMA_COMPANY_FIRST, PHARMA_COMPANY_SECOND))
PHARMA_DATE_LIST = np.concatenate((PHARMA_DATE_FIRST, PHARMA_DATE_SECOND))
PHARMA_LOCATION_LIST = np.concatenate((PHARMA_LOCATION_FIRST, PHARMA_LOCATION_SECOND))
PHARMA_STATUS_LIST = np.concatenate((PHARMA_STATUS_FIRST, PHARMA_STATUS_SECOND))
PHARMA_SALARY_LIST = np.concatenate((PHARMA_SALARY_FIRST, PHARMA_SALARY_SECOND))
PHARMA_SALARY_MIN_LIST = np.concatenate((PHARMA_SALARY_MIN_FIRST, PHARMA_SALARY_MIN_SECOND))
PHARMA_SALARY_MAX_LIST = np.concatenate((PHARMA_SALARY_MAX_FIRST, PHARMA_SALARY_MAX_SECOND))
PHARMA_DESCRIPTION_LIST = np.concatenate((PHARMA_DESCRIPTION_FIRST, PHARMA_DESCRIPTION_SECOND))

In [33]:
#Creating Data Frame for Pharmacy (JOBLUM) 
PHARMA={'Website': "Joblum",
      'Job Title': PHARMA_TITLE_LIST, 
      'Category': "Pharmacy", 
      'Company': PHARMA_COMPANY_LIST, 
      'Date Posted': PHARMA_DATE_LIST, 
      'Location': PHARMA_LOCATION_LIST, 
      'Status': PHARMA_STATUS_LIST, 
      'Salary': PHARMA_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': PHARMA_DESCRIPTION_LIST,
      'Min Salary': PHARMA_SALARY_MIN_LIST,
      'Max Salary': PHARMA_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Medicine"}
PHARMA_df = pd.DataFrame(data=PHARMA)
PHARMA_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Pharmacist,Pharmacy,Farmacia ni Dok-Calamba Branch,2021-05-30,Laguna (Calamba City),Not Specified,"12,000-16,800",Not Specified / In Description,Not Specified / In Description,experienced in handling retail or community ph...,"12,000","16,800",Not Specified,Not Specified,Medicine
1,Joblum,REGULATORY PHARMACIST (ENTRY LEVEL),Pharmacy,Despina Pharma Inc,2021-05-29,National Capital Reg,Not Specified,"18,000-23,000",Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Professional L...,"18,000","23,000",Not Specified,Not Specified,Medicine
2,Joblum,PHARMACIST for TARLAC,Pharmacy,"Watsons Personal Care Stores (Philippines), Inc.",2021-05-29,"Tarlac, National Capital Reg",Full Time,,Not Specified / In Description,Not Specified / In Description,JOB SUMMARY: The Pharmacist is responsible for...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
3,Joblum,Pharmacist,Pharmacy,Springfield Pharma And Medical Supplies Inc.,2021-05-29,Quezon City,Not Specified,,Not Specified / In Description,Not Specified / In Description,Responsibilities: Organize the pharmacy in an ...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
4,Joblum,(3) LICENSED PHARMACIST,Pharmacy,"Euro Health Care Exponents, Inc.",2021-05-29,Quezon City,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Professional L...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,Joblum,SYKES Priority Hour | Healthcare Agents (CSR) ...,Pharmacy,Sykes Philippines,2021-04-30,National Capital Reg,Contract,,Not Specified / In Description,Not Specified / In Description,Want to have your application prioritized? App...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
404,Joblum,PHARMACIST for IBA ZAMBLES,Pharmacy,"Watsons Personal Care Stores (Philippines), Inc.",2021-04-29,Zambales (Others),Not Specified,,Not Specified / In Description,Not Specified / In Description,JOB SUMMARY: The Pharmacist is responsible for...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
405,Joblum,PHARMACIST for VALENCIA BUKIDNON,Pharmacy,"Watsons Personal Care Stores (Philippines), Inc.",2021-04-29,Bukidnon,Not Specified,,Not Specified / In Description,Not Specified / In Description,JOB SUMMARY: The Pharmacist is responsible for...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine
406,Joblum,PHARMACIST for DIPOLOG CITY,Pharmacy,"Watsons Personal Care Stores (Philippines), Inc.",2021-04-29,Zamboanga del Norte,Not Specified,,Not Specified / In Description,Not Specified / In Description,JOB SUMMARY: The Pharmacist is responsible for...,Not Specified,Not Specified,Not Specified,Not Specified,Medicine


In [34]:
PHARMA_df.to_csv ('JOBLUM-PHARMA.csv', index = False)

In [35]:
#Scraping data of Quality Assurance (JOBLUM) - FIRST HALF

QUALITY_TITLE_FIRST = []
QUALITY_COMPANY_FIRST = []
QUALITY_DATE_FIRST = []
QUALITY_LOCATION_FIRST = []
QUALITY_STATUS_FIRST = []
QUALITY_SALARY_FIRST = []
QUALITY_SALARY_MIN_FIRST = []
QUALITY_SALARY_MAX_FIRST = []
QUALITY_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-quality-assurance?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, QUALITY_TITLE_FIRST, QUALITY_COMPANY_FIRST, QUALITY_DATE_FIRST, 
              QUALITY_STATUS_FIRST, QUALITY_LOCATION_FIRST, QUALITY_SALARY_FIRST, 
              QUALITY_SALARY_MIN_FIRST, QUALITY_SALARY_MAX_FIRST, QUALITY_DESCRIPTION_FIRST)

In [36]:
#Scraping data of Quality Assurance (JOBLUM) - SECOND HALF

QUALITY_TITLE_SECOND = []
QUALITY_COMPANY_SECOND = []
QUALITY_DATE_SECOND = []
QUALITY_LOCATION_SECOND = []
QUALITY_STATUS_SECOND = []
QUALITY_SALARY_SECOND = []
QUALITY_SALARY_MIN_SECOND = []
QUALITY_SALARY_MAX_SECOND = []
QUALITY_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, QUALITY_TITLE_SECOND, QUALITY_COMPANY_SECOND, QUALITY_DATE_SECOND, 
              QUALITY_STATUS_SECOND, QUALITY_LOCATION_SECOND, QUALITY_SALARY_SECOND, 
              QUALITY_SALARY_MIN_SECOND, QUALITY_SALARY_MAX_SECOND, QUALITY_DESCRIPTION_FIRST)

In [37]:
#Joining Scrape data of Quality Assurance (JOBLUM) 

QUALITY_TITLE_LIST = np.concatenate((QUALITY_TITLE_FIRST, QUALITY_TITLE_SECOND))
QUALITY_COMPANY_LIST = np.concatenate((QUALITY_COMPANY_FIRST, QUALITY_COMPANY_SECOND))
QUALITY_DATE_LIST = np.concatenate((QUALITY_DATE_FIRST, QUALITY_DATE_SECOND))
QUALITY_LOCATION_LIST = np.concatenate((QUALITY_LOCATION_FIRST, QUALITY_LOCATION_SECOND))
QUALITY_STATUS_LIST = np.concatenate((QUALITY_STATUS_FIRST, QUALITY_STATUS_SECOND))
QUALITY_SALARY_LIST = np.concatenate((QUALITY_SALARY_FIRST, QUALITY_SALARY_SECOND))
QUALITY_SALARY_MIN_LIST = np.concatenate((QUALITY_SALARY_MIN_FIRST, QUALITY_SALARY_MIN_SECOND))
QUALITY_SALARY_MAX_LIST = np.concatenate((QUALITY_SALARY_MAX_FIRST, QUALITY_SALARY_MAX_SECOND))
QUALITY_DESCRIPTION_LIST = np.concatenate((QUALITY_DESCRIPTION_FIRST, QUALITY_DESCRIPTION_SECOND))

In [38]:
#Creating Data Frame for Quality Assurance (JOBLUM) 
QUALITY={'Website': "Joblum",
      'Job Title': QUALITY_TITLE_LIST, 
      'Category': "Quality Assurance", 
      'Company': QUALITY_COMPANY_LIST, 
      'Date Posted': QUALITY_DATE_LIST, 
      'Location': QUALITY_LOCATION_LIST, 
      'Status': QUALITY_STATUS_LIST, 
      'Salary': QUALITY_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': QUALITY_DESCRIPTION_LIST,
      'Min Salary': QUALITY_SALARY_MIN_LIST,
      'Max Salary': QUALITY_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
QUALITY_df = pd.DataFrame(data=QUALITY)
QUALITY_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field


In [39]:
QUALITY_df.to_csv ('JOBLUM-QUALITY.csv', index = False)

In [40]:
#Scraping data of Quantity Surveying (JOBLUM) - FIRST HALF

QUANTITY_TITLE_FIRST = []
QUANTITY_COMPANY_FIRST = []
QUANTITY_DATE_FIRST = []
QUANTITY_LOCATION_FIRST = []
QUANTITY_STATUS_FIRST = []
QUANTITY_SALARY_FIRST = []
QUANTITY_SALARY_MIN_FIRST = []
QUANTITY_SALARY_MAX_FIRST = []
QUANTITY_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-quantity-surveying?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, QUANTITY_TITLE_FIRST, QUANTITY_COMPANY_FIRST, QUANTITY_DATE_FIRST, 
              QUANTITY_STATUS_FIRST, QUANTITY_LOCATION_FIRST, QUANTITY_SALARY_FIRST, 
              QUANTITY_SALARY_MIN_FIRST, QUANTITY_SALARY_MAX_FIRST, QUANTITY_DESCRIPTION_FIRST)

In [41]:
#Scraping data of Quantity Surveying (JOBLUM) - SECOND HALF

QUANTITY_TITLE_SECOND = []
QUANTITY_COMPANY_SECOND = []
QUANTITY_DATE_SECOND = []
QUANTITY_LOCATION_SECOND = []
QUANTITY_STATUS_SECOND = []
QUANTITY_SALARY_SECOND = []
QUANTITY_SALARY_MIN_SECOND = []
QUANTITY_SALARY_MAX_SECOND = []
QUANTITY_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, QUANTITY_TITLE_SECOND, QUANTITY_COMPANY_SECOND, QUANTITY_DATE_SECOND, 
              QUANTITY_STATUS_SECOND, QUANTITY_LOCATION_SECOND, QUANTITY_SALARY_SECOND, 
              QUANTITY_SALARY_MIN_SECOND, QUANTITY_SALARY_MAX_SECOND, QUANTITY_DESCRIPTION_FIRST)

In [42]:
#Joining Scrape data of Quantity Surveying (JOBLUM) 

QUANTITY_TITLE_LIST = np.concatenate((QUANTITY_TITLE_FIRST, QUANTITY_TITLE_SECOND))
QUANTITY_COMPANY_LIST = np.concatenate((QUANTITY_COMPANY_FIRST, QUANTITY_COMPANY_SECOND))
QUANTITY_DATE_LIST = np.concatenate((QUANTITY_DATE_FIRST, QUANTITY_DATE_SECOND))
QUANTITY_LOCATION_LIST = np.concatenate((QUANTITY_LOCATION_FIRST, QUANTITY_LOCATION_SECOND))
QUANTITY_STATUS_LIST = np.concatenate((QUANTITY_STATUS_FIRST, QUANTITY_STATUS_SECOND))
QUANTITY_SALARY_LIST = np.concatenate((QUANTITY_SALARY_FIRST, QUANTITY_SALARY_SECOND))
QUANTITY_SALARY_MIN_LIST = np.concatenate((QUANTITY_SALARY_MIN_FIRST, QUANTITY_SALARY_MIN_SECOND))
QUANTITY_SALARY_MAX_LIST = np.concatenate((QUANTITY_SALARY_MAX_FIRST, QUANTITY_SALARY_MAX_SECOND))
QUANTITY_DESCRIPTION_LIST = np.concatenate((QUANTITY_DESCRIPTION_FIRST, QUANTITY_DESCRIPTION_SECOND))

In [43]:
#Creating Data Frame for Quantity Surveying (JOBLUM) 
QUANTITY={'Website': "Joblum",
      'Job Title': QUANTITY_TITLE_LIST, 
      'Category': "Quantity Surveying", 
      'Company': QUANTITY_COMPANY_LIST, 
      'Date Posted': QUANTITY_DATE_LIST, 
      'Location': QUANTITY_LOCATION_LIST, 
      'Status': QUANTITY_STATUS_LIST, 
      'Salary': QUANTITY_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': QUANTITY_DESCRIPTION_LIST,
      'Min Salary': QUANTITY_SALARY_MIN_LIST,
      'Max Salary': QUANTITY_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Engineering"}
QUANTITY_df = pd.DataFrame(data=QUANTITY)
QUANTITY_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,Quantity Surveyor (Laguna/Antipolo),Quantity Surveying,MAKATI DEVELOPMENT CORPORATION,2021-05-29,"National Capital Reg, Laguna (Calamba City), L...",Contract,,Not Specified / In Description,Not Specified / In Description,• Candidate must possess at least a Bachelor's...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
1,Joblum,SR. QUANTITY SURVEYOR,Quantity Surveying,"Global Estate Resorts, Inc. (a subsidiary of M...",2021-05-29,"Quezon City, Taguig City",Contract,,Not Specified / In Description,Not Specified / In Description,Coordinate and compile documents for the pre-q...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
2,Joblum,Quantity Surveyor,Quantity Surveying,Asya Design Partner,2021-05-28,National Capital Reg,Contract,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Bachelor's/Col...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
3,Joblum,QS Manager,Quantity Surveying,Not Specified,2021-05-28,National Capital Reg,Contract,"52,000-72,800",Not Specified / In Description,Not Specified / In Description,· Candidate must possess at least Bachelor's/C...,"52,000","72,800",Not Specified,Not Specified,Engineering
4,Joblum,Senior Quantity Surveyor,Quantity Surveying,J.E. Abraham C. Lee Construction & Dev't. Corp.,2021-05-27,Cebu (Cebu City),Contract,,Not Specified / In Description,Not Specified / In Description,The hirer is still in the process of verificat...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Joblum,URGENT! Land Surveyor - Cebu City (Must be Lic...,Quantity Surveying,HR Network Inc.,2021-05-01,Central Visayas,Contract,"18,000-23,400",Not Specified / In Description,Not Specified / In Description,Job Qualification and Skills: A Graduate of Ge...,"18,000","23,400",Not Specified,Not Specified,Engineering
59,Joblum,Quantity Surveyor,Quantity Surveying,"SMCC Philippines, Inc.",2021-04-30,"National Capital Reg, Calabarzon & Mimaropa",Contract,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least Bachelor's/Col...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
60,Joblum,QUANTITY SURVEYOR,Quantity Surveying,1ST DYNAMIC PERSONNEL RESOURCES INC.,2021-04-30,Papua New Guinea,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Professional...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering
61,Joblum,"Assistant Quantity Surveyor (Pampanga, Tarlac)",Quantity Surveying,MAKATI DEVELOPMENT CORPORATION,2021-04-30,National Capital Reg,Full Time,,Not Specified / In Description,Not Specified / In Description,Candidate must possess at least a Bachelor's/C...,Not Specified,Not Specified,Not Specified,Not Specified,Engineering


In [44]:
QUANTITY_df.to_csv ('JOBLUM-QUANTITY.csv', index = False)

In [45]:
#Scraping data of Science & Technology (JOBLUM) - FIRST HALF

SNT_TITLE_FIRST = []
SNT_COMPANY_FIRST = []
SNT_DATE_FIRST = []
SNT_LOCATION_FIRST = []
SNT_STATUS_FIRST = []
SNT_SALARY_FIRST = []
SNT_SALARY_MIN_FIRST = []
SNT_SALARY_MAX_FIRST = []
SNT_DESCRIPTION_FIRST = []

JOBLUM_URLs = 'https://ph.joblum.com/jobs-spec-science-amp-technology?p='
soup = getSoup(JOBLUM_URLs)
NUM_JOBS = getNumJobs(soup)
NUM_PAGES = getNumPages(NUM_JOBS)
JOB_LINKS = getLinks(NUM_PAGES, JOBLUM_URLs)
FIRST_HALF = math.ceil(NUM_PAGES/2)

for i in range(FIRST_HALF):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, SNT_TITLE_FIRST, SNT_COMPANY_FIRST, SNT_DATE_FIRST, 
              SNT_STATUS_FIRST, SNT_LOCATION_FIRST, SNT_SALARY_FIRST, 
              SNT_SALARY_MIN_FIRST, SNT_SALARY_MAX_FIRST, SNT_DESCRIPTION_FIRST)

In [46]:
#Scraping data of Science & Technology (JOBLUM) - SECOND HALF

SNT_TITLE_SECOND = []
SNT_COMPANY_SECOND = []
SNT_DATE_SECOND = []
SNT_LOCATION_SECOND = []
SNT_STATUS_SECOND = []
SNT_SALARY_SECOND = []
SNT_SALARY_MIN_SECOND = []
SNT_SALARY_MAX_SECOND = []
SNT_DESCRIPTION_SECOND = []

for i in range(FIRST_HALF, NUM_PAGES):
    JOBLUM_SOUP = getSoup(JOB_LINKS[i])
    URL = getJobURL(JOBLUM_SOUP)
    scrapeJob(URL, SNT_TITLE_SECOND, SNT_COMPANY_SECOND, SNT_DATE_SECOND, 
              SNT_STATUS_SECOND, SNT_LOCATION_SECOND, SNT_SALARY_SECOND, 
              SNT_SALARY_MIN_SECOND, SNT_SALARY_MAX_SECOND, SNT_DESCRIPTION_FIRST)

In [47]:
#Joining Scrape data of Science & Technology (JOBLUM) 

SNT_TITLE_LIST = np.concatenate((SNT_TITLE_FIRST, SNT_TITLE_SECOND))
SNT_COMPANY_LIST = np.concatenate((SNT_COMPANY_FIRST, SNT_COMPANY_SECOND))
SNT_DATE_LIST = np.concatenate((SNT_DATE_FIRST, SNT_DATE_SECOND))
SNT_LOCATION_LIST = np.concatenate((SNT_LOCATION_FIRST, SNT_LOCATION_SECOND))
SNT_STATUS_LIST = np.concatenate((SNT_STATUS_FIRST, SNT_STATUS_SECOND))
SNT_SALARY_LIST = np.concatenate((SNT_SALARY_FIRST, SNT_SALARY_SECOND))
SNT_SALARY_MIN_LIST = np.concatenate((SNT_SALARY_MIN_FIRST, SNT_SALARY_MIN_SECOND))
SNT_SALARY_MAX_LIST = np.concatenate((SNT_SALARY_MAX_FIRST, SNT_SALARY_MAX_SECOND))
SNT_DESCRIPTION_LIST = np.concatenate((SNT_DESCRIPTION_FIRST, SNT_DESCRIPTION_SECOND))

In [48]:
#Creating Data Frame for Science & Technology (JOBLUM) 
SNT={'Website': "Joblum",
      'Job Title': SNT_TITLE_LIST, 
      'Category': "Science & Technology", 
      'Company': SNT_COMPANY_LIST, 
      'Date Posted': SNT_DATE_LIST, 
      'Location': SNT_LOCATION_LIST, 
      'Status': SNT_STATUS_LIST, 
      'Salary': SNT_SALARY_LIST,
      'Education': "Not Specified / In Description",
      'Years of Work Expirience': "Not Specified / In Description",
      'Job Description': SNT_DESCRIPTION_LIST,
      'Min Salary': SNT_SALARY_MIN_LIST,
      'Max Salary': SNT_SALARY_MAX_LIST,
      'Min Years of Work Expirience': "Not Specified",
      'Max Years of Work Expirience': "Not Specified",
      'Field': "Science"}
SNT_df = pd.DataFrame(data=SNT)
SNT_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,IP Certified Medical Coder,Science & Technology,Lancesoft Inc,2021-05-17,Taguig - WFH,Full Time,"30,000-60,000",Not Specified / In Description,Not Specified / In Description,- Philippine Registered Nurse (can consider BS...,"30,000","60,000",Not Specified,Not Specified,Science
1,Joblum,USRN,Science & Technology,Lancesoft Inc,2021-05-17,Taguig - WFH,Full Time,"35,000-60,000",Not Specified / In Description,Not Specified / In Description,- Philippine Registered Nurse - Mainland USRN ...,"35,000","60,000",Not Specified,Not Specified,Science
2,Joblum,UM Appeals and Grievances Quality Review Nurse,Science & Technology,Lancesoft Inc,2021-05-17,Taguig - WFH,Full Time,"35,000-60,000",Not Specified / In Description,Not Specified / In Description,- Philippine Registered Nurse - Mainland USRN ...,"35,000","60,000",Not Specified,Not Specified,Science
3,Joblum,"LABORATORY ANALYST - LIPA, BATANGAS (URGENT)",Science & Technology,"LIPA QUALITY CONTROL CENTER, INC.",2021-05-29,Batangas,OJT,,Not Specified / In Description,Not Specified / In Description,Laboratory Analyst Conducts laboratory tests i...,Not Specified,Not Specified,Not Specified,Not Specified,Science
4,Joblum,Science Research Specialist I,Science & Technology,Philippine Nuclear Research Institute - Govern...,2021-05-29,National Capital Reg,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 30, ...",Not Specified,Not Specified,Not Specified,Not Specified,Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Joblum,Science Research Specialist II,Science & Technology,Mines and Geosciences Bureau Regional Office I...,2021-05-01,Cagayan Valley,Contract,"36,628",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 5, 2...","36,628","36,628",Not Specified,Not Specified,Science
74,Joblum,Science Research Specialist I,Science & Technology,BUKIDNON STATE COLLEGE - Government,2021-05-01,Caraga,Contract,"28,276",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 08, ...","28,276","28,276",Not Specified,Not Specified,Science
75,Joblum,Senior Technical Consultant,Science & Technology,Research Triangle Institute-Philippine Branch,2021-05-01,Makati City,Contract,,Not Specified / In Description,Not Specified / In Description,METHOD OF SUBMITTAL Applications will be accep...,Not Specified,Not Specified,Not Specified,Not Specified,Science
76,Joblum,LABORATORY TECHNICIAN I,Science & Technology,Provincial Government of South Cotabato,2021-05-01,Soccsksargen,Contract,"16,200",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 07, ...","16,200","16,200",Not Specified,Not Specified,Science


In [49]:
SNT_df.to_csv ('JOBLUM-SNT.csv', index = False)

# Data Cleaning (JOBLUM)

In [229]:
#concatenate all df (JOBLUM)
JOBLUM_df = pd.concat([STAT_df, AGRI_df, ARCHI_df, AVI_df, BIOMED_df, BIOTECH_df, 
                       CHEMENG_df, CHEM_df, CIVILENG_df, CONSTRUCTION_df, DIAGNOSIS_df, 
                       DOCTOR_df, ELEC_df, ELECENG_df, ELECTRO_df, ELECTROENG_df, ENVI_df, 
                       ENVIENG_df, NUTRI_df, GEO_df, INDUSENG_df, IT_HARDWARE_df, IT_SYS_df, 
                       IT_SOFTWARE_df, MAINTENANCE_df, MECH_df, MECHENG_df, NURSE_df, OIL_df, 
                       OILENG_df, ENG_df, PHARMA_df, QUALITY_df, QUANTITY_df, SNT_df], 
                      ignore_index=True, sort=False)
JOBLUM_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,With Signing Bonus* | Data Visualization Senio...,Actuarial/Statistics,Accenture,2021-05-29,"Central Luzon, National Capital Reg, Calabarzo...",Not Specified,,Not Specified / In Description,Not Specified / In Description,Go through a faster and more convenient recrui...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
1,Joblum,Statistician 1,Actuarial/Statistics,Philippine National Police Region X - Government,2021-05-29,Northern Mindanao,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 11,...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
2,Joblum,Statistician I,Actuarial/Statistics,"Municipal Government of Surallah, South Cotaba...",2021-05-29,Soccsksargen,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : June 11,...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
3,Joblum,Risk Underwriter,Actuarial/Statistics,Lightspeed,2021-05-28,Pasig City,Full Time,,Not Specified / In Description,Not Specified / In Description,Perks and Benefits Participate in the Lightspe...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
4,Joblum,Statistician III,Actuarial/Statistics,Civil Service Commission - Government,2021-05-28,National Capital Reg,Not Specified,,Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 31, ...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Joblum,Aircraft Mechanic II (34-1998),Aviation,PHILIPPINE AIR FORCE - Government,2021-05-12,National Capital Reg,OJT,"18,251",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 14, ...","18,251","18,251",Not Specified,Not Specified,Mathematics
1310,Joblum,Aircraft Mechanic II (27-1998),Aviation,PHILIPPINE AIR FORCE - Government,2021-05-12,National Capital Reg,OJT,"18,251",Not Specified / In Description,Not Specified / In Description,"Deadline for accepting applications : May 14, ...","18,251","18,251",Not Specified,Not Specified,Mathematics
1311,Joblum,Service Performance Manager,Aviation,1Aviation Groundhandling Services Corporation,2021-05-11,Pasay City,OJT,,Not Specified / In Description,Not Specified / In Description,Job Summary Responsible for managing a team to...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
1312,Joblum,Line Mechanic,Aviation,Sumifru (Philippines) Corporation,2021-05-09,Davao,Full Time,,Not Specified / In Description,Not Specified / In Description,The Line Mechanic is responsible for conductin...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics


In [232]:
#drop all test job post (JOBLUM)
test_index = JOBLUM_df[JOBLUM_df['Job Title'].str.contains('Test')].index
JOBLUM_df = JOBLUM_df.drop(test_index)

In [243]:
def find_education(description):
    eduoutput = (description.replace('/', ''))
    education_list1 = ['Bachelor','Degree','BS', 'College graduate', 'University graduate', 'Graduate']
    education_list2 = ['Master', 'Post graduate']
    if any(x in eduoutput for x in education_list2):
        return "Master's Degree"
    if any(x in eduoutput for x in education_list1):
        return "Bachelor's Degree"
    else:
        return "Not Specified"

In [244]:
JOBLUM_df["Education"] = JOBLUM_df["Job Description"].apply(find_education)
JOBLUM_df

,Website,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Min Salary,Max Salary,Min Years of Work Expirience,Max Years of Work Expirience,Field
0,Joblum,With Signing Bonus* | Data Visualization Senio...,Actuarial/Statistics,Accenture,2021-05-29,"Central Luzon, National Capital Reg, Calabarzo...",Not Specified,,Bachelor's Degree,Not Specified / In Description,Go through a faster and more convenient recrui...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
1,Joblum,Statistician 1,Actuarial/Statistics,Philippine National Police Region X - Government,2021-05-29,Northern Mindanao,Not Specified,,Bachelor's Degree,Not Specified / In Description,"Deadline for accepting applications : June 11,...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
2,Joblum,Statistician I,Actuarial/Statistics,"Municipal Government of Surallah, South Cotaba...",2021-05-29,Soccsksargen,Not Specified,,Bachelor's Degree,Not Specified / In Description,"Deadline for accepting applications : June 11,...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
3,Joblum,Risk Underwriter,Actuarial/Statistics,Lightspeed,2021-05-28,Pasig City,Full Time,,Bachelor's Degree,Not Specified / In Description,Perks and Benefits Participate in the Lightspe...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
4,Joblum,Statistician III,Actuarial/Statistics,Civil Service Commission - Government,2021-05-28,National Capital Reg,Not Specified,,Bachelor's Degree,Not Specified / In Description,"Deadline for accepting applications : May 31, ...",Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Joblum,Aircraft Mechanic II (34-1998),Aviation,PHILIPPINE AIR FORCE - Government,2021-05-12,National Capital Reg,OJT,"18,251",Not Specified,Not Specified / In Description,"Deadline for accepting applications : May 14, ...","18,251","18,251",Not Specified,Not Specified,Mathematics
1310,Joblum,Aircraft Mechanic II (27-1998),Aviation,PHILIPPINE AIR FORCE - Government,2021-05-12,National Capital Reg,OJT,"18,251",Not Specified,Not Specified / In Description,"Deadline for accepting applications : May 14, ...","18,251","18,251",Not Specified,Not Specified,Mathematics
1311,Joblum,Service Performance Manager,Aviation,1Aviation Groundhandling Services Corporation,2021-05-11,Pasay City,OJT,,Bachelor's Degree,Not Specified / In Description,Job Summary Responsible for managing a team to...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics
1312,Joblum,Line Mechanic,Aviation,Sumifru (Philippines) Corporation,2021-05-09,Davao,Full Time,,Bachelor's Degree,Not Specified / In Description,The Line Mechanic is responsible for conductin...,Not Specified,Not Specified,Not Specified,Not Specified,Mathematics


In [242]:
JOBLUM_df.to_csv ('JOBLUM.csv', index = False)